In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# exercise

Build your own GPT-4 Tokenizer!

### Step 1

Write the `BasicTokenizer` class, with the following three core functions:

- `def train(self, text, vocab_size, verbose=False)`
- `def encode(self, text)`
- `def decode(self, ids)`

Train your tokenizer on whatever text you like and visualize the merged tokens. Do they look reasonable? One default test you may wish to use is the text file `tests/taylorswift.txt`.

### Step 2

Convert your `BasicTokenizer` into a `RegexTokenizer`, which takes a regex pattern and splits the text exactly as GPT-4 would. Process the parts separately as before, then concatenate the results. Retrain your tokenizer and compare the results before and after. You should see that you will now have no tokens that go across categories (numbers, letters, punctuation, more than one whitespace). Use the GPT-4 pattern:

```
GPT4_SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""
```


### Step 3

You're now ready to load the merges from the GPT-4 tokenizer and show that your tokenizer produces the identical results for both `encode` and `decode`, matching [tiktoken](https://github.com/openai/tiktoken).

```
# match this
import tiktoken
enc = tiktoken.get_encoding("cl100k_base") # this is the GPT-4 tokenizer
ids = enc.encode("hello world!!!? (안녕하세요!) lol123 😉")
text = enc.decode(ids) # get the same text back
```

Unfortunately, you will run into two issues:

1. It is not trivial to recover the raw merges from the GPT-4 tokenizer. You can easily recover what we call `vocab` here, and what they call and store under `enc._mergeable_ranks`. Feel free to copy paste the `recover_merges` function in `minbpe/gpt4.py`, which takes these ranks and returns the raw merges. If you wish to know how this function works, read [this](https://github.com/openai/tiktoken/issues/60) and [this](https://github.com/karpathy/minbpe/issues/11#issuecomment-1950805306). Basically, under some conditions it is enough to only store the parent nodes (and their rank) and get rid of the precise details of which children merged up to any parent.
2. Second, the GPT-4 tokenizer for some reason permutes its raw bytes. It stores this permutation in the first 256 elements of the mergeable ranks, so you can recover this byte shuffle relatively simply as `byte_shuffle = {i: enc._mergeable_ranks[bytes([i])] for i in range(256)}`. In both your encode and decode, you'll have to shuffle bytes around accordingly. If you're stuck, reference the `minbpe/gpt4.py` file for hints.

### Step 4

(Optional, irritating, not obviously useful) Add the ability to handle special tokens. You'll then be able to match the output of tiktoken even when special tokens are present, e.g.:

```
import tiktoken
enc = tiktoken.get_encoding("cl100k_base") # this is the GPT-4 tokenizer
ids = enc.encode("<|endoftext|>hello world", allowed_special="all")
```

Without `allowed_special` tiktoken will error.

### Step 5

If you've made it this far, you're now a pro at LLM Tokenization! Sadly, you're not exactly done yet because a lot of LLMs outside of OpenAI (e.g. Llama, Mistral) use [sentencepiece](https://github.com/google/sentencepiece) instead. Primary difference being that sentencepiece runs BPE directly on Unicode code points instead of on UTF-8 encoded bytes. Feel free to explore sentencepiece on your own (good luck, it's not too pretty), and stretch goal if you really experience and suffer from the burden of time, re-write your BPE to be on Unicode code points and match the Llama 2 tokenizer.

In [2]:
taylorswift_txt = '''Copy paste of the Wikipedia article on Taylor Swift, as of Feb 16, 2024.
---

Main menu

WikipediaThe Free Encyclopedia

Search
Create account
Log in

Personal tools
Contents  hide
(Top)
Life and career
Toggle Life and career subsection
Artistry
Toggle Artistry subsection
Accolades and achievements
Cultural status
Toggle Cultural status subsection
Wealth
Toggle Wealth subsection
Discography
Filmography
Tours
See also
Footnotes
References
Toggle References subsection
External links
Taylor Swift

136 languages
Article
Talk
Read
View source
View history

Tools
 Featured article
Page semi-protected
From Wikipedia, the free encyclopedia
For the album, see Taylor Swift (album).
Taylor Swift
Portrait of Taylor Swift in a cocktail dress
Swift at the 2023 MTV Video Music Awards
Born	Taylor Alison Swift
December 13, 1989 (age 34)
West Reading, Pennsylvania, US
Occupations
Singer-songwriter producer director businesswoman actress
Years active	2004–present
Works
Albumssinglessongsvideosperformances
Relatives
Austin Swift (brother)
Marjorie Finlay (grandmother)
Awards	Full list
Musical career
Origin	Nashville, Tennessee, US
Genres
Pop country folk rock alternative
Instruments
Vocals guitar banjo piano ukulele
Labels
RCA Republic Big Machine
Website	www.taylorswift.com Edit this at Wikidata
Signature

Taylor Alison Swift (born December 13, 1989) is an American singer-songwriter. Her versatile artistry, songwriting, and entrepreneurship have influenced the music industry, popular culture, and politics, and her life is a subject of widespread media coverage.
Swift began professional songwriting at 14 and signed with Big Machine Records in 2005 to become a country singer. She released six studio albums under the label, four of them to country radio, starting with Taylor Swift (2006). Her next, Fearless (2008), explored country pop, and its singles "Love Story" and "You Belong with Me" catapulted her to mainstream fame. Speak Now (2010) infused rock influences, while Red (2012) experimented with electronic elements and featured Swift's first Billboard Hot 100 number-one song, "We Are Never Ever Getting Back Together". She departed from her country image with 1989 (2014), a synth-pop album supported by the chart-topping songs "Shake It Off", "Blank Space", and "Bad Blood". Media scrutiny inspired the hip-hop-influenced Reputation (2017) and its number-one single "Look What You Made Me Do".
After signing with Republic Records in 2018, Swift released the eclectic pop album Lover (2019) and the autobiographical documentary Miss Americana (2020). She explored indie folk styles on the 2020 albums Folklore and Evermore, subdued electropop on Midnights (2022), and re-recorded four albums subtitled Taylor's Version after a dispute with Big Machine. These albums spawned the number-one songs "Cruel Summer", "Cardigan", "Willow", "Anti-Hero", "All Too Well", and "Is It Over Now?". Her Eras Tour (2023–2024) and its accompanying concert film became the highest-grossing tour and concert film of all time, respectively. Swift has directed several music videos and films such as Folklore: The Long Pond Studio Sessions (2020) and All Too Well: The Short Film (2021).
One of the world's best-selling musicians, with over 200 million records sold as of 2019, Swift has been named Global Recording Artist of the Year three times by the International Federation of the Phonographic Industry, whereas six of her albums have opened with over a million sales in a week. She is the highest-grossing female touring act, the most-streamed woman on Spotify and Apple Music, and the first billionaire with music as the main source of income. The 2023 Time Person of the Year, Swift has appeared on lists such as Rolling Stone's 100 Greatest Songwriters of All Time, Billboard's Greatest of All Time Artists, and Forbes' World's 100 Most Powerful Women. Her accolades include 14 Grammy Awards (featuring a record four Album of the Year wins), a Primetime Emmy Award, 40 American Music Awards, 40 Billboard Music Awards, and 23 MTV Video Music Awards.
Life and career
Early life

Swift's childhood home in Wyomissing, Pennsylvania
Taylor Alison Swift was born on December 13, 1989,[1] in West Reading, Pennsylvania.[2] She is named after singer-songwriter James Taylor.[3] Her father, Scott Kingsley Swift, is a former stockbroker for Merrill Lynch[4] and her mother, Andrea Gardner Swift (née Finlay), worked for a time as a mutual fund marketing executive.[5] Taylor has a younger brother, actor Austin Swift.[6]
Swift's mother is of Scottish and German descent, and her father is of Scottish and English descent with distant Italian ancestry.[7][8] Swift's paternal great-great-grandfather, Charles Carmine Antonio Baldi, was an Italian immigrant entrepreneur and community leader who opened several businesses in Philadelphia in the 1800s.[9][10][8] Her maternal grandmother, Marjorie (née Moehlenkamp) Finlay, was an opera singer.[11]
Swift spent her early years on a Christmas tree farm that her father had purchased from one of his clients.[12][13] She is a Christian.[14] She attended preschool and kindergarten at Alvernia Montessori School, run by Bernardine Franciscan sisters,[15] before transferring to the Wyndcroft School.[16] The family moved to a rented house in Wyomissing, Pennsylvania,[17] where Swift attended Wyomissing Area Junior/Senior High School.[18]
At age nine, Swift became interested in musical theater and performed in four Berks Youth Theatre Academy productions.[19] She also traveled regularly to New York City for vocal and acting lessons.[20] Swift later shifted her focus toward country music, inspired by Shania Twain's songs, which made her "want to just run around the block four times and daydream about everything".[21] She spent weekends performing at local festivals and events.[22][23] After watching a documentary about Faith Hill, Swift felt she needed to move to Nashville, Tennessee, to pursue a career in music.[24] She traveled there with her mother at age eleven to visit record labels and submitted demo tapes of Dolly Parton and Dixie Chicks karaoke covers.[25] She was rejected, however, because "everyone in that town wanted to do what I wanted to do. So, I kept thinking to myself, I need to figure out a way to be different."[26] She spent summers in Stone Harbor, New Jersey until she was 14 years old, performing in a local coffee shop.[27][28]
When Swift was around 12 years old, musician Ronnie Cremer taught her to play guitar. Cremer helped with her first efforts as a songwriter, leading her to write "Lucky You".[29] In 2003, Swift and her parents started working with New York–based talent manager Dan Dymtrow. With his help, Swift modeled for Abercrombie & Fitch as part of their "Rising Stars" campaign, had an original song included on a Maybelline compilation CD, and met with major record labels.[30] After performing original songs at an RCA Records showcase, Swift, then 13 years old, was given an artist development deal and began making frequent trips to Nashville with her mother.[31][32][33] To help Swift break into the country music scene, her father transferred to Merrill Lynch's Nashville office when she was 14 years old, and the family relocated to Hendersonville, Tennessee.[12][34] Swift attended Hendersonville High School[35] before transferring to Aaron Academy after two years, which better accommodated her touring schedule through homeschooling. She graduated one year early.[36][37]
2004–2008: Career beginnings and first album
In Nashville, Swift worked with experienced Music Row songwriters such as Troy Verges, Brett Beavers, Brett James, Mac McAnally, and the Warren Brothers[38][39] and formed a lasting working relationship with Liz Rose.[40] They began meeting for two-hour writing sessions every Tuesday afternoon after school.[41] Rose called the sessions "some of the easiest I've ever done. Basically, I was just her editor. She'd write about what happened in school that day. She had such a clear vision of what she was trying to say. And she'd come in with the most incredible hooks." Swift became the youngest artist signed by the Sony/ATV Tree publishing house,[42] but left then BMG-owned RCA Records (later bought by Sony Music) at the age of 14 due to the label's lack of care and them "cut[ting] other people's stuff". She was also concerned that development deals can shelve artists[33][23] and recalled: "I genuinely felt that I was running out of time. I wanted to capture these years of my life on an album while they still represented what I was going through."[43]
Taylor Swift singing on a microphone and playing a guitar
Swift opening for Brad Paisley in 2007. To promote her first album, she opened tours for other country musicians in 2007 and 2008.[44]
At an industry showcase at Nashville's Bluebird Cafe in 2005, Swift caught the attention of Scott Borchetta, a DreamWorks Records executive who was preparing to form an independent record label, Big Machine Records. She had first met Borchetta in 2004.[45] She was one of Big Machine's first signings,[33] and her father purchased a three-percent stake in the company for an estimated $120,000.[46][47] She began working on her eponymous debut album with Nathan Chapman.[23] Swift wrote or co-wrote all album tracks, and co-writers included Rose, Robert Ellis Orrall, Brian Maher, and Angelo Petraglia.[48] Taylor Swift was released on October 24, 2006.[49] Country Weekly critic Chris Neal deemed Swift better than previous aspiring teenage country singers because of her "honesty, intelligence and idealism".[50] The album peaked at number five on the US Billboard 200, on which it spent 157 weeks—the longest stay on the chart by any release in the US in the 2000s decade.[51] Swift became the first female country music artist to write or co-write every track on a US platinum-certified debut album.[52]
Big Machine Records was still in its infancy during the June 2006 release of the lead single, "Tim McGraw", which Swift and her mother helped promote by packaging and sending copies of the CD single to country radio stations. As there was not enough furniture at the label yet, they would sit on the floor to do so.[53] She spent much of 2006 promoting Taylor Swift with a radio tour and television appearances; she opened for Rascal Flatts on select dates during their 2006 tour,[54] as a replacement for Eric Church.[55] Borchetta said that although record industry peers initially disapproved of his signing a 15-year-old singer-songwriter, Swift tapped into a previously unknown market—teenage girls who listen to country music.[53][12]
Following "Tim McGraw", four more singles were released throughout 2007 and 2008: "Teardrops on My Guitar", "Our Song", "Picture to Burn" and "Should've Said No". All appeared on Billboard's Hot Country Songs, with "Our Song" and "Should've Said No" reaching number one. With "Our Song", Swift became the youngest person to single-handedly write and sing a number-one song on the chart.[56] "Teardrops on My Guitar" reached number thirteen on the US Billboard Hot 100.[57] Swift also released two EPs, The Taylor Swift Holiday Collection in October 2007 and Beautiful Eyes in July 2008.[58][59] She promoted her debut album extensively as the opening act for other country musicians' tours in 2006 and 2007, including those by George Strait,[60] Brad Paisley,[61] and Tim McGraw and Faith Hill.[62]
Swift won multiple accolades for Taylor Swift. She was one of the recipients of the Nashville Songwriters Association's Songwriter/Artist of the Year in 2007, becoming the youngest person given the title.[63] She also won the Country Music Association's Horizon Award for Best New Artist,[64] the Academy of Country Music Awards' Top New Female Vocalist,[65] and the American Music Awards' Favorite Country Female Artist honor.[66] She was also nominated for Best New Artist at the 50th Annual Grammy Awards.[67] In 2008, she opened for Rascal Flatts again[68] and briefly dated the singer Joe Jonas.[69][70]
2008–2010: Fearless
Taylor Swift in 2009
Swift at the 2009 premiere of Hannah Montana: The Movie. She had a cameo appearance in the film and wrote two songs for its soundtrack.[71][72]
Swift's second studio album, Fearless, was released on November 11, 2008, in North America,[73] and in March 2009 in other markets.[74] Critics lauded Swift's honest and vulnerable songwriting in contrast to other teenage singers.[75] Five singles were released in 2008–2009: "Love Story", "White Horse", "You Belong with Me", "Fifteen", and "Fearless". The first single peaked at number four on the Billboard Hot 100 and number one in Australia.[57][76] It was the first country song to top Billboard's Pop Songs chart.[77] "You Belong with Me" was the album's highest-charting single on the Billboard Hot 100, peaking at number two,[78] and was the first country song to top Billboard's all-genre Radio Songs chart.[79] All five singles were Hot Country Songs top-10 entries, with "Love Story" and "You Belong with Me" topping the chart.[80] Fearless became her first number-one album on the Billboard 200 and 2009's top-selling album in the US.[81] The Fearless Tour, Swift's first headlining concert tour, grossed over $63 million.[82] Journey to Fearless, a documentary miniseries, aired on television and was later released on DVD and Blu-ray.[83] Swift performed as a supporting act for Keith Urban's Escape Together World Tour in 2009.[84]
In 2009, the music video for "You Belong with Me" was named Best Female Video at the 2009 MTV Video Music Awards.[85] Her acceptance speech was interrupted by rapper Kanye West,[86] an incident that became the subject of controversy, widespread media attention and Internet memes.[87] That year she won five American Music Awards, including Artist of the Year and Favorite Country Album.[88] Billboard named her 2009's Artist of the Year.[89] She won Video of the Year and Female Video of the Year for "Love Story" at the 2009 CMT Music Awards, where she made a parody video of the song with rapper T-Pain called "Thug Story".[90] At the 52nd Annual Grammy Awards, Fearless was named Album of the Year and Best Country Album, and "White Horse" won Best Country Song and Best Female Country Vocal Performance. Swift was the youngest artist to win Album of the Year.[note 1] At the 2009 Country Music Association Awards, Swift won Album of the Year for Fearless and was named Entertainer of the Year, the youngest person to win the honor.[93]
Swift featured on John Mayer's single "Half of My Heart" and Boys Like Girls' single "Two Is Better Than One", the latter of which she co-wrote.[94][95] She co-wrote and recorded "Best Days of Your Life" with Kellie Pickler,[96] and wrote two songs for the Hannah Montana: The Movie soundtrack—"You'll Always Find Your Way Back Home" and "Crazier".[72] She contributed two songs to the Valentine's Day soundtrack, including the single "Today Was a Fairytale", which was her first number-one on the Canadian Hot 100 and peaked at number two on the US Hot 100.[97][98] While shooting her film debut Valentine's Day in October 2009, Swift dated co-star Taylor Lautner.[99] In 2009, she made her television debut as a rebellious teenager in an CSI: Crime Scene Investigation episode,[100] and she hosted and performed as the musical guest on Saturday Night Live; she was the first host ever to write their own opening monologue.[101][102]
2010–2014: Speak Now and Red
Swift singing into a mic while playing a banjo
Swift performing at the Speak Now World Tour in 2012
In August 2010, Swift released "Mine", the lead single from her third studio album, Speak Now. The single entered the Hot 100 at number three.[103] Swift wrote the album alone and co-produced every track.[104] The album was released on October 25, 2010,[105] opening atop the Billboard 200 with over one million copies sold.[106] It became the fastest-selling digital album by a female artist, with 278,000 downloads in a week.[107] Critics appreciated Swift's grown-up perspectives:[108] Rob Sheffield of Rolling Stone wrote, "in a mere four years, the 20-year-old Nashville firecracker has put her name on three dozen or so of the smartest songs released by anyone in pop, rock or country."[109] "Back to December", "Mean", "The Story of Us", "Sparks Fly", and "Ours" became subsequent singles, with the latter two reaching number one on the Hot Country Songs[80] and the first two peaking in the top ten in Canada.[98] She dated actor Jake Gyllenhaal in 2010.[110]
At the 54th Annual Grammy Awards in 2012, Swift won Best Country Song and Best Country Solo Performance for "Mean", which she performed during the ceremony.[111] Swift won other awards for Speak Now, including Songwriter/Artist of the Year by the Nashville Songwriters Association (2010 and 2011),[112][113] Woman of the Year by Billboard (2011),[114] and Entertainer of the Year by the Academy of Country Music (2011 and 2012)[115] and the Country Music Association in 2011.[116] At the American Music Awards of 2011, Swift won Artist of the Year and Favorite Country Album.[117] Rolling Stone named Speak Now amongst its "50 Best Female Albums of All Time" (2012), writing: "She might get played on the country station, but she's one of the few genuine rock stars we've got these days, with a flawless ear for what makes a song click."[118]
The Speak Now World Tour ran from February 2011 to March 2012 and grossed over $123 million,[119] followed up by the live album, Speak Now World Tour: Live.[120] She contributed two original songs to The Hunger Games soundtrack album: "Eyes Open" and "Safe & Sound", co-written and recorded with the Civil Wars and T-Bone Burnett. "Safe & Sound" won the Grammy Award for Best Song Written for Visual Media and was nominated for the Golden Globe Award for Best Original Song.[121][122] Swift featured on B.o.B's single "Both of Us", released in May 2012.[123] She dated Conor Kennedy that year.[124]
Taylor Swift on the Red Tour
Swift on the Red Tour (2013)
In August 2012, Swift released "We Are Never Ever Getting Back Together", the lead single from her fourth studio album, Red. It became her first number one single in the US and New Zealand,[125][126] and became the fastest-selling single in digital history.[127] Other singles from the album were "Begin Again", "I Knew You Were Trouble", "22", "Everything Has Changed", "The Last Time", and "Red". "I Knew You Were Trouble" reached the top five on charts in Australia, Canada, Denmark, Ireland, New Zealand, the UK and the US.[128] "Begin Again", "22", and "Red" reached the top 20 in the US.[57] On Red, released on October 22, 2012,[129] Swift worked with Chapman and Rose, as well as the new producers Max Martin and Shellback.[130] It incorporated many pop and rock styles such as heartland rock, dubstep and dance-pop.[131] Randall Roberts of Los Angeles Times said Swift "strives for something much more grand and accomplished" with Red.[132] It opened at number one on the Billboard 200 with 1.21 million sales.[133] Red was Swift's first number-one album in the UK.[134] It earned several accolades, including four nominations at the 56th Annual Grammy Awards (2014).[135] Swift received American Music Awards for Best Female Country Artist in 2012, Artist of the Year in 2013,[136][137] and the Nashville Songwriters Association's Songwriter/Artist Award for the fifth and sixth consecutive years.[138] The Red Tour ran from March 2013 to June 2014 and grossed over $150 million, becoming the highest-grossing country tour ever.[139] Swift was honored with the Pinnacle Award, making her the second recipient of the accolade after Garth Brooks.[140] During this time, she briefly dated the English singer Harry Styles.[141]
In 2013, Swift recorded "Sweeter than Fiction", a song she wrote and produced with Jack Antonoff for the One Chance soundtrack. The song received a Best Original Song nomination at the 71st Golden Globe Awards.[142] She provided guest vocals for Tim McGraw's song "Highway Don't Care", also featuring Keith Urban.[143] Swift performed "As Tears Go By" with the Rolling Stones in Chicago, Illinois, as part of the band's 50 & Counting tour,[144] and joined Florida Georgia Line at their set at the 2013 Country Radio Seminar to sing "Cruise".[145] Swift voiced Audrey in the animated film The Lorax (2012),[146] made a cameo in the sitcom New Girl (2013),[147] and had a supporting role in the dystopian film The Giver (2014).[148]
2014–2018: 1989 and Reputation
Swift performing on a mic, dressed in a blue skirt
Swift at the 1989 World Tour, the highest-grossing tour of 2015
In March 2014, Swift began living in New York City.[note 2] She hired Tree Paine as her publicist[151] and worked on her fifth studio album, 1989, with the producers Jack Antonoff, Max Martin, Shellback, Imogen Heap, Ryan Tedder, and Ali Payami.[152] She promoted the album extensively, including inviting fans to secret album-listening sessions.[153] 1989 was released on October 27, 2014, and opened atop the Billboard 200 with 1.28 million copies sold.[154] Its singles "Shake It Off", "Blank Space" and "Bad Blood" reached number one in Australia, Canada and the US, the first two making Swift the first woman to replace herself at the Hot 100 top spot;[155] other singles include "Style", "Wildest Dreams", "Out of the Woods" and "New Romantics".[156] The 1989 World Tour (2015) was the highest-grossing tour of the year with $250 million in total revenue.[157]
Prior to 1989's release, Swift stressed the importance of albums to artists and fans.[158] In November 2014, she removed her entire catalog from Spotify, arguing that its ad-supported, free service undermined the premium service, which provides higher royalties for songwriters.[159] In a June 2015 open letter, Swift criticized Apple Music for not offering royalties to artists during the streaming service's free three-month trial period and stated that she would pull 1989 from the catalog.[160] The following day, Apple Inc. announced that it would pay artists during the free trial period,[161] and Swift agreed to let 1989 on the streaming service.[162] She then returned her entire catalog plus 1989 to Spotify, Amazon Music and Google Play and other digital streaming platforms in June 2017.[163] Swift was named Billboard's Woman of the Year in 2014, becoming the first artist to win the award twice.[164] At the 2014 American Music Awards, Swift received the inaugural Dick Clark Award for Excellence.[165] On her 25th birthday in 2014, the Grammy Museum at L.A. Live opened an exhibit in her honor in Los Angeles that ran until October 4, 2015, and broke museum attendance records.[166][167] In 2015, Swift won the Brit Award for International Female Solo Artist.[168] The video for "Bad Blood" won Video of the Year and Best Collaboration at the 2015 MTV Video Music Awards.[169] At the 58th Grammy Awards (2016), 1989 won Album of the Year and Best Pop Vocal Album, making Swift the first woman and fifth act overall to win Album of the Year twice.[170]
Swift wearing a sparkling blazer singing on a mic
Swift on her Reputation Stadium Tour (2018), the highest-grossing North American tour ever
Swift dated the Scottish DJ Calvin Harris from March 2015 to June 2016.[171] They co-wrote the song "This Is What You Came For", featuring vocals from the Barbadian singer Rihanna; Swift was initially credited under the pseudonym Nils Sjöberg.[172] In April 2016, Swift criticized the lyrics of Kanye West's single "Famous", in which he sings "I made that bitch famous" in reference to his interruption of her acceptance speech at the 2009 MTV Video Music Awards. West claimed he had received her approval for the line, and his then-wife Kim Kardashian released video clips of Swift and West discussing the single amicably over the phone; a full recording leaked in 2020 established that West did not disclose that he would call her a "bitch".[173][174]
After briefly dating the English actor Tom Hiddleston,[175] Swift entered a six-year relationship with the English actor Joe Alwyn in September 2016.[176][177][178] She wrote the song "Better Man" for the band Little Big Town, which earned her the Song of the Year award at the 51st CMA Awards.[179] Swift and English singer Zayn Malik released the joint single "I Don't Wanna Live Forever" for Fifty Shades Darker: Original Motion Picture Soundtrack (2017). The song reached number two in the US.[180]
In August 2017, Swift successfully countersued David Mueller, a former radio jockey for KYGO-FM, who sued her for damages from loss of employment. Four years earlier, she informed Mueller's bosses that he had sexually assaulted her by groping her at an event.[181] Also that month, after a one-year hiatus from the spotlight, Swift cleared her social media accounts and released "Look What You Made Me Do" as the lead single from her sixth album, Reputation.[182][183] The single was Swift's first UK number-one single.[184] It topped charts in Australia, Ireland, New Zealand, and the US.[185] Reputation, released on November 10, 2017,[186] incorporated electropop, hip hop, R&B, and EDM.[187] Reviews praised Swift's mature artistry, but some denounced the themes of fame and gossip.[188] The album opened atop the Billboard 200 with 1.21 million US sales[189] and topped the charts in the UK, Australia, and Canada.[190] Its singles "...Ready for It?", "End Game" (featuring Ed Sheeran and Future), and "Delicate" were released to pop radio.[191] Reputation was nominated for a Grammy Award for Best Pop Vocal Album.[192] Swift featured on the country duo Sugarland's "Babe" (2018).[193]
At the 2018 American Music Awards, Swift won four awards, which made her accumulate 23 trophies in total and become the AMAs' most awarded female musician, surpassing Whitney Houston.[194] The same year, she embarked on her Reputation Stadium Tour,[195] which became the highest-grossing North American concert tour in history and grossed $345.7 million worldwide.[196]
2018–2020: Lover, Folklore, and Evermore
In November 2018, she signed a new deal with the Universal Music Group; her subsequent releases were promoted by Republic Records. Swift said the contract included a provision for her to maintain ownership of her masters. In addition, in the event that Universal sold any part of its stake in Spotify, it agreed to distribute a non-recoupable portion of the proceeds among its artists.[197] Vox called it a huge commitment from Universal, which was "far from assured" until Swift intervened.[198]
A portrait of Swift
Swift at the American Music Awards of 2019, where she was named Artist of the Decade
Swift's first album with Republic Records, Lover, was released on August 23, 2019.[199] Besides Antonoff, she worked with Louis Bell, Frank Dukes, and Joel Little.[200] Lover was her sixth consecutive album to sell more than 500,000 US copies in one week.[201] Critics commended the album's free-spirited mood and emotional intimacy.[202][203] The singles "Me!" and "You Need to Calm Down" both peaked at number two on the Hot 100,[204] and other singles were the top-10 single "Lover", the top-40 single "The Man",[57] and the 2023 resurgent success, chart topper "Cruel Summer".[205] Lover was the world's best-selling album by a solo artist of 2019,[206] and along with its singles earned nominations at the 62nd Annual Grammy Awards in 2020.[207] At the 2019 MTV Video Music Awards, "Me!" won Best Visual Effects, and "You Need to Calm Down" won Video of the Year and Video for Good. Swift was the first female and second artist overall to win Video of the Year for a video that they directed.[208]
While promoting Lover, Swift became embroiled in a public dispute with the talent manager Scooter Braun and Big Machine over the purchase of the masters of her back catalog.[209][210] Swift said she had been trying to buy the masters, but Big Machine would only allow her to do so if she exchanged one new album for each older one under a new contract, which she refused to sign.[209][211] Swift began re-recording her back catalog in November 2020.[212] Besides music, she played Bombalurina in the film adaptation of Andrew Lloyd Webber's musical Cats (2019), for which she co-wrote and recorded the Golden Globe-nominated original song "Beautiful Ghosts".[213][214] Critics panned the film but praised Swift's performance.[215] The documentary Miss Americana, which chronicled parts of Swift's life and career, premiered at the 2020 Sundance Film Festival.[216] Swift signed a global publishing deal with Universal Music Publishing Group in February 2020 after her 16-year contract with Sony/ATV expired.[217]
Amidst the COVID-19 pandemic in 2020, Swift surprise-released two "sister albums" that she recorded with Antonoff and Aaron Dessner: Folklore on July 24, and Evermore on December 11.[218][219] Alwyn co-wrote and co-produced a few songs under the pseudonym William Bowery.[220] Both explore indie folk with a more muted production compared to her previous upbeat pop songs[221][222] and earned Swift widespread critical acclaim and artistic recognition.[223][224] Each album was supported by three singles catering to US pop, country, and triple A radio formats. The singles were "Cardigan", "Betty", and "Exile" from Folklore, and "Willow", "No Body, No Crime", and "Coney Island" from Evermore.[225] Folklore made Swift the first woman to win the Grammy Award for Album of the Year three times at the 63rd Annual Grammy Awards[226] and was the best-selling album of 2020 in the US.[227] Swift became the first artist to debut a US number-one album and a number-one song at the same time with Folklore's "Cardigan".[228] At the 2020 American Music Awards, she won three awards, including Artist of the Year for a record third consecutive time.[229] According to Billboard, she was 2020's highest-paid musician in the US and highest-paid solo musician worldwide.[230]
2020–2023: Re-recordings and Midnights

Swift performing in 2022
Following the masters dispute, Swift released re-recordings of her first six studio albums, beginning with Fearless (Taylor's Version) and Red (Taylor's Version) in April and November 2021, respectively. Both peaked atop the Billboard 200,[231] and the former was the first re-recorded album to do so.[232] Fearless (Taylor's Version) was preceded by "Love Story (Taylor's Version)", which made her the second artist after Dolly Parton to have both the original and re-recorded versions of a song reach number one on the Hot Country Songs chart.[233] Red (Taylor's Version) was supported by "All Too Well (10 Minute Version)", which became the longest song in history to top the Hot 100.[234] The song was accompanied by a short film, which won a Grammy Award for Best Music Video[235] and Swift's record third MTV Video Music Award for Video of the Year.[236]
Swift's tenth studio album, Midnights, was released on October 21, 2022.[237] Characterized by a restrained electropop[238][239] and synth-pop[240] sound, the album was dubbed by Rolling Stone critics as an instant classic.[241][242] The album was her fifth to open atop the Billboard 200 with first-week sales of over one million copies and broke various sales and streaming records,[243] including the most single-day streams and most single-week streams on Spotify.[244] Its tracks, led by single "Anti-Hero", monopolized the top 10 of the Hot 100, making Swift the first artist to do so.[245] Two other singles, "Lavender Haze" and "Karma", peaked at number two on the Hot 100.[246] Swift won nine awards at the 2023 MTV Video Music Awards, including Video of the Year ("Anti-Hero") for a record fourth time.[247] At the 66th Annual Grammy Awards, she received Best Pop Vocal Album, and her fourth Album of the Year—the most for any artist.[248]
Swift released the third re-recorded album, Speak Now (Taylor's Version), on July 7, 2023, becoming the woman with the most number-one albums (12) in Billboard 200 history, surpassing Barbra Streisand.[249] 1989 (Taylor's Version), released on October 27, 2023, became Swift's record-extending sixth album to sell one million copies in a single week in the US and surpassed Midnights for her career's largest album sales week.[250] Its single "Is It Over Now?" debuted atop the Billboard Hot 100. Swift was 2023's most streamed artist on Spotify,[251] Apple Music,[252] and Amazon Music;[253] the first act to place number one on the year-end Billboard top artists list in three different decades (2009, 2015 and 2023);[254] and the first living artist to simultaneously chart five albums in the top 10 of the Billboard 200.[255] She had five out of the 10 best-selling albums of 2023 in the United States, a record since Luminate began tracking US music sales in 1991.[256][257]
Beyond her albums, Swift featured on five songs from 2021 to 2023: "Renegade" and "Birch" by Big Red Machine,[258] a remix of "Gasoline" by Haim,[259] "The Joker and the Queen" by Ed Sheeran,[260] and "The Alcott" by the National.[261] For the soundtrack of the 2022 film Where the Crawdads Sing, she recorded "Carolina", which received nominations for Best Original Song at the Golden Globes and Best Song Written for Visual Media at the Grammy Awards.[262] Outside of music, Swift had a supporting role in the 2022 period comedy film Amsterdam and has signed to direct an upcoming feature film for Searchlight Pictures.[263][264]
2023–present: The Eras Tour and The Tortured Poets Department
Swift singing into a mic
Swift on the Eras Tour in 2023
In March 2023, Swift embarked on the Eras Tour, a retrospective tour covering all her studio albums. Media outlets extensively covered the tour's cultural and economic impact,[265] and its US leg broke the record for the most tickets sold in a day.[245] Ticketmaster received public and political criticisms for mishandling the tour's ticket sales.[266] The Eras Tour became the highest-grossing tour in history, collecting over $1 billion.[267][268] Its concert film, released to theaters worldwide on October 13, 2023, grossed over $250 million to become the highest-grossing concert film, and was nominated for the Golden Globe Award for Cinematic and Box Office Achievement.[269][270] Swift's music releases, touring, and related activities culminated in an unprecedented height of popularity post-pandemic.[271] Music Business Worldwide remarked that she entered a "new stratosphere of global career success" in 2023.[272]
Swift began dating Kansas City Chiefs' tight end Travis Kelce in 2023.[273] In January 2024, AI-generated fake pornographic images portraying Swift were posted to X (formerly Twitter) and spread to other social media platforms, spurring criticism and demands for legal reform.[274][275] At the 66th Grammy Awards, Swift announced her eleventh studio album, The Tortured Poets Department, set for release on April 19, 2024.[248][276]
Artistry
Influences
One of Swift's earliest memories of music is listening to her maternal grandmother, Marjorie Finlay, sing in church.[5] As a child, she enjoyed Disney film soundtracks: "My parents noticed that, once I had run out of words, I would just make up my own."[277] Swift said she owes her confidence and "fascination with writing and storytelling" to her mother, who helped her prepare for class presentations as a child.[278][279]
Swift was drawn to the storytelling aspect of country music,[280] which was introduced to the genre by female country artists of the 1990s: Shania Twain, Faith Hill, and the Dixie Chicks.[281][282] Twain, both as a songwriter and performer, was her biggest musical influence.[283] Hill was Swift's childhood role model, and she would often imitate her.[284] She admired the Chicks' defiant attitude and the way they played their instruments,[285] and was also influenced by older country stars like Patsy Cline, Loretta Lynn, Tammy Wynette, and Dolly Parton,[22] the last of whom she believes is exemplary to female songwriters.[114] As a songwriter, Swift was influenced by Joni Mitchell's emotional and autobiographical lyrics, highlighting Mitchell's 1971 album Blue as a favorite "because it explores somebody's soul so deeply".[286] She also spoke of influence from 1990s songwriters such as Melissa Etheridge, Sarah McLachlan, and Alanis Morissette,[287][288] and alt-country artists like Patty Griffin[289] and Lori McKenna.[290]
Various pop and rock artists have also influenced Swift. She lists Paul McCartney, Bruce Springsteen, Emmylou Harris, and Kris Kristofferson as her career role models.[12][291] 1989 was influenced by some of her favorite 1980s pop acts, including Peter Gabriel, Annie Lennox, Phil Collins, and Madonna.[292][293] She also cited Keith Urban's musical style and Fall Out Boy's lyrics as major influences.[294][295]
Genres
"If there's one thing that Swift has proven throughout her career, it's that she refuses to be put in a box. Her ever-evolving sound took her from country darling to pop phenom to folk's newest raconteur."
—The Recording Academy, 2021[296]
Swift is known for venturing into various music genres and undergoing artistic reinventions,[297][264] having been described as a "music chameleon".[298][299] She self-identified as a country musician until 2012, when she released her fourth studio album, Red.[300] Her albums were promoted to country radio, but music critics noted wide-ranging styles of pop and rock[301][302] and said that the melodies of her songs were rooted in pop, and the country music elements were limited to instruments such as banjo, mandolin, and fiddle, and her slight twang.[303][304] Some commented that her country music identity was an indicator of her narrative songwriting rather than musical style.[305][306] Although the Nashville music industry was receptive of Swift's status as a country musician, critics accused her of abandoning her roots in favor of crossover success in mainstream pop.[307][308] Red's eclectic pop, rock, and electronic styles intensified the critical debate, to which Swift responded, "I leave the genre labeling to other people."[309]
Music journalist Jody Rosen commented that by originating her musical career in Nashville, Swift made a "bait-and-switch maneuver, planting roots in loamy country soil, then pivoting to pop".[310] She abandoned her country music identity in 2014 with the release of her synth-pop fifth studio album, 1989. Swift described it as her first "documented, official pop album".[311] Her subsequent albums Reputation (2017) and Lover (2019) have an upbeat pop production; the former incorporates hip hop, trap, and EDM elements.[312][313][314] Midnights (2022), on the other hand, is distinguished by a more experimental, "subdued and amorphous pop sound".[315][316] Although reviews of Swift's pop albums were generally positive, some critics lamented that the pop music production indicated Swift's pursuit of mainstream success, eroding her authenticity as a songwriter nurtured by her country music background—a criticism that has been retrospectively described as rockist.[317][318] Musicologist Nate Sloan remarked that Swift's pop music transition was rather motivated by her need to expand her artistry.[319] Swift eschewed mainstream pop in favor of alternative, folk and indie rock styles with her 2020 studio albums Folklore and Evermore.[320][321] Clash said her career "has always been one of transcendence and covert boundary-pushing", reaching a point at which "Taylor Swift is just Taylor Swift", not defined by any genre.[322]
Voice

"Cardigan"
Duration: 22 seconds.0:22
Swift uses her lower register in "Cardigan" (2020).[323]
"Lavender Haze"
Duration: 18 seconds.0:18
"Lavender Haze" (2022) features Swift's falsetto vocals in the refrain.[324]
Problems playing these files? See media help.
Swift possesses a mezzo-soprano vocal range,[325] and a generally soft but versatile timbre.[326][327] As a country singer, her vocals were criticized by some as weak and strained compared to those of her contemporaries.[328] Swift admitted her vocal ability often concerned her in her early career and has worked hard to improve.[329] Reviews of her vocals remained mixed after she transitioned to pop music with 1989; critics complained that she lacked proper technique but appreciated her usage of her voice to communicate her feelings to the audience, prioritizing "intimacy over power and nuance".[330] They also praised her for refraining from correcting her pitch with Auto-Tune.[331]
The Los Angeles Times remarked that Swift's defining vocal feature is her attention to detail to convey an exact feeling—"the line that slides down like a contented sigh or up like a raised eyebrow".[332] With Reputation, critics noted she was "learning how to use her voice as a percussion instrument of its own",[333] swapping her "signature" expressive vocals for "cool, conversational, detached" cadences and rhythms similar to hip hop and R&B styles.[334][335][336] Alternative Press stated that her "evocative" vocal stylings are more reminiscent of pop-punk and emo genres.[337]
Reviews of Swift's later albums and performances were more appreciative of her vocals, finding them less nasal, richer, more resonant, and more powerful.[304][338][339] With Folklore and Evermore, Swift received praise for her sharp and agile yet translucent and controlled voice.[340][341][342] Pitchfork described it as "versatile and expressive".[343] With her 2021 re-recorded albums, critics began to praise the mature, deeper and "fuller" tone of her voice.[344][345][346] An i review said Swift's voice is "leagues better now".[347] The Guardian highlighted "yo-yoing vocal yelps" and passionate climaxes as the trademarks of Swift's voice,[348] and that her country twang faded away.[349] Midnights received acclaim for Swift's nuanced vocal delivery.[350] She ranked 102nd on the 2023 Rolling Stone list of the 200 Greatest Singers of All Time.[327] In a review of the Eras Tour, The New Yorker critic Amanda Petrusich praised the clarity and tone of Swift's live vocals.[351] Musicologist Alyssa Barna said that Swift's timbre is "breathy and bright" in her upper register and "full and dark" in the lower.[222]
Songwriting
Further information: List of songs by Taylor Swift
Swift has been referred to as one of the greatest songwriters ever by several publications.[352][353][354] Literature scholars like Jonathan Bate and Stephanie Burt have noted that her literary and melodic sensibility and writing style are rare amongst her peers.[355][356] Swift's bridges are often noted as one of the best aspects of her songs,[357][358] earning her the title "Queen of Bridges" from Time.[359] Mojo described her as "a sharp narrator with a gift for the extended metaphor".[360]
In The New Yorker in 2011, Swift said she identifies as a songwriter first: "I write songs, and my voice is just a way to get those lyrics across".[12] Her personal experiences were a common inspiration for her early songs, which helped her navigate life.[361][362] Her "diaristic" technique began with identifying an emotion, followed by a corresponding melody.[363][364] On her first three studio albums, love, heartbreak, and insecurities, from an adolescent perspective, were dominant themes.[365][366] She delved into the tumult of toxic relationships on Red,[367] and embraced nostalgia and post-romance positivity on 1989.[292] Reputation was inspired by the downsides of Swift's fame,[368] and Lover detailed her realization of the "full spectrum of love".[369] Other themes in Swift's music include family dynamics, friendship,[370][371] alienation, self-awareness, and tackling vitriol, especially sexism.[279][372]
Her confessional lyrics received positive reviews from critics,[373][12][374] who highlighted their vivid details and emotional engagement, which they found uncommon in pop music.[375][376][377] Critics also praised her melodic compositions; Rolling Stone described Swift as "a songwriting savant with an intuitive gift for verse-chorus-bridge architecture".[378][379] NPR dubbed Swift "a master of the vernacular in her lyrics",[335] remarking that her songs offer emotional engagement because "the wit and clarity of her arrangements turn them from standard fare to heartfelt disclosures".[379] Despite the positive reception, The New Yorker stated she was generally portrayed "more as a skilled technician than as a Dylanesque visionary".[12] Tabloid media often speculated and linked the subjects of her songs with her ex-lovers, a practice reviewers and Swift herself criticized as sexist.[380][381][382] Aside from clues in album liner notes, Swift avoided talking about the subjects of her songs.[383]
On her 2020 albums Folklore and Evermore, Swift was inspired by escapism and romanticism to explore fictional narratives.[384] She imposed emotions onto imagined characters and story arcs, which liberated her from tabloid attention and suggested new paths for her artistry.[363] Swift explained that she welcomed the new songwriting direction after she stopped worrying about commercial success.[384] According to Spin, she explored complex emotions with "precision and devastation" on Evermore.[385] Consequence stated her 2020 albums convinced skeptics of her songwriting prowess, noting her transformation from "teenage wunderkind to a confident and careful adult".[358]
Swift divides her writing into three types: "quill lyrics", songs rooted in antiquated poeticism; "fountain pen lyrics", based on modern and vivid storylines; and "glitter gel pen lyrics", which are lively and frivolous.[386] Critics note the fifth track of every Swift album as the most "emotionally vulnerable" of the album.[387] Awarding her with the Songwriter Icon Award in 2021, the National Music Publishers' Association remarked that "no one is more influential when it comes to writing music today".[388] The Week deemed her the foremost female songwriter of modern times,[389] and the Nashville Songwriters Association International named her Songwriter-Artist of the Decade in 2022.[245] Swift has also published two original poems: "Why She Disappeared" and "If You're Anything Like Me".[390]
Performances
Further information: List of Taylor Swift live performances

Swift performing on the Reputation Stadium Tour in Seattle in May 2018
Journalists have described Swift as one of the best live performers. Often praised for her showmanship and stage presence,[391][392][393][394][395] Swift commands large audiences,[396][397][398] without having to rely on dance like her contemporaries do.[399] According to V magazine's Greg Krelenstein, she possesses "a rare gift of turning a stadium spectacle into an intimate setting", irrespective of whether she is "plucking a guitar or leading an army of dancers".[400] In a 2008 review of Swift's early performances, Sasha Frere-Jones of The New Yorker called Swift a "preternaturally skilled" entertainer with a vibrant stage presence, adding "she returned the crowd's energy with the professionalism she has shown since the age of fourteen."[401] In 2023, Adrian Horton of The Guardian noted her "seemingly endless stamina" on the Eras Tour,[402] and i critic Ilana Kaplan called her showmanship "unparalleled".[403]
Critics have highlighted Swift's versatility as an entertainer, praising her ability to switch onstage personas and performance styles depending on the varying themes and aesthetics of her albums.[404][405] Her concert productions have been characterized by elaborate Broadway theatricality and high technology,[406] and her performances frequently incorporate a live band, with whom she has played and toured since 2007.[407] Swift also often accompanies herself with musical instruments such as electric guitar;[408] acoustic guitar; piano;[409] and sometimes twelve-string guitar,[410][411] banjo,[412] or ukulele.[413] Interacting frequently with the audience, her solo acoustic performances are considered intimate and emotionally resonant, complementing her story-based lyrics and fan connection.[351][414] Lydia Burgham of The Spinoff opined that this intimacy remains "integral to her singer-songwriter origins".[415][409] Chris Willman of Variety called Swift "pop's most approachable superstar",[416] and the 21st century's most popular performer.[417]
Video and film
Further information: Taylor Swift videography
Swift emphasizes visuals as a key creative component of her music-making process.[418] She has collaborated with different directors to produce her music videos, and over time she has become more involved with writing and directing. She developed the concept and treatment for "Mean" in 2011[419] and co-directed the music video for "Mine" with Roman White the year before.[420] In an interview, White said that Swift "was keenly involved in writing the treatment, casting and wardrobe. And she stayed for both the 15-hour shooting days, even when she wasn't in the scenes."[421]
From 2014 to 2018, Swift collaborated with director Joseph Kahn on eight music videos—four each from her albums 1989 and Reputation. Kahn has praised Swift's involvement.[422] She worked with American Express for the "Blank Space" music video (which Kahn directed), and served as an executive producer for the interactive app AMEX Unstaged: Taylor Swift Experience, for which she won a Primetime Emmy Award for Outstanding Interactive Program in 2015.[423] Swift produced the music video for "Bad Blood" and won a Grammy Award for Best Music Video in 2016.[424]
Her production company, Taylor Swift Productions, is credited with producing all of her visual media starting with the 2018 concert documentary Reputation Stadium Tour.[425] She continued to co-direct music videos for the Lover singles "Me!" with Dave Meyers, and "You Need to Calm Down" (also serving as a co-executive producer) and "Lover" with Drew Kirsch,[426] but first ventured into sole direction with the video for "The Man" (which won her the MTV Video Music Award for Best Direction).[427] After Folklore: The Long Pond Studio Sessions, Swift debuted as a filmmaker with All Too Well: The Short Film,[245] which made her the first artist to win the Grammy Award for Best Music Video as a sole director.[428] Swift has cited Chloé Zhao, Greta Gerwig, Nora Ephron, Guillermo del Toro, John Cassavetes, and Noah Baumbach as filmmaking influences.[418]
Accolades and achievements
Further information: List of awards and nominations received by Taylor Swift

In 2009, Swift became the first country singer to win an MTV Video Music Award.
Swift's discography is a "critically hailed songbook", as per Time's Sam Lansky.[429] She has won 14 Grammy Awards (including four for Album of the Year—the most won by an artist),[430] an Emmy Award,[431] 40 American Music Awards (the most won by an artist),[432] 39 Billboard Music Awards (the most won by an artist—tying with Drake),[433] 118 Guinness World Records,[434] 23 MTV Video Music Awards (including four Video of the Year wins—the most by an act),[247] 12 Country Music Association Awards (including the Pinnacle Award),[435] eight Academy of Country Music Awards,[436] and two Brit Awards.[168] As a songwriter, she has been honored by the Nashville Songwriters Association,[63][437] the Songwriters Hall of Fame, and the National Music Publishers' Association and was the youngest person on Rolling Stone's list of the 100 Greatest Songwriters of All Time in 2015.[438][439] At the 64th BMI Awards in 2016, Swift was the first woman to be honored with an award named after its recipient.[440]
Commercially, from available data, Swift has amassed over 50 million album sales and 150 million single sales as of 2019,[441][442][443] and 114 million units globally, including 78 billion streams as of 2021.[444][445] The International Federation of the Phonographic Industry ranked her as the Global Recording Artist of the Year for a record three times (2014, 2019 and 2022).[446] Swift has the most number-one albums in the United Kingdom and Ireland for a female artist this millennium,[447][448] earned the highest income for an artist on Chinese digital music platforms (RMB 159,000,000 as of 2021),[449] and is the first artist to replace themselves at the top spot and occupy the entire top five[note 3] of the Australian albums chart.[452][453] Swift remains the world's highest-grossing female touring act ever, with cumulative ticket sales at $1.96 billion as of November 2023 according to Pollstar.[454] The Eras Tour is the highest-grossing tour of all time as of December 2023, and the first to surpass $1 billion in revenue.[455] Beginning with Fearless, each of her studio albums have opened with over one million global units.[456][457] Swift is the most streamed female act on Spotify and Apple Music.[458][459] On Spotify, she is the only artist to have received more than 200 and 250 million streams in one day (260 million on October 27, 2023),[460] and the only female act to reach 100 million monthly listeners.[234] The most entries and the most simultaneous entries for an artist on the Billboard Global 200, with 143 and 31 songs, respectively, are among her feats.[461][462]
In the US, Swift has sold over 37.3 million albums as of 2019,[443] when Billboard placed her eighth on its Greatest of All Time Artists Chart.[463] Eleven of her songs have topped the Billboard Hot 100.[257] She is the longest-reigning act of the Billboard Artist 100 (97 weeks);[464] the soloist with the most cumulative weeks atop the Billboard 200 (68);[465] the woman with the most Billboard 200 number-ones (13),[234] Hot 100 entries (232),[234][156] number-one debuts (6),[note 4] top-ten songs (49),[257] and weeks atop the Top Country Albums chart (101);[467] and the act with the most number-one songs on Pop Airplay (12) and Digital Songs (28).[468][469] Swift is the first woman to simultaneously chart five albums in the top 10 and eleven albums on the entire Billboard 200;[470][471] and the first act to occupy the top four spots and chart seven albums[note 5] in the top 10 on the Top Album Sales chart.[473][474] She is the second highest-certified female digital singles artist (and fifth overall) in the US, with 137.5 million total units certified by the Recording Industry Association of America (RIAA),[475] and the first woman to have both an album (Fearless) and a song ("Shake It Off") certified Diamond.[476] Swift is the only artist in Luminate history to have six albums sell over a million copies in a week.[477]
Swift has appeared in various power listings. Time included her on its annual list of the 100 most influential people in 2010, 2015, and 2019.[478] She was one of the "Silence Breakers" that the magazine spotlighted as Person of the Year in 2017 for speaking up about sexual assault,[479] and received the honor again in 2023 for her cultural domination that year.[429] Time described Swift as the first Person of the Year to be recognized for "achievement in the arts", as well as the first woman to be recognized and appear on a Person of the Year cover more than once.[480][481] In 2014, she was named to Forbes' 30 Under 30 list in the music category[482] and again in 2017 in its "All-Star Alumni" category.[483] Swift became the youngest woman to be included on Forbes' list of the 100 most powerful women in 2015, ranked at number 64.[484] In 2023, she was ranked by Forbes as the fifth-most powerful woman in the world, the first entertainer to place in the top five.[485] Swift received an honorary Doctor of Fine Arts degree from New York University and served as its commencement speaker on May 18, 2022.[245]
Cultural status
Main articles: Cultural impact of Taylor Swift and Public image of Taylor Swift

Swift at the 2010 Time 100 Gala
Swift has been credited with making a profound impact on the music industry, popular culture and the economy.[486][487] She dominates cultural conversations,[488][489] which has led publications to describe her as a cultural "vitality" or zeitgeist.[490][491][492] Her music, life and public image are points of attention in global celebrity culture.[297] Initially a teen idol,[493] she has been referred to as a pop icon;[312][494] publications describe her immense popularity and longevity as unwitnessed since the 20th century.[495][496] In 2013, New York magazine's Jody Rosen dubbed Swift the "world's biggest pop star" and opined that the trajectory of her stardom has defied established patterns. Rosen added that Swift "falls between genres, eras, demographics, paradigms, trends", leaving her contemporaries "vying for second place".[310] Critics regard Swift as a rare yet successful combination of the pop star and singer-songwriter archetypes.[497]
Her fans are known as Swifties.[266] Billboard noted only few artists have had her chart success, critical acclaim, and fan support.[498] Swift's million-selling albums are considered an anomaly in the streaming-dominated industry following the end of the album era in the 2010s.[499][500] Economist Alan Krueger described Swift as an "economic genius".[501]
Although labeled by the media in her early career as "America's Sweetheart" for her girl next door persona,[502][503] Swift has been accused by detractors of being "calculated" and manipulative of her image, a narrative bolstered by her 2016 dispute with West.[173][174] Critics have also noted that her personal life and career have been subject to intense misogyny and "slut-shaming",[504][505] as well as rampant media scrutiny and tabloid speculation.[506]
Swift's private jet use has drawn scrutiny for its carbon emissions.[507][508] In 2023, a spokesperson for Swift stated that she had purchased more than double the required carbon credits to offset all tour travel and personal flights.[509][510] In December 2023, Swift's lawyers sent a cease and desist letter to American programmer Jack Sweeney over tracking her private jet, citing safety concerns and stalking.[511][512] Swift has been a victim of numerous house break-ins and stalkers, some of whom were armed.[513][514]
Legacy
"You have different artists dominating different sectors of the industry: Some are huge at streaming, some are big draws on the road. But we're at this moment where there's no one better than Taylor Swift, whether that's on the radio, with streaming, ticket sales or just cultural impact."
– Jason Lipshutz, Billboard executive director, 2023[515]
Swift helped shape the modern country music scene,[516] having extended her success beyond the Anglosphere,[310][516] pioneered the use of internet (Myspace) as a marketing tool,[33][53] and introduced the genre to a younger generation.[517][310] Country labels have since become interested in signing young singers who write their own music;[518] her guitar performances contributed to the "Taylor Swift factor", a phenomenon to which an upsurge in guitar sales to women, a previously ignored demographic, is attributed.[519][520]
According to publications, Swift changed the music landscape with her genre transitions, a discography that accommodates cultural shifts,[521] and her ability to popularize any sound in mainstream music.[522] Lyrically, in being personal and vulnerable in her songs, music journalist Nick Catucci opined Swift helped make space for later singers like Billie Eilish, Ariana Grande, and Halsey to do the same.[523] Scholars have highlighted the literary sensibility and poptimist implications of Swift.[355][524] She has been credited with legitimizing and popularizing the concept of album "eras".[525][526] Swift is a subject of academic study and scholarly media research.[297] Various educational institutions offer courses on Swift in literary, cultural and sociopolitical contexts.[527][297]
Swift has influenced numerous music artists, and her albums have inspired a generation of singer-songwriters.[517][320][528] Journalists praise her ability to reform industry practices, noting how her actions changed streaming policies, prompted awareness of intellectual property in new musicians,[529][530] and reshaped ticketing models.[531] Various sources deem Swift's music a paradigm representing the millennial generation;[532] Vox called her the "millennial Bruce Springsteen",[533] and The Times named her "the Bob Dylan of our age".[534] Swift earned the title Woman of the Decade (2010s) from Billboard,[535] Artist of the Decade (2010s) at the American Music Awards,[536] and Global Icon at the Brit Awards for her impact.[445] Senior artists such as Paul McCartney,[537] Mick Jagger,[538] Madonna,[539] and Dolly Parton have praised her musicianship.[540] Carole King regards Swift her "professional grand daughter" and thanked Swift for "carrying the torch forward".[541] Springsteen called her a "tremendous" writer,[542] while Ringo Starr and Billy Joel considered Swift the Beatles' successor.[543][544] Britney Spears labeled Swift "the most iconic pop woman of our generation".[545]
Entrepreneurship
Media outlets describe Swift as a savvy businesswoman;[546][547] in 2024, she topped Billboard's annual Power 100 ranking of the top music industry executives.[548] Swift is known for her traditional album rollouts, consisting of a variety of promotional activities that Rolling Stone termed as an inescapable "multimedia bonanza".[549][550] Easter eggs and cryptic teasers became a common practice in contemporary pop music because of Swift.[551] Publications describe her discography as a music "universe" subject to analyses by fans, critics and journalists.[552][553][554] Swift maintains an active presence on social media and a close relationship with fans, to which many journalists attribute her success.[555][487][556] Her in-house management team is called 13 Management.[557]
Swift has endorsed many brands and businesses, having launched clothing lines with L.E.I. and Stella McCartney,[558][559] designed American Greetings cards and Jakks Pacific dolls,[560][561] released a number of fragrances with Elizabeth Arden,[562] and signed multi-year deals with AT&T and Capital One.[563][564] She was a spokesperson for the National Hockey League's Nashville Predators and Sony Cyber-shot digital cameras,[565][566] and became the global ambassador for New York City in 2014 and Record Store Day in 2022.[567][568]
Social activism
Further information: Political impact of Taylor Swift
Swift identifies as a pro-choice feminist,[569] and is a founding signatory of the Time's Up movement against sexual harassment.[570] Specifically, she criticized the US Supreme Court's decision to end federal abortion rights in 2022.[571] Swift also advocates for LGBT rights,[572] and has called for the passing of the Equality Act, which prohibits discrimination based on sex, sexual orientation, and gender identity.[573][574] She performed during WorldPride NYC 2019 at the Stonewall Inn, a gay rights monument, and has donated to the LGBT organizations Tennessee Equality Project and GLAAD.[575][576][577]
A supporter of the March for Our Lives movement and gun control reform in the US,[578] Swift is a vocal critic of white supremacy, racism, and police brutality.[579][569] Following the George Floyd protests, she donated to the NAACP Legal Defense and Educational Fund and the Black Lives Matter movement,[580] called for the removal of Confederate monuments in Tennessee,[581] and advocated for Juneteenth to become a national holiday.[582] In 2020, Swift urged her fans to check their voter registration ahead of elections, which resulted in 65,000 people registering to vote within one day of her post,[583] and endorsed Joe Biden and Kamala Harris in the US presidential election.[584] She has openly criticized former president Donald Trump.[585]
Wealth
Swift's net worth is estimated by Forbes and Bloomberg News at $1.1 billion as of October 2023, making her the first musician to achieve billionaire status "solely based on her songs and performances".[586][587] Forbes named her the annual top-earning female musician in 2016, 2019, 2021, and 2022.[588] She was the highest-paid celebrity of 2016 with $170 million—a feat recognized by the Guinness World Records as the highest annual earnings ever for a female musician,[589] which she herself surpassed with $185 million in 2019.[590] Overall, Forbes listed Swift as the highest-paid female artist of the 2010s, earning $825 million.[591] She has also developed a real estate portfolio worth $150 million as of 2023, with properties in Nashville; Tribeca, Manhattan; Los Angeles (Samuel Goldwyn Estate); and Rhode Island (High Watch).[592]
Philanthropy
Swift is known for her philanthropic efforts.[593] She ranked first on DoSomething's 2015 "Gone Good" list,[594] having received the Star of Compassion from the Tennessee Disaster Services and the Big Help Award from the Nickelodeon Kids' Choice Awards for her "dedication to helping others" and "inspiring others through action".[595][596] She donated $100,000 to the Red Cross to help the victims of the Iowa flood of 2008.[597] In 2009, she sang at BBC's Children in Need concert and raised £13,000 for the cause.[598] Swift has performed at charity relief events, including Sydney's Sound Relief concert.[599] In response to the May 2010 Tennessee floods, Swift donated $500,000.[600] In 2011, Swift used a dress rehearsal of her Speak Now tour as a benefit concert for victims of recent tornadoes in the US, raising more than $750,000.[601] In 2016, she donated $1 million to Louisiana flood relief efforts and $100,000 to the Dolly Parton Fire Fund.[602][603] Swift donated to food banks after Hurricane Harvey struck Houston in 2017 and at every stop of the Eras Tour in 2023;[604][605] she also directly employed local businesses throughout the tour and gave $55 million in bonus payments to her entire crew.[606][607] Swift donated $1 million for Tennessee tornado relief in 2020 and again in 2023.[608][609]
She is a supporter of the arts. A benefactor of the Nashville Songwriters Hall of Fame,[610] Swift has donated $75,000 to Nashville's Hendersonville High School to help refurbish the school auditorium,[611] $4 million to build a new education center at the Country Music Hall of Fame and Museum in Nashville,[612] $60,000 to the music departments of six US colleges,[613] and $100,000 to the Nashville Symphony.[614] Also a promoter of children's literacy, she has donated money and books to schools around the country.[615][616] In 2007, Swift partnered with the Tennessee Association of Chiefs of Police to launch a campaign to protect children from online predators.[617] She has donated items to several charities for auction, including the UNICEF Tap Project and MusiCares.[618] As recipient of the Academy of Country Music's Entertainer of the Year in 2011, Swift donated $25,000 to St. Jude Children's Research Hospital, Tennessee.[619] In 2012, Swift participated in the Stand Up to Cancer telethon, performing the charity single "Ronan", which she wrote in memory of a four-year-old boy who died of neuroblastoma.[620] She has also donated $100,000 to the V Foundation for Cancer Research[621] and $50,000 to the Children's Hospital of Philadelphia.[622] Swift has encouraged young people to volunteer in their local communities as part of Global Youth Service Day.[623]
Swift donated to fellow singer-songwriter Kesha to help with her legal battles against Dr. Luke and to actress Mariska Hargitay's Joyful Heart Foundation.[593][624] During the COVID-19 pandemic, Swift donated to the World Health Organization and Feeding America,[625] and supported independent record stores.[626][627] Swift performed "Soon You'll Get Better" on the One World: Together At Home television special, a benefit concert curated by Lady Gaga for Global Citizen to raise funds for the World Health Organization's COVID-19 Solidarity Response Fund.[628] In 2018 and 2021, Swift donated to the Rape, Abuse & Incest National Network in honor of Sexual Assault Awareness and Prevention Month.[593][629] She has made donations to her fans several times for their medical or academic expenses.[630] In December 2023, Swift attended Ramy Youssef's fundraiser for the Gaza Strip.[631]
Discography
Main articles: Taylor Swift albums discography, Taylor Swift singles discography, and List of songs by Taylor Swift
Studio albums
Taylor Swift (2006)
Fearless (2008)
Speak Now (2010)
Red (2012)
1989 (2014)
Reputation (2017)
Lover (2019)
Folklore (2020)
Evermore (2020)
Midnights (2022)
The Tortured Poets Department (2024)
Re-recorded albums
Fearless (Taylor's Version) (2021)
Red (Taylor's Version) (2021)
Speak Now (Taylor's Version) (2023)
1989 (Taylor's Version) (2023)
Filmography
Main article: Taylor Swift videography
This section lists select works only. Refer to the main article for further information.
Valentine's Day (2010)
The Lorax (2012)
The Giver (2014)
Cats (2019)
All Too Well: The Short Film (also director) (2021)
Amsterdam (2022)
Documentary and concert films
Journey to Fearless (2010)
Speak Now World Tour – Live (2011)
The 1989 World Tour Live (2015)
Taylor Swift: Reputation Stadium Tour (2018)
Miss Americana (2020)
Taylor Swift: City of Lover (2020)
Folklore: The Long Pond Studio Sessions (also director) (2020)
Taylor Swift: The Eras Tour (2023)
Tours
Main article: List of Taylor Swift live performances
Fearless Tour (2009–2010)
Speak Now World Tour (2011–2012)
The Red Tour (2013–2014)
The 1989 World Tour (2015)
Reputation Stadium Tour (2018)
The Eras Tour (2023–2024)
See also
List of American Grammy Award winners and nominees
List of highest-certified music artists in the United States
List of most-followed Instagram accounts
List of most-followed Twitter accounts
List of most-subscribed YouTube channels
Footnotes
 Swift held the record until the 62nd Annual Grammy Awards in 2020.[91][92]
 Though Swift has properties throughout the US, she identifies Nashville as her home.[149][150]
 Swift has occupied the top five of the ARIA Albums Chart twice. She achieved this feat first on the issue published on July 7, 2023,[450] followed by a second time on the issue published on February 9, 2024.[451]
 In a tie with Ariana Grande.[466]
 Swift has charted seven titles in the top 10 of the Top Album Sales chart twice—on the issues dated January 6, 2024, and January 20, 2024.[472]
References
 "Taylor Swift: The record-breaking artist in numbers". Newsround. March 2, 2020. Archived from the original on March 8, 2020. Retrieved April 20, 2020.
 Sutherland, Mark (May 23, 2015). "Taylor Swift interview: 'A relationship? No one's going to sign up for this'". The Daily Telegraph. Archived from the original on January 10, 2022. Retrieved April 20, 2020.
 Scott, Walter (June 11, 2015). "What Famous Pop Star Is Named After James Taylor?". Parade. Archived from the original on October 15, 2016. Retrieved December 12, 2018.
 "Taylor Swift is not an "underdog": The real story about her 1 percent upbringing that the New York Times won't tell you". Salon.com. May 23, 2015. Archived from the original on May 25, 2022. Retrieved December 26, 2020.
 Jepson 2013, p. 1.
 Roth, Madeline (May 19, 2015). "Taylor Swift's Brother Had The Most Epic Graduation Weekend Ever". MTV News. Archived from the original on July 23, 2016. Retrieved July 25, 2016.
 McKay, Gabriel (July 6, 2023). "Taylor Swift Edinburgh: Is star the real queen of Scotland?". The Herald. Archived from the original on February 1, 2024. Retrieved February 4, 2024.
 Eleftheriou-Smith, Loulla-Mae (June 24, 2015). "Taylor Swift tells Scotland: 'I am one of you'". The Independent. Archived from the original on May 26, 2022. Retrieved July 10, 2019.
 Vadala, Nick (July 14, 2017). "Taylor Swift ancestor's home added to Philly Register of Historic Places". The Philadelphia Inquirer. Archived from the original on May 25, 2022. Retrieved January 15, 2021.
 "Taylor Swift's Great-Great-Grandfather's Philly Home Gets Historic Landmark Status". AP NEWS. July 25, 2017. Archived from the original on May 22, 2022. Retrieved January 15, 2021.
 "Taylor Swift stammt aus dem Freistaat" (in German). BR24. September 17, 2015. Archived from the original on December 31, 2021. Retrieved July 23, 2023.
 Widdicombe, Lizzie (October 10, 2011). "You Belong With Me". The New Yorker. Archived from the original on July 24, 2014. Retrieved October 11, 2011.
 Raab, Scott (October 20, 2014). "Taylor Swift Interview". Esquire. Archived from the original on February 16, 2015. Retrieved April 11, 2015.
 "Taylor Swift on Politicians Co-opting Faith: 'I'm a Christian. That's Not What We Stand For'". Relevant. January 31, 2020. Archived from the original on November 25, 2022. Retrieved April 2, 2020.
 Uhrich, Bill (February 13, 2010). "Photos Students at Alvernia Montessori School sending Taylor Swift a valentine". Reading Eagle. Archived from the original on October 16, 2013. Retrieved February 25, 2013.
 Hatza, George (December 8, 2008). "Taylor Swift: Growing into superstardom". Reading Eagle. Archived from the original on April 1, 2012. Retrieved April 17, 2012.
 Mennen, Lauren (November 12, 2014). "Taylor Swift's Wyomissing childhood home on the market for $799,500". Philadelphia Daily News. Archived from the original on October 17, 2016. Retrieved October 13, 2016.
 Chang, David (February 22, 2016). "Taylor Swift Returns to Reading Pennsylvania as Maid of Honor in Friend's Wedding". WCAU. Archived from the original on September 16, 2016. Retrieved August 26, 2016.
 "Taylor Swift, Age 12". New York Daily News. Archived from the original on August 27, 2016. Retrieved August 26, 2016.
 Cooper, Brittany Joy (April 15, 2012). "Taylor Swift Opens Up About a Future in Acting and Admiration for Emma Stone". Taste of Country. Archived from the original on April 17, 2012. Retrieved April 17, 2012.
 MacPherson, Alex (October 18, 2012). "Taylor Swift: 'I want to believe in pretty lies'". The Guardian. Archived from the original on August 26, 2016. Retrieved August 3, 2016.
 Rolling Stone Interview: The Unabridged Taylor Swift, December 2, 2008
 Morris, Edward (December 1, 2006). "When She Thinks 'Tim McGraw', Taylor Swift Savors Payoff: Hardworking Teen to Open for George Strait Next Year". CMT. Archived from the original on June 26, 2015. Retrieved March 11, 2010.
 Diu, Nisha Lilia (April 3, 2011). "Taylor Swift: 'I won't do sexy shoots'". The Daily Telegraph. Archived from the original on May 6, 2013. Retrieved April 17, 2012.
 "News : CMT Insider Interview: Taylor Swift (Part 1 of 2)". CMT. November 26, 2008. Archived from the original on January 23, 2015. Retrieved July 1, 2012.
 Malec, Jim (May 2, 2011). "Taylor Swift: The Garden In The Machine". American Songwriter. Archived from the original on May 10, 2012. Retrieved May 21, 2012.
 Qureshi, Hira. "Visit this Stone Harbor café where Taylor Swift was 'always coming in to play' as a child". Courier-Post. Archived from the original on October 26, 2023. Retrieved December 12, 2022.
 Kuperinsky, Amy (July 28, 2020). "Taylor Swift shouts out Jersey Shore town in video for surprise album". NJ.com. Archived from the original on December 12, 2022. Retrieved December 12, 2022.
 Martino, Andy (January 10, 2015). "EXCLUSIVE: The real story of Taylor Swift's guitar 'legend'". New York Daily News. Archived from the original on November 22, 2015. Retrieved August 28, 2017.
 "Dymtrow v. Swift et al: Federal Civil LawsuitNew York Southern District Court, Case No. 1:07-cv-11277-RJS" (PDF). American Bar Association. Archived from the original (PDF) on October 11, 2012. Retrieved April 18, 2012.
 "On tour with Taylor Swift". NBC News. May 31, 2009. Archived from the original on October 5, 2013. Retrieved July 1, 2012.
 Castro, Vicky (February 6, 2015). "How to Succeed as an Entrepreneur, Taylor Swift Style". Inc. Archived from the original on June 7, 2016. Retrieved February 9, 2015.
 Willman, Chris (July 25, 2007). "Getting to know Taylor Swift". Entertainment Weekly. Archived from the original on May 24, 2022. Retrieved January 25, 2022.
 Jo, Nancy (January 2, 2014). "Taylor Swift and the Growing of a Superstar: Her Men, Her Moods, Her Music". Vanity Fair. Archived from the original on November 10, 2015. Retrieved November 11, 2015.
 "News : Taylor Swift's High School Names Auditorium in Her Honor". CMT. September 23, 2010. Archived from the original on November 21, 2014. Retrieved April 18, 2012.
 Grigoriadis, Vanessa (March 5, 2009). "The Very Pink, Very Perfect Life of Taylor Swift". Rolling Stone. Archived from the original on May 3, 2019. Retrieved July 28, 2019.
 "Taylor Swift receives her high school diploma". Houston Chronicle. July 27, 2008. Archived from the original on January 1, 2024. Retrieved January 2, 2024.
 "Taylor Swift: The Garden In The Machine". American Songwriter. May 2, 2011. Archived from the original on August 7, 2013. Retrieved May 21, 2012.
 "Songwriter Taylor Swift Signs Publishing Deal With Sony/ATV". Broadcast Music, Inc. May 12, 2005. Archived from the original on December 4, 2012. Retrieved April 20, 2012.
 Kosser, Michael (June 3, 2010). "Liz Rose: Co-Writer to the Stars". American Songwriter. Archived from the original on December 24, 2011. Retrieved April 19, 2012.
 Leahey, Andrew (October 24, 2014). "Songwriter Spotlight: Liz Rose". Rolling Stone. Archived from the original on September 26, 2016. Retrieved September 24, 2016.
 DeLuca, Dan (November 11, 2008). "Focused on 'great songs' Taylor Swift isn't thinking about 'the next level' or Joe Jon as gossip". Philadelphia Daily News. p. 1. Archived from the original on November 18, 2012. Retrieved April 17, 2012.
 Preston, John (April 26, 2009). "Taylor Swift: the 19-year-old country music star conquering America – and now Britain". The Daily Telegraph. Archived from the original on January 5, 2012. Retrieved August 30, 2012.
 Rosa, Christopher (March 24, 2015). "Opening Acts Who Became Bigger Than The Headliner". VH1. Archived from the original on November 10, 2015. Retrieved November 11, 2015.
 Rapkin, Mickey (July 27, 2017). "Oral History of Nashville's Bluebird Cafe: Taylor Swift, Maren Morris, Dierks Bentley & More on the Legendary Venue". Billboard. Archived from the original on July 29, 2017. Retrieved July 28, 2017.
 Hiatt, Brian (October 25, 2012). "Taylor Swift in Wonderland". Rolling Stone. Archived from the original on July 31, 2016. Retrieved August 1, 2016.
 Greenburg, Zack O'Malley (June 26, 2013). "Toby Keith, Cowboy Capitalist: Country's $500 Million Man". Forbes. Archived from the original on August 27, 2016. Retrieved August 1, 2016.
 Taylor Swift (CD). Big Machine Records. 2006. BMR120702.
 Tamarkin, Jeff. "Taylor Swift – Taylor Swift". AllMusic. Archived from the original on October 20, 2015. Retrieved February 14, 2021.
 Neal, Chris (December 4, 2006). "Taylor Swift Review". Country Weekly. Archived from the original on July 22, 2012. Retrieved March 31, 2010.
 Trust, Gary (October 29, 2009). "Chart Beat Thursday: Taylor Swift, Tim McGraw Linked Again". Billboard. Archived from the original on March 7, 2013. Retrieved November 8, 2016.
 "Taylor Swift". Songwriters' Hall of Fame. Archived from the original on February 12, 2021. Retrieved September 21, 2022.
 Willman, Chris (February 5, 2008). "Taylor Swift's Road to Fame". Entertainment Weekly. p. 3. Archived from the original on February 21, 2015. Retrieved April 22, 2012.
 "Taylor Swift Joins Rascal Flatts Tour". CMT. October 18, 2006. Archived from the original on January 7, 2015. Retrieved March 11, 2010.
 Whitaker, Sterling; Hammar, Ania (May 27, 2019). "How Eric Church's Rascal Flatts Feud Helped Launch Taylor Swift's Career". Taste of Country. Townsquare Media. Archived from the original on June 6, 2019. Retrieved June 10, 2019.
 "Taylor Swift No. 1 on iTunes". Great American Country. December 19, 2007. Archived from the original on March 3, 2012. Retrieved July 5, 2010.
 "Taylor Swift – Chart history". Billboard. Archived from the original on August 9, 2016. Retrieved July 26, 2016.
 "Taylor Swift owns top of country chart". Country Standard Time. July 23, 2008. Archived from the original on July 31, 2008. Retrieved December 26, 2008.
 "Wal-Mart "Eyes" New Taylor Swift Project". Great American Country. Archived from the original on July 23, 2008. Retrieved July 24, 2008.
 "Taylor Swift Joins George Strait's 2007 Tour". CMT. November 17, 2006. Archived from the original on August 11, 2017. Retrieved February 16, 2020.
 "Brad Paisley Plans Tour With Three Opening Acts". CMT. January 9, 2007. Archived from the original on August 11, 2017. Retrieved February 16, 2020.
 "Taylor Swift Joins Tim McGraw, Faith Hill on Tour". CMT. June 1, 2007. Archived from the original on August 11, 2017. Retrieved February 16, 2020.
 "Taylor Swift Youngest Winner of Songwriter/Artist Award". Great American Country. October 16, 2007. Archived from the original on January 11, 2015. Retrieved February 2, 2015.
 "Photos : All Taylor Swift Pictures : Horizon Award Winner Poses in the Pressroom". CMT. September 7, 2007. Archived from the original on November 13, 2012. Retrieved May 21, 2012.
 "Photos : 43rd Annual ACM Awards – Onstage: Winners : Acceptance Speech". CMT. May 18, 2008. Archived from the original on November 13, 2012. Retrieved May 21, 2012.
 "Taylor Swift, Rascal Flatts, Carrie Underwood Score at 2008 AMA Awards" (Blog). Roughstock.com. November 24, 2008. Archived from the original on July 10, 2014. Retrieved May 21, 2012.
 "Amy Winehouse Wins Best New Artist, Kanye West Pays Tribute to Mom – Grammy Awards 2008, Grammy Awards". People. October 2, 2008. Archived from the original on November 13, 2012. Retrieved May 21, 2012.
 "Rascal Flatts Announce Summer Tour With Taylor Swift". CMT. May 5, 2008. Archived from the original on April 18, 2022. Retrieved June 5, 2019.
 Caplan, David (September 8, 2008). "Scoop". People. Archived from the original on February 2, 2016. Retrieved March 6, 2012.
 Rizzo, Monica (November 24, 2008). "Scoop – Couples, Camilla Belle, Joe Jonas". People. Archived from the original on March 3, 2016. Retrieved March 6, 2012.
 Akers, Shelley (June 9, 2008). "Taylor Swift to Appear in Hannah Montana Movie". People. Archived from the original on October 27, 2017. Retrieved October 27, 2017.
 "Hannah Montana: The Movie (Original Motion Picture Soundtrack) by Hannah Montana". iTunes Store. January 2009. Archived from the original on May 2, 2016. Retrieved August 2, 2016.
 "CD Taylor Swift – Fearless" (in Portuguese). Universal Music Group. Archived from the original on January 18, 2021. Retrieved February 14, 2021.
 Raphael, Amy (February 1, 2009). "First, she conquered Nashville. Now she's set for world domination". The Observer. ProQuest 250507223. Archived from the original on December 9, 2022. Retrieved December 9, 2022.
 Widdicombe, Lizzie (October 10, 2011). "You Belong with Me". The New Yorker. Archived from the original on July 24, 2014. Retrieved July 24, 2014.
 "Discography Taylor Swift". ARIA Charts. Archived from the original on March 21, 2012. Retrieved January 2, 2010.
 Trust, Gary (December 15, 2009). "Best of 2009: Part 1". Billboard. Archived from the original on March 3, 2013. Retrieved September 20, 2022.
 Ben-Yehuda, Ayala (August 13, 2009). "Black Eyed Peas, Jason Mraz Tie Records on Billboard Hot 100". Billboard. Archived from the original on May 8, 2013. Retrieved March 13, 2010.
 Trust, Gary (September 24, 2009). "Taylor Swift Climbs Hot 100, Black Eyed Peas Still No. 1". Billboard. Archived from the original on February 1, 2013. Retrieved September 20, 2022.
 "Taylor Swift Chart History (Hot Country Songs)". Billboard. Archived from the original on January 31, 2021. Retrieved February 14, 2021.
 Grein, Paul (March 16, 2012). "Chart Watch Extra: Top Albums Of Last 10 Years" (Blog). Yahoo! Music. Archived from the original on April 2, 2015. Retrieved June 10, 2011.
 Mapes, Jillian (November 23, 2010). "Taylor Swift Announces 'Speak Now' World Tour". Billboard. Archived from the original on May 8, 2013. Retrieved May 15, 2012.
 Weiss, Dan (December 12, 2011). "Taylor Swift: Journey To Fearless DVD". American Songwriter. Archived from the original on August 15, 2016. Retrieved August 2, 2016.
 Ryan, Sarah (August 10, 2009). "Taylor Swift Pranks Keith Urban" (Blog). Great American Country. Archived from the original on September 6, 2015. Retrieved November 11, 2015.
 "Kanye calls Taylor Swift after 'View' appearance". MSNBC. September 15, 2009. Archived from the original on October 6, 2013. Retrieved September 16, 2009.
 "Taylor Swift Thanks "Gracious" Beyonce for Inviting Her Onstage After Kanye Stunt at VMAs". Rolling Stone. September 14, 2009. Archived from the original on June 15, 2012. Retrieved May 15, 2012.
 Anderson, Kyle (September 16, 2009). "Kanye West's VMA Interruption Gives Birth To Internet Photo Meme". MTV. Archived from the original on January 16, 2016. Retrieved October 3, 2009.
 Ditzian, Eric (2009). "Taylor Swift, Michael Jackson Big Winners at American Music Awards". MTV. Archived from the original on September 8, 2014. Retrieved May 15, 2012.
 "2009 Artists of the Year". Billboard. December 10, 2009. Archived from the original on January 6, 2010. Retrieved May 21, 2012.
 "Taylor Swift Raps 'Thug Story' With T-Pain On CMT Awards". MTV. June 17, 2009. Archived from the original on May 25, 2022. Retrieved November 11, 2015.
 Kreps, Daniel (February 1, 2010). "Beyonce, Taylor Swift Dominate 2010 Grammy Awards". Rolling Stone. Archived from the original on February 15, 2012. Retrieved February 13, 2012.
 "Billie Eilish replaces Taylor Swift as youngest artist to win a Grammy for Album of the Year". MSN. January 27, 2020. Archived from the original on November 20, 2020. Retrieved September 1, 2020.
 Kaufman, Gil (November 12, 2009). "Taylor Swift Dominates CMA Awards". MTV News. Archived from the original on March 6, 2016. Retrieved September 13, 2016.
 Vena, Jocelyn (November 6, 2009). "John Mayer Talks Taylor Swift Collaboration 'Half of My Heart'". MTV. Archived from the original on September 8, 2014. Retrieved May 15, 2012.
 "Boys Like Girls featuring Taylor Swift, 'Two Is Better Than One'". Billboard. December 2, 2009. Archived from the original on August 18, 2021. Retrieved December 4, 2020.
 "Kellie Pickler Has Her 'Best Days' Thanks To Taylor Swift". MTV. Archived from the original on January 1, 2016. Retrieved November 11, 2015.
 Vena, Jocelyn (December 28, 2009). "New Taylor Swift Song Included In 'Valentine's Day' Featurette". MTV. Archived from the original on January 16, 2016. Retrieved November 11, 2015.
 "Taylor Swift – Chart history on Canadian Hot 100". Billboard. Archived from the original on August 9, 2016. Retrieved August 3, 2016.
 Park, Michael Y.; Sia, Nicole (December 29, 2009). "Taylor & Taylor Romance Was Overblown, Says Source". People. Archived from the original on November 13, 2012. Retrieved March 6, 2012.
 Caramanica, Jon (March 6, 2009). "OMG! Taylor Swift Does 'CSI'!". The New York Times (Blog). Archived from the original on August 14, 2011. Retrieved May 7, 2012.
 Strecker, Erin (January 2, 2015). "Remember When Taylor Swift Shined as 'Saturday Night Live' Host?". Billboard. Archived from the original on January 24, 2015. Retrieved January 15, 2015.
 Dukes, Billy (October 22, 2012). "10 Things You Didn't Know About Taylor Swift". Taste of Country. Townsquare Media. Archived from the original on May 22, 2022. Retrieved July 26, 2020.
 Pietroluongo, Silvio (August 11, 2010). "Taylor Swift Makes Sparkling Hot 100 Entrance". Billboard. Archived from the original on January 31, 2016. Retrieved July 25, 2016.
 Caramanica, Jon (October 20, 2010). "Taylor Swift, Angry on 'Speak Now'". The New York Times. Archived from the original on October 21, 2010. Retrieved October 23, 2010.
 "Taylor Swift's New Album, Speak Now, Set for Oct. 25 Release". CMT. July 20, 2010. Archived from the original on June 21, 2019. Retrieved February 14, 2020.
 Kaufman, Gil (November 3, 2010). "Taylor Swift's Speak Now Tops 1 Million in First Week". MTV. Archived from the original on August 10, 2016. Retrieved August 8, 2016.
 "Fastest-selling digital album in the US by a female artist". Guinness World Records. Archived from the original on June 22, 2015. Retrieved June 16, 2015.
 Knopper, Steve (November 25, 2010). "Taylor Swift's Speak Now Tops the Charts". Rolling Stone. Archived from the original on March 3, 2021. Retrieved November 25, 2010.
 Sheffield, Rob (October 26, 2010). "Speak Now (2010)". Rolling Stone. Archived from the original on September 11, 2013. Retrieved December 20, 2019.
 Hammel, Sara (January 4, 2011). "Taylor Swift & Jake Gyllenhaal Break Up: Source". People. Archived from the original on May 9, 2012. Retrieved March 6, 2012.
 Wyland, Sarah (February 12, 2012). "Taylor Swift Takes Home Two GRAMMYs at Tribute-Filled Show" (Blog). Great American Country. Archived from the original on September 6, 2015. Retrieved February 13, 2012.
 Shelburne, Craig (October 18, 2010). "Taylor Swift Named NSAI's Songwriter-Artist of the Year". CMT. Archived from the original on January 16, 2016. Retrieved November 21, 2015.
 Smith, Hazel (October 24, 2011). "News : Hot Dish: Taylor Swift Sings Alan Jackson's Masterpiece at Nashville Songwriters Celebration". CMT. Archived from the original on November 29, 2014. Retrieved April 22, 2012.
 Roland, Tom (December 2, 2011). "Taylor Swift: Billboard's Woman of the Year". Billboard. Archived from the original on February 3, 2013. Retrieved May 15, 2012.
 Talbott, Chris; Silva, Cristina (April 2, 2012). "Taylor Swift wins ACM entertainer of the year". Yahoo!. Associated Press. Archived from the original on August 23, 2016. Retrieved May 15, 2012.
 "CMA Awards 2011: Taylor Swift wins entertainer of the year". CBS News. November 9, 2011. Archived from the original on September 8, 2014. Retrieved May 15, 2012.
 Kellogg, Jane (November 20, 2011). "AMAs 2011: Winners and Nominees Complete List". The Hollywood Reporter. Archived from the original on June 27, 2015. Retrieved November 21, 2015.
 Sheffield, Rob (June 23, 2012). "Women Who Rock: The 50 Greatest Albums of All Time". Rolling Stone. Archived from the original on December 10, 2016. Retrieved July 15, 2017.
 Allen, Bob (March 29, 2012). "Hot Tours: Taylor Swift, George Strait, Cirque Du Soleil". Billboard. Archived from the original on February 21, 2013. Retrieved May 10, 2012.
 "Taylor Swift News and Blog". taylorswift.com. September 21, 2011. Archived from the original on October 11, 2011. Retrieved September 21, 2011.
 Herrera, Monica (March 15, 2012). "Taylor Swift, Arcade Fire Talk 'Hunger Games'". Rolling Stone. Archived from the original on June 27, 2015. Retrieved May 10, 2012.
 "Nominations 2013 — Golden Globe Awards". goldenglobes.org. December 13, 2012. Archived from the original on December 14, 2012. Retrieved December 13, 2012.
 Horowitz, Steven J. (April 20, 2012). "B.o.B Explains Origins of Taylor Swift Collaboration 'Both of Us'". HipHopDX. Archived from the original on September 10, 2015. Retrieved May 15, 2012.
 Toomedy, Alyssa (October 25, 2012). "Taylor Swift and Conor Kennedy Breakup: Anatomy of a Split". E! News. Archived from the original on December 4, 2015. Retrieved November 10, 2015.
 Trust, Gary (August 22, 2012). "Taylor Swift Scores First Hot 100 No. 1". Billboard. Archived from the original on February 13, 2013. Retrieved August 22, 2012.
 "Discography Taylor Swift". New Zealand Charts. Archived from the original on April 20, 2017. Retrieved July 26, 2016.
 Lynch, Kevin (September 4, 2013). "Calvin Harris trumps Michael Jackson feat to join Taylor Swift, Rihanna and One Direction in Guinness World Records™ 2014 book". Guinness World Records. Archived from the original on June 22, 2015. Retrieved June 16, 2015.
 Chart positions:
 • "Taylor Swift – I Knew You Were Trouble". ARIA Charts. Archived from the original on May 27, 2016. Retrieved February 14, 2021.
 • "Official Singles Charts Top 100". Official Charts Company. Archived from the original on July 13, 2021. Retrieved February 14, 2021.
 • "Taylor Swift Leads Record Breaking Digital Sales Week". Billboard. January 3, 2013. Archived from the original on April 8, 2017. Retrieved February 14, 2021.
 Lewis, Randy (October 30, 2012). "Taylor Swift raises the bar with a savvy 'Red' marketing campaign". Los Angeles Times. Archived from the original on December 28, 2020. Retrieved December 28, 2020.
 Mansfield, Brian (October 17, 2012). "Taylor Swift sees 'Red' all over". USA Today. Archived from the original on December 21, 2012.
 English, J. (August 28, 2017). "Taylor Swift's 'Red': A Canonical Coming-Of-Age Album". NPR. Archived from the original on April 12, 2021. Retrieved February 14, 2021.
 Roberts, Randall (October 31, 2012). "'The Last Time' connects Taylor Swift with Arcade Fire". Los Angeles Times. Archived from the original on March 6, 2016. Retrieved December 27, 2020.
 Greenwald, David (September 6, 2013). "Taylor Swift, Rihanna, Justin Bieber Among 2014 Guinness Record-Setters". Billboard. Archived from the original on June 8, 2015. Retrieved July 27, 2016.
 Sexton, Paul (August 31, 2019). "Taylor Swift Scores Fourth U.K. No. 1 With 'Lover' Album". Billboard. Archived from the original on August 16, 2021. Retrieved October 15, 2019.
 "Grammys 2014: The complete list of nominees and winners". Los Angeles Times. January 26, 2014. Archived from the original on March 4, 2015. Retrieved January 25, 2015.
 Gregoire, Carolyn (November 19, 2012). "Taylor Swift AMA Awards 2012: Pop Star Performs 'I Knew You Were Trouble' (Video)". HuffPost. Archived from the original on May 26, 2013. Retrieved June 10, 2013.
 Payne, Chris (November 25, 2013). "Taylor Swift & Justin Timberlake Win Big at American Music Awards". Billboard. Archived from the original on November 24, 2015. Retrieved November 21, 2015.
 "NSAI Songwriter/Artists of the Year". Nashville Songwriters Association International. Archived from the original on August 19, 2016. Retrieved August 2, 2016.
 Allen, Bob (July 3, 2014). "Taylor Swift's Red Wraps as All-Time Country Tour". Billboard. Archived from the original on June 1, 2015. Retrieved April 11, 2015.
 Caramanica, Jon (November 7, 2013). "Country Awards Hold Swift Close". The New York Times. Archived from the original on November 11, 2013. Retrieved April 3, 2014.
 "Taylor Swift's Boyfriend Timeline: 10 Relationships & Their Songs". Billboard. December 30, 2014. Archived from the original on August 26, 2016. Retrieved August 26, 2016.
 Labrecque, Jeff (December 12, 2013). "'12 Years a Slave' and 'American Hustle' lead Golden Globe nominees". Entertainment Weekly. Archived from the original on March 23, 2015. Retrieved December 12, 2013.
 Bonaguro, Alison (January 25, 2013). "News : Offstage: Tim McGraw Wanted to Make Taylor Swift Duet an Event". CMT. Archived from the original on July 12, 2015. Retrieved February 25, 2013.
 Blistein, Doyle (June 4, 2013). "Taylor Swift Joins Rolling Stones for 'As Tears Go By'". Rolling Stone. Archived from the original on June 9, 2013. Retrieved June 10, 2013.
 "Taylor Swift Joins Florida Georgia Line Onstage for 'Cruise'". Taste of Country. Townsquare Media. March 2, 2013. Archived from the original on March 16, 2013. Retrieved March 29, 2013.
 Collin, Robbie (July 26, 2012). "The Lorax, review". The Daily Telegraph. Archived from the original on September 17, 2016. Retrieved August 3, 2016.
 Beard, Lanford (May 1, 2013). "Taylor Swift says 'I do' to 'New Girl'". Entertainment Weekly. Archived from the original on November 8, 2016. Retrieved August 4, 2016.
 Busis, Hillary (September 27, 2013). "Taylor Swift will co-star in long-awaited adaptation of 'The Giver'". Entertainment Weekly. Archived from the original on December 21, 2016. Retrieved August 2, 2016.
 Peterson, Price (March 31, 2014). "Taylor Swift Moves into NYC Apartment Built Over Mysterious River of Pink Slime". The Atlantic. Archived from the original on September 27, 2016. Retrieved July 31, 2016.
 Rogers, Alex (March 7, 2014). "Why Taylor Swift Thinks Nashville Is the Best Place on Earth". Time. Archived from the original on May 24, 2022. Retrieved September 27, 2016.
 Milzoff, Rebecca; Brown, Eric Renner; Denis, Kyle (August 24, 2023). "Taylor Swift and Beyoncé Are So Big, Even Their Publicists Have Fans". Billboard. Archived from the original on August 24, 2023. Retrieved August 24, 2023.
 Zollo, Paul (February 17, 2016). "The Oral History of Taylor Swift's 1989". The Recording Academy. Archived from the original on June 3, 2021. Retrieved February 14, 2021.
 Stutz, Colin (October 16, 2014). "Watch Taylor Swift's '1989' Secret Sessions Behind The Scenes Video". Billboard. Archived from the original on June 5, 2016. Retrieved August 2, 2016.
 Caulfield, Keith (November 4, 2014). "Taylor Swift's "1989" debuts with 1.287 million copies sold". Billboard. Archived from the original on November 5, 2014. Retrieved November 4, 2014.
 Chart positions:
 • "Discography Taylor Swift". ARIA Charts. Archived from the original on August 26, 2019. Retrieved February 14, 2021.
 • "Taylor Swift Chart History (Canadian Hot 100)". Billboard. Archived from the original on August 9, 2016. Retrieved February 14, 2021.
 • "Taylor Swift's 'Bad Blood' Blasts to No. 1 on Hot 100". Billboard. May 28, 2015. Archived from the original on September 6, 2015. Retrieved May 28, 2015.
 "Taylor Swift – Chart History: Hot 100". Billboard. Archived from the original on March 8, 2018. Retrieved September 18, 2016.
 "Live Music's $20 Billion Year: The Grateful Dead's Fare Thee Well Reunion, Taylor Swift, One Direction Top Boxscore's Year-End". Billboard. Archived from the original on December 14, 2015.
 Weissmann, Jordan (July 7, 2014). "Taylor Swift Has Written an Op-Ed in the Wall Street Journal". Slate (Blog). Archived from the original on January 23, 2015. Retrieved January 23, 2015.
 Knopper, Steve (November 8, 2014). "Taylor Swift's Label Head Explains Spotify Removal". Rolling Stone. Archived from the original on April 21, 2015. Retrieved April 11, 2015.
 Peters, Mitchell (June 21, 2015). "Taylor Swift Pens Open Letter Explaining Why '1989' Won't Be on Apple Music". Billboard. Archived from the original on June 22, 2015. Retrieved June 22, 2015.
 Halperin, Shirley (June 21, 2015). "Apple Changes Course After Taylor Swift Open Letter: Will Pay Labels During Free Trial". Billboard. Archived from the original on June 22, 2015. Retrieved June 22, 2015.
 Rosen, Christopher (June 25, 2015). "Taylor Swift is putting 1989 on Apple Music". Entertainment Weekly. Archived from the original on June 26, 2015. Retrieved June 25, 2015.
 "Taylor Swift returns to Spotify on the day Katy Perry's album comes out". BBC News. June 9, 2017. Archived from the original on June 9, 2017.
 "Taylor Swift: 2014 Billboard Woman of the Year". Billboard. October 10, 2014. Archived from the original on June 1, 2015. Retrieved April 11, 2015.
 Payne, Chris (November 23, 2014). "Taylor Swift Wins Dick Clark Award of Excellence at 2014, Presented by Diana Ross". Billboard. Archived from the original on July 4, 2015. Retrieved April 11, 2015.
 "The Taylor Swift Experience". GRAMMY Museum. Archived from the original on November 25, 2022. Retrieved April 22, 2022.
 Boehrer, Kat (January 7, 2016). "Watch Taylor Swift's Stunning Acoustic Performance of 'Blank Space' at the Grammy Museum". Complex. Archived from the original on April 22, 2022. Retrieved April 22, 2022.
 Jonze, Tim (February 25, 2015). "Taylor Swift wins international female solo artist at Brit awards 2015". The Guardian. Archived from the original on January 1, 2016. Retrieved April 11, 2015.
 Stutz, Colin (July 21, 2015). "2015 MTV Video Music Awards Nominees Revealed: Taylor Swift, Kendrick Lamar, Ed Sheeran & More". Billboard. Archived from the original on July 24, 2015. Retrieved August 15, 2016.
 Lynch, Joe (February 19, 2016). "Taylor Swift Joins Elite Club to Win Grammy Album of the Year More Than Once: See the Rest". Billboard. Archived from the original on March 1, 2016. Retrieved July 31, 2016.
 Chiu, Melody (June 1, 2016). "Taylor Swift and Calvin Harris Split After 15 Months Together". People. Archived from the original on June 22, 2016. Retrieved June 1, 2016.
 Spanos, Brittany (July 13, 2016). "Taylor Swift Co-Wrote Calvin Harris' Smash Hit 'This Is What You Came For'". Rolling Stone. Archived from the original on July 30, 2016. Retrieved July 31, 2016.
 Grady, Constance (August 26, 2019). "How the Taylor Swift-Kanye West VMAs scandal became a perfect American morality tale". Vox. Archived from the original on December 2, 2022. Retrieved October 12, 2023.
 Grady, Constance (March 21, 2020). "Newly leaked footage shows Taylor Swift and Kanye West talking "Famous"". Vox. Archived from the original on March 24, 2022. Retrieved October 12, 2023.
 Lewis, Anna (July 15, 2016). "Tom Hiddleston finally tells us the truth about his relationship with Taylor Swift". Cosmopolitan. Archived from the original on July 17, 2016. Retrieved January 8, 2019.
 "Taylor Swift Finally Reveals When She Started Dating Joe Alwyn in Lover Album". Yahoo!. August 23, 2019. Archived from the original on June 11, 2020. Retrieved June 11, 2020.
 "Paul McCartney & Taylor Swift". Rolling Stone. November 13, 2020. Archived from the original on November 30, 2020. Retrieved September 15, 2021. McCartney: So how does that go? Does your partner sympathize with that and understand? Swift: Oh, absolutely.
 Irvin, Jack (April 10, 2023). "Inside Taylor Swift and Joe Alwyn's 'Differences' That Led to Their Breakup: Sources (Exclusive)". People. Archived from the original on September 25, 2023. Retrieved September 25, 2023.
 "Taylor Swift, pop princess, wins song of the year at the CMA Awards". USA Today. Archived from the original on November 9, 2017. Retrieved November 9, 2017.
 Trust, Gary (February 21, 2017). "Ed Sheeran Tops Hot 100, Katy Perry Debuts at No. 4 & Bruno Mars, Rihanna & The Weeknd All Hit Top 10". Billboard. Archived from the original on February 22, 2017. Retrieved February 22, 2017.
 Grady, Constance (August 11, 2017). "Taylor Swift won her day in court. Here's what you need to know". Vox. Archived from the original on October 17, 2022. Retrieved October 4, 2023.
 "Taylor Swift wipes social media profiles, fuelling expectations of new album". The Daily Telegraph. August 18, 2017. Archived from the original on January 8, 2019. Retrieved August 19, 2019.
 Aswad, Jem (August 24, 2017). "Taylor Swift's New Single, 'Look What You Made Me Do,' Arrives (Listen)". Variety. Archived from the original on August 28, 2017. Retrieved August 29, 2017.
 White, Jack (September 1, 2017). "Taylor Swift scores first Number 1 on the Official Singles Chart with 'LWYMMD'". Official Charts Company. Archived from the original on September 2, 2017. Retrieved September 1, 2017.
 Peak positions:
 • "Taylor Swift Scores Fifth No. 1 Single". Australian Recording Industry Association. September 2, 2017. Archived from the original on September 2, 2017. Retrieved September 2, 2017.
 • "IRMA – Irish Charts". Irish Recorded Music Association. Archived from the original on June 14, 2017. Retrieved September 2, 2017.
 • "NZ Top 40 Singles Chart". Recorded Music NZ. September 4, 2017. Archived from the original on September 1, 2017. Retrieved September 1, 2017.
 • "Taylor Swift at Nos. 1 & 4 on Billboard Hot 100, as Cardi B Moves Up to No. 2". Billboard. September 11, 2017. Archived from the original on September 21, 2017. Retrieved October 30, 2017.
 Shaw, Lucas (November 7, 2017). "Taylor Swift Will Keep New Album From Streaming for a Week". Bloomberg. Bloomberg News. Archived from the original on November 8, 2017. Retrieved November 7, 2017.
 Powers, Ann (November 10, 2019). "The Old Taylor's Not Dead". NPR. Archived from the original on June 9, 2020. Retrieved June 29, 2020.
 McDermott, Maeve (October 11, 2017). "Taylor Swift 'Reputation': Here's what critics are saying". USA Today. Archived from the original on March 4, 2020. Retrieved October 15, 2020.
 "Official: Taylor Swift's 'Reputation' Album Sells 1.2M Copies in US During First Week". Billboard. Archived from the original on November 30, 2017. Retrieved May 29, 2018.
 Chart positions:
 • "Taylor Swift's 'Reputation' Rules Australia's Albums Chart". Billboard. November 20, 2017. Archived from the original on November 20, 2017. Retrieved December 2, 2017.
 • "Taylor Swift Chart History". Billboard. Archived from the original on November 22, 2021. Retrieved November 19, 2021.
 Unterberger, Andrew (July 6, 2018). "Taylor Swift's 'Delicate' Became the Biggest Reputation Radio Hit While You Weren't Looking". Billboard. Archived from the original on December 13, 2020. Retrieved April 13, 2020.
 "61st Grammy Nominees". The Recording Academy. December 7, 2018. Archived from the original on December 7, 2018. Retrieved December 7, 2018.
 Hudak, Joseph (April 12, 2018). "Sugarland Announce New Album Bigger, Taylor Swift Collaboration". Rolling Stone. Archived from the original on April 14, 2018. Retrieved April 13, 2018.
 Havens, Lyndsey (October 9, 2018). "Taylor Swift Breaks an All-Time AMA Record – And Urges People to Vote in Midterm Elections". Billboard. Archived from the original on October 10, 2018. Retrieved October 10, 2018.
 Stubblebine, Allison (November 13, 2017). "Taylor Swift Announces First Round of Reputation Stadium Tour Dates". Billboard. Archived from the original on November 14, 2017. Retrieved November 18, 2017.
 Frankenberg, Eric (December 6, 2018). "Taylor Swift Closes Reputation Stadium Tour with $345 Million". Billboard. Archived from the original on December 9, 2018. Retrieved December 22, 2018.
 Wang, Amy X. (November 19, 2018). "Taylor Swift's New Record Deal Affects Thousands of Other Musicians". Rolling Stone. Archived from the original on November 26, 2018. Retrieved November 26, 2018.
Willman, Chris (August 27, 2018). "Taylor Swift Stands to Make Music Business History as a Free Agent". Variety. Archived from the original on August 29, 2018. Retrieved August 29, 2018.
Aswad, Jem; Willman, Chris (November 19, 2018). "Taylor Swift Signs New Deal With Universal Music Group". Variety. Archived from the original on November 19, 2018. Retrieved November 19, 2018.
 Grady, Constance (November 19, 2018). "What Taylor Swift's new record deal means for the music industry — and for her image". Vox. Archived from the original on December 20, 2021. Retrieved December 20, 2021.
 McKenna, Lyndsey (August 23, 2019). "Stream Taylor Swift's New Album, 'Lover'". NPR. Archived from the original on February 19, 2020. Retrieved September 10, 2019.
 Catucci, Nick (August 23, 2019). "Taylor Swift Reaches For New Heights of Personal and Musical Liberation on 'Lover'". Rolling Stone. Archived from the original on August 23, 2019. Retrieved February 15, 2021.
 Caulfield, Keith (September 1, 2019). "Official: Taylor Swift's 'Lover' Debuts at No. 1 on Billboard 200 Chart With 867,000 Units Earned in First Week in U.S." Billboard. Archived from the original on September 1, 2019. Retrieved September 2, 2019.
 White, Adam (August 23, 2019). "Taylor Swift Lover Review Round-Up: Critics Say Album Feels 'Evolutionary Rather Than Revolutionary'". The Independent. Archived from the original on June 13, 2022. Retrieved June 13, 2022.
 Moniuszko, Sara M. (August 23, 2019). "Taylor Swift Lover Reviews: Critics Are Enamored by the 'Earnest,' 'Romantic' New Album". USA Today. Archived from the original on August 23, 2019. Retrieved November 5, 2020.
 Trust, Gary (May 6, 2019). "Lil Nas X's 'Old Town Road' Tops Billboard Hot 100 For Fifth Week, Taylor Swift's 'Me!' Vaults to No. 2". Billboard. Archived from the original on May 6, 2019. Retrieved June 14, 2019.
 Trust, Gary (October 23, 2023). "Taylor Swift's 'Cruel Summer' Hits No. 1 on Billboard Hot 100, Becoming Her 10th Leader". Billboard. Archived from the original on October 24, 2023. Retrieved October 24, 2023.
 "Arashi Best-Of Tops Taylor Swift for IFPI's Best-Selling Album of 2019". Billboard. March 19, 2020. Archived from the original on March 19, 2020. Retrieved March 21, 2020.
 "2020 Grammy Awards: Complete Winners List". The Recording Academy. November 20, 2019. Archived from the original on May 22, 2020. Retrieved February 15, 2021.
 Grein, Paul (August 26, 2019). "12 Records That Were Set at the 2019 VMAs". Billboard. Archived from the original on January 30, 2020. Retrieved January 11, 2020.
 Grady, Constance (September 1, 2019). "The Taylor Swift/Scooter Braun controversy, explained". Vox. Archived from the original on February 11, 2020. Retrieved August 23, 2019.
 Beth, John (January 2, 2024). "Taylor Swift's Chart Triumph". Square News.
 "The Taylor Swift, Scooter Braun, Justin Bieber row explained". BBC News. July 1, 2019. Archived from the original on December 11, 2021. Retrieved July 18, 2021.
 Willman, Chris (November 16, 2020). "Taylor Swift Confirms Sale of Her Masters, Says She Is Already Re-Recording Her Catalog". Variety. Archived from the original on December 3, 2022. Retrieved November 18, 2020.
 Aniftos, Rania (November 15, 2019). "Taylor Swift Releases 'Beautiful Ghosts,' Co-Written With Andrew Lloyd Webber for 'Cats' Film". Billboard. Archived from the original on November 19, 2019. Retrieved November 15, 2019.
 "Golden Globes 2020: full list of nominations". The Guardian. December 9, 2019. Archived from the original on December 10, 2019. Retrieved December 20, 2019.
 Rooney, David (December 18, 2019). "'Cats': Film Review". The Hollywood Reporter. Archived from the original on December 20, 2019. Retrieved December 21, 2019.
 Mamo, Heran (January 15, 2020). "Taylor Swift Miss Americana Netflix Doc Has a Release Date & We're So Ready for It". Billboard. Archived from the original on April 24, 2020. Retrieved January 19, 2020.
 Willman, Chris (February 6, 2020). "Taylor Swift Moves to Universal Music Publishing Group with New Pact". Variety. Archived from the original on February 12, 2020. Retrieved February 6, 2020.
 Opperman, Jeff (March 12, 2021). "Taylor Swift Is Singing Us Back to Nature". The New York Times. Archived from the original on December 28, 2021. Retrieved May 24, 2021.
 "Taylor Swift to release surprise ninth album 'Evermore' tonight". NME. December 10, 2020. Archived from the original on December 10, 2020. Retrieved December 10, 2020.
 Atkinson, Katie (December 15, 2020). "Taylor Swift Isn't So Sure She & Joe Alwyn Would Have Made Music Together If It Weren't for Lockdown". Billboard. Archived from the original on August 11, 2021. Retrieved February 18, 2021.
 Schaffer, Claire (December 18, 2020). "Aaron Dessner on How His Collaborative Chemistry With Taylor Swift Led to Evermore". Rolling Stone. Archived from the original on December 22, 2020. Retrieved February 18, 2021.
 Barna, Alyssa (December 16, 2020). "These are the musicological reasons Taylor Swift's new album sounds dull". The Washington Post. Archived from the original on February 25, 2021. Retrieved November 3, 2021.
 Snapes, Laura (October 14, 2022). "'Genuine': why Taylor Swift can celebrate more than an album release". The Guardian. Archived from the original on October 14, 2022. Retrieved October 14, 2022.
 McGrath 2023, p. 79; Fogarty & Arnold 2021, p. 5.
 Trust, Gary (January 28, 2021). "Taylor Swift's 'Coney Island' and 'No Body, No Crime' Debut on Airplay Charts, Joining 'Willow'". Billboard. Archived from the original on August 16, 2021. Retrieved February 2, 2021.
 Willman, Chris (March 14, 2021). "Taylor Swift Becomes First Woman to Win Album of the Year Grammy Three Times". Variety. Archived from the original on December 2, 2021. Retrieved March 15, 2021.
 Caulfield, Keith (January 7, 2021). "Lil Baby's My Turn Is MRC Data's Top Album of 2020, Roddy Ricch's 'The Box' Most-Streamed Song". Billboard. Archived from the original on January 7, 2021. Retrieved January 7, 2021.
 Trust, Gary (December 21, 2020). "Taylor Swift's 'Willow' Debuts at No. 1 on Billboard Hot 100". Billboard. Archived from the original on December 22, 2020. Retrieved February 18, 2021.
 Willman, Chris (November 23, 2020). "Taylor Swift Wins Three American Music Awards, Says She's MIA Because of 'Recording All of My Old Music'". Variety. Archived from the original on December 2, 2021. Retrieved November 25, 2020.
 Christman, Ed (July 19, 2021). "Billboard's U.S. Money Makers: The Top Paid Musicians of 2020". Billboard. Archived from the original on July 24, 2021. Retrieved July 19, 2021.
 • Christman, Ed (July 19, 2021). "Billboard's 2020 Global Money Makers: The 5 Top Highest Paid Musicians". Billboard. Archived from the original on July 23, 2021. Retrieved July 19, 2021.
 Caulfield, Keith (July 11, 2023). "Taylor Swift's Re-Recorded Speak Now Already Has 2023's Biggest Week After 4 Days of Release". Billboard. Archived from the original on July 11, 2023. Retrieved July 11, 2023.
 Caulfield, Keith (April 18, 2021). "Taylor Swift's Re-Recorded Fearless Album Debuts at No. 1 on Billboard 200 Chart With Year's Biggest Week". Billboard. Archived from the original on April 18, 2021. Retrieved April 19, 2021.
 Asker, Jim; Trust, Gary (February 22, 2021). "Taylor Swift's 'Love Story (Taylor's Version)' Debuts at No. 1 on Hot Country Songs Chart: 'I'm So Grateful to the Fans'". Billboard. Archived from the original on April 22, 2021. Retrieved February 22, 2021.
 McCluskey, Megan (December 8, 2023). "Breaking Down Taylor Swift's 2023 Impact By the Numbers". Time. Archived from the original on December 26, 2023. Retrieved December 26, 2023.
 Horton, Adrian; Lee, Benjamin (February 6, 2023). "Grammy awards 2023: list of winners". The Guardian. Archived from the original on February 6, 2023. Retrieved February 6, 2023.
 Corcoran, Nina (August 28, 2022). "Taylor Swift Announces New Album Midnights, Breaks Record for Most Video of the Year Wins at 2022 VMAs". Pitchfork. Archived from the original on August 29, 2022. Retrieved August 28, 2022.
 "Taylor Swift's new album breaks Spotify streaming record". The Guardian. October 22, 2022. Archived from the original on October 22, 2022. Retrieved October 22, 2022.
 Harvilla, Rob (October 25, 2022). "The Anti-Hero We Deserve: Taylor Swift and Her Polarizing 'Midnights'". The Ringer. Archived from the original on November 3, 2022. Retrieved November 3, 2022.
 Light, Alan (October 24, 2022). "Taylor Swift's Midnights Does Something Astonishing. Even For Her". Esquire. Archived from the original on January 17, 2023. Retrieved January 17, 2023.
 Petridis, Alexis (October 21, 2022). "Taylor Swift: Midnights Review – Small-Hours Pop Rich with Self-Loathing and Stereotype-Smashing". The Guardian. Archived from the original on October 22, 2022. Retrieved October 21, 2022.
 Spanos, Brittany (October 21, 2022). "Taylor Swift Lets Us Into Her Darkest Dreams On Midnights". Rolling Stone. Archived from the original on October 21, 2022. Retrieved October 21, 2022.
 Sheffield, Rob (October 21, 2022). "Welcome to the Lavender Labyrinth: Taylor Swift's Midnights Is the Mastermind's Ultimate Power Move". Rolling Stone. Archived from the original on October 22, 2022. Retrieved October 23, 2022.
 Balasaygun, Kaitlin (November 1, 2022). "How Taylor Swift went back to the past and turned Midnights into her biggest album success yet". CNBC. Archived from the original on December 30, 2022. Retrieved December 30, 2022.
 Shafer, Ellise (October 21, 2022). "Taylor Swift's Midnights Breaks Spotify Record for Most-Streamed Album in a Single Day". Variety. Archived from the original on October 21, 2022. Retrieved October 22, 2022.
 Dailey, Hannah (December 6, 2022). "Here Are All of Taylor Swift's Biggest Accomplishments in 2022". Billboard. Archived from the original on April 30, 2023. Retrieved December 7, 2022.
 Trust, Gary (June 5, 2023). "Morgan Wallen's 'Last Night' No. 1 on Hot 100 for Ninth Week, Taylor Swift & Ice Spice's 'Karma' Blasts to No. 2". Billboard. Archived from the original on October 26, 2023. Retrieved August 21, 2023.
 Blistein, Jon; Guglielmi, Jodi (September 13, 2023). "Taylor Swift Makes History at 2023 VMAs". Rolling Stone. Archived from the original on September 13, 2023. Retrieved September 13, 2023.
 West, Bryan (February 5, 2024). "Taylor Swift makes Grammys history with fourth album of the year win for Midnights". USA Today. Archived from the original on February 5, 2024. Retrieved February 5, 2024.
 Caulfield, Keith (July 16, 2023). "Taylor Swift's Re-Recorded 'Speak Now' Debuts at No. 1 on Billboard 200 With 2023's Biggest Week". Billboard. Archived from the original on July 19, 2023. Retrieved July 16, 2023.
 Caulfield, Keith (November 5, 2023). "Taylor Swift's 1989 (Taylor's Version) Debuts at No. 1 on Billboard 200 With Biggest Week in Nearly a Decade". Billboard. Archived from the original on November 5, 2023. Retrieved November 5, 2023.
 Sherman, Maria (November 29, 2023). "Taylor Swift is Spotify's most-streamed artist of 2023, ending Bad Bunny's 3-year reign". ABC News. Archived from the original on November 29, 2023. Retrieved November 29, 2023.
 Garcia, Thania (November 28, 2023). "Taylor Swift Named Apple Music's Artist of the Year; Morgan Wallen Tops Global Songs Chart". Variety. Archived from the original on November 29, 2023. Retrieved November 29, 2023.
 "Best of 2023 (Taylor's Version) Playlist on Amazon Music". Curated by Amazon's Music Experts. Amazon Music. Retrieved December 3, 2023. Our most streamed artist of 2023 globally. It's Taylor's world and we are just living for it.
 Caulfield, Keith (November 21, 2023). "Taylor Swift Is Billboard's Top Artist of 2023". Billboard. Archived from the original on November 21, 2023. Retrieved November 21, 2023.
 Caulfield, Keith (December 3, 2023). "Taylor Swift's '1989 (Taylor's Version)' Returns to No. 1 on Billboard 200". Billboard. Retrieved December 3, 2023.
 Caulfield, Keith (January 10, 2024). "Morgan Wallen's One Thing at a Time Is Luminate's Top Album of 2023 in U.S." Billboard. Archived from the original on January 10, 2024. Retrieved January 12, 2024.
 Trust, Gary (November 6, 2023). "Taylor Swift's 'Is It Over Now? (Taylor's Version)' Debuts at No. 1 on Billboard Hot 100". Billboard. Retrieved November 6, 2023.
 Cohen, Jonathan (June 29, 2021). "Aaron Dessner, Justin Vernon Rev Up Big Red Machine With Help From Taylor Swift". Variety. Archived from the original on May 23, 2022. Retrieved February 10, 2022.
 Strauss, Matthew (February 19, 2021). "HAIM Enlist Taylor Swift for New "Gasoline" Remix". Pitchfork. Archived from the original on February 19, 2021. Retrieved February 10, 2022.
 Dailey, Hannah (February 11, 2022). "Ed Sheeran & Taylor Swift Release 'The Joker and the Queen' Remix: Watch the Video". Billboard. Archived from the original on February 11, 2022. Retrieved February 11, 2022.
 Rowley, Glenn (January 18, 2023). "The National Unveils 'First Two Pages of Frankenstein' Tracklist With Taylor Swift, Phoebe Bridgers & Sufjan Stevens". Billboard. Archived from the original on January 18, 2023. Retrieved January 18, 2023.
 Davis, Clayton (December 21, 2022). "Taylor Swift Doesn't Make Oscar Shortlist for All Too Well Short Film, but Advances for 'Carolina' Original Song". Variety. Archived from the original on December 24, 2022. Retrieved December 24, 2022.
 Utley, Riley (October 13, 2022). "Every Taylor Swift Movie Performance, Ranked". CinemaBlend. Archived from the original on October 13, 2022. Retrieved October 14, 2022.
 Lang, Brent (December 9, 2022). "Taylor Swift Making Feature Directing Debut for Searchlight Pictures". Variety. Archived from the original on December 10, 2022. Retrieved December 9, 2022.
 Wood, Mikael; Brown, August (August 1, 2023). "It's a love story, L.A. just says yes: How Taylormania took over the world". Los Angeles Times. Archived from the original on August 1, 2023. Retrieved August 1, 2023.
 Mahdawi, Arwa (November 20, 2022). "Swifties know: the Ticketmaster fiasco shows America has a monopoly problem". The Guardian. Archived from the original on December 10, 2022. Retrieved November 20, 2022.
 Murray, Conor (December 8, 2023). "Taylor Swift's Eras Tour Is First In History To Gross Over $1 Billion, Report Says". Forbes. Archived from the original on December 8, 2023. Retrieved December 8, 2023.
 Tapp, Tom (December 8, 2023). "Taylor Swift's 'The Eras Tour' Grosses Over $1 Billion In 2023, The Biggest Haul For Any Act Ever". Deadline. Archived from the original on February 6, 2024. Retrieved February 10, 2024.
 Kaufman, Gil (November 28, 2023). "Taylor Swift's 'Eras Tour' Concert Movie Passes $250 Million in Worldwide Grosses". Billboard. Archived from the original on December 1, 2023. Retrieved December 1, 2023.
 Nordyke, Kimberly (January 7, 2024). "Golden Globes 2024 Winners List". The Hollywood Reporter. Archived from the original on January 8, 2024. Retrieved January 8, 2024.
 Adamczyk, Alicia; Abrams, Joseph (July 25, 2023). "The brilliant marketing synergy of Taylor Swift's Eras Tour and her rerecorded albums". Fortune. Archived from the original on August 21, 2023. Retrieved August 21, 2023.
 Ingham, Tim (June 14, 2023). "Reliving the Taylor Swift Catalog Sale Saga (And Following the Money...)". Music Business Worldwide. Archived from the original on June 14, 2023. Retrieved June 15, 2023.
 Blanchet, Brenton (November 20, 2023). "Travis Kelce Shares the Real Story of How Taylor Swift Romance Began in Wide-Ranging Interview". People. Archived from the original on November 20, 2023. Retrieved November 20, 2023.
 Kelly, Samantha Murphy (January 25, 2024). "Explicit, AI-generated Taylor Swift images spread quickly on social media". CNN Business. Archived from the original on January 25, 2024. Retrieved January 25, 2024.
 Phillips, Zoe G. (January 27, 2024). "SAG-AFTRA and White House Issue Statements on Taylor Swift AI Nudes: "We Have It in Our Power to Control These Technologies"". The Hollywood Reporter. Archived from the original on January 27, 2024. Retrieved January 27, 2024.
 "Taylor Swift Announces 'Brand New Album' 'The Tortured Poets Department' with 13th Grammy Win". Peoplemag. Archived from the original on February 10, 2024. Retrieved February 10, 2024.
 Cairns, Dan (March 5, 2009). "Swift rise of the anti-diva". The Australian. Archived from the original on December 24, 2014. Retrieved July 2, 2012.
 Bream, Jon (December 7, 2007). "Music: OMG! Taylor's senior year". Star Tribune. Archived from the original on July 12, 2015. Retrieved July 1, 2012.
 Newman, Melinda (December 19, 2008). "Taylor Swift Sessions Interview". AOL. Archived from the original on October 9, 2012. Retrieved March 25, 2011.
 "Swift starts world tour in Asia, pushes "Speak Now' in NY". Country Standard Time. October 23, 2007. Archived from the original on November 13, 2012. Retrieved July 1, 2012.
 "News : 20 Questions With Taylor Swift". Country Music Television. November 12, 2007. Archived from the original on November 17, 2014. Retrieved April 18, 2012.
 McCafferty, Dennis (April 13, 2008). "Taylor's Swift rise". USA Weekend. Archived from the original on November 14, 2012. Retrieved April 17, 2012.
 "Interview with Taylor Swift". Time. April 23, 2009. Archived from the original on October 23, 2013. Retrieved July 1, 2012.
 "Taylor Swift Style: Singer Won't Take Her Clothes Off, Wants People To Focus On Music". HuffPost. October 23, 2012. Archived from the original on January 4, 2015. Retrieved January 4, 2015.
 "InStyle meets country singing sensation Taylor Swift". InStyle UK. October 26, 2010. Archived from the original on May 27, 2013. Retrieved May 29, 2012.
 "Joni Mitchell: 15 Great Artists Influenced by the 'Blue' Singer". Rolling Stone. June 22, 2016. Archived from the original on December 26, 2021. Retrieved December 26, 2020.
 Jenkins, Sally (September 28, 2023). "You thought you knew the NFL. Now meet Taylor's Version". The Washington Post. Archived from the original on October 26, 2023. Retrieved September 29, 2023.
 Bonaguro, Alison (November 8, 2012). "Offstage: Taylor Swift Inspired by Female Singer-Songwriters of the '90s". CMT. Archived from the original on October 4, 2023. Retrieved September 29, 2023.
 "Taylor Swift's Favorite Music". The Oprah Winfrey Show. Archived from the original on January 16, 2016. Retrieved October 23, 2012.
 Widdicombe, Lizzie (October 10, 2011). "You Belong With Me". The New Yorker. Archived from the original on July 24, 2014. Retrieved October 11, 2011.
 Mansfield, Brian (October 23, 2010). "Taylor Swift learns to 'Speak Now,' reveal her maturity". USA Today. Archived from the original on November 4, 2012. Retrieved July 1, 2012.
 Block, Melissa (October 31, 2014). "'Anything That Connects': A Conversation With Taylor Swift". NPR Music. Archived from the original on February 6, 2015. Retrieved October 26, 2019.
 Eells, Josh (September 8, 2014). "The Reinvention of Taylor Swift". Rolling Stone. Archived from the original on June 4, 2016. Retrieved June 8, 2016.
 Reid, Poppy (November 2, 2021). "The Curious Case of Keith Urban". Rolling Stone. Archived from the original on November 19, 2021. Retrieved November 3, 2021.
 Hiatt, Brian (June 18, 2019). "Taylor Swift: The Rolling Stone Interview". Rolling Stone. Archived from the original on September 18, 2019. Retrieved April 14, 2022.
 Weatherby, Taylor (March 10, 2021). "Taylor Swift's Road to Folklore". The Recording Academy. Archived from the original on November 25, 2021. Retrieved November 24, 2021.
 Franssen, Gaston (January 2, 2022). "Policing the celebrity of Taylor Swift: introduction". Celebrity Studies. 13 (1): 90–92. doi:10.1080/19392397.2022.2026148. S2CID 246997248.
 Savage, Mark (October 19, 2022). "Midnights: What we know about Taylor Swift's songwriting". BBC News. Archived from the original on October 19, 2022. Retrieved October 20, 2022.
 Bruner, Raisa (August 24, 2020). "Let's Break Down Taylor Swift's Tender New Album Folklore". Time. Archived from the original on July 31, 2020. Retrieved October 20, 2022.
 McNutt 2020, p. 77.
 Hughes 2017, p. 206; Perone 2017, p. 33.
 "Taylor Swift: Album Guide". Rolling Stone. Archived from the original on December 5, 2012. Retrieved December 5, 2012.
 "Pop and Rock Listings July 22 – 28". The New York Times. July 21, 2011. Archived from the original on January 28, 2012. Retrieved July 12, 2012.
 "Taylor Swift Remade Fearless as Taylor's Version. Let's Discuss". The New York Times. April 9, 2021. Archived from the original on April 9, 2021. Retrieved April 21, 2021.
 Petridis, Alexis (March 6, 2009). "Taylor Swift: Fearless". The Guardian. Archived from the original on October 16, 2013. Retrieved August 13, 2022.
 Jones, Sasha-Frere (November 11, 2008). "Prodigy". The New Yorker. Archived from the original on October 21, 2016. Retrieved August 14, 2022.
 Hughes 2017, p. 206.
 Malec, Jim (May 2, 2011). "Taylor Swift: The Garden In The Machine". American Songwriter. p. 5. Archived from the original on November 20, 2022. Retrieved August 12, 2022.
 McNutt 2020, p. 78.
 Rosen, Jody (November 17, 2013). "Why Taylor Swift Is the Reigning Queen of Pop". Vulture. Archived from the original on November 19, 2013. Retrieved November 9, 2020.
 McNutt 2020, p. 79.
 Levine, Nick (August 21, 2019). "Taylor Swift's Lover: The struggle to maintain superstardom". BBC. Archived from the original on March 1, 2021. Retrieved October 29, 2021.
 da Silva, Michelle (November 13, 2017). "Taylor Swift Has Changed for the Worse on Reputation". Now. Archived from the original on July 26, 2020. Retrieved July 27, 2020.
 Tucker, Ken (November 13, 2017). "Taylor Swift Pushes Further Into Electro-Pop With 'Reputation'". NPR. Archived from the original on October 27, 2021. Retrieved April 8, 2023.
 Moreland, Quinn (October 24, 2022). "Taylor Swift: Midnights". Pitchfork. Archived from the original on October 24, 2022. Retrieved April 8, 2023.
 Ryan, Elise (October 21, 2022). "Review: Taylor Swift gets dark, electric on 'Midnights'". Associated Press News. Archived from the original on October 21, 2022. Retrieved April 8, 2023.
 Winter, Velvet (November 12, 2022). "Like The Beatles, Madonna and Kylie Minogue before her, Taylor Swift is masterful at pivoting". ABC News. Archived from the original on November 13, 2022. Retrieved November 13, 2022.
 McNutt 2020, p. 79; Sloan 2021, p. 17.
 Sloan 2021, p. 17.
 Hyden, Steven (March 10, 2021). "Taylor Swift, Indie-rock star? Long, Long ago, this might have felt strange". The New York Times. Archived from the original on April 13, 2021. Retrieved April 13, 2021.
 Caramanica, Jon (July 26, 2020). "Taylor Swift, A Pop Star Done with Pop". The New York Times. Archived from the original on September 10, 2020. Retrieved August 14, 2022.
 Harbron, Lucy (November 11, 2021). "Why Taylor Swift's 'Red' Is Her Turning Point". Clash. Archived from the original on November 12, 2021. Retrieved November 12, 2021.
 Gerber, Brady (July 27, 2020). "The Story Behind Every Song on Taylor Swift's folklore". Vulture. Retrieved December 12, 2023.
 Willman, Chris (October 21, 2022). "Taylor Swift's Midnights Marks a Return to Electronic, Confessional Pop That's Worth Losing Sleep Over: Album Review". Variety. Archived from the original on October 21, 2022. Retrieved October 21, 2022.
 Fulford 2014, p. 192.
 "Taylor Swift Deepens Her Goth-Folk Vision on the Excellent 'Evermore'". Rolling Stone. December 11, 2020. Archived from the original on December 11, 2020. Retrieved August 23, 2022.
 "The 200 Greatest Singers of All Time". Rolling Stone. January 1, 2023. Archived from the original on January 1, 2023. Retrieved January 1, 2023.
 Provenzano 2018, p. 173.
 Roland, Tom (October 15, 2010). "Taylor Swift: The Billboard Cover Story". Billboard. Archived from the original on October 18, 2010. Retrieved July 3, 2012.
 Provenzano 2018, pp. 173–174.
 Provenzano 2018, p. 174.
 Powers, Ann (October 25, 2010). "Album review: Taylor Swift's Speak Now". Los Angeles Times. Archived from the original on October 28, 2010. Retrieved October 25, 2010.
 Willman, Chris (November 10, 2017). "Album Review: Taylor Swift's 'Reputation'". Variety. Retrieved April 8, 2023.
 Cox, Jamieson (November 13, 2017). "Taylor Swift: Reputation". Pitchfork. Retrieved April 8, 2023.
 Powers, Ann (November 10, 2017). "The Old Taylor's Not Dead". NPR. Retrieved April 8, 2023.
 Wilson, Carl (November 13, 2017). "On Reputation, the "Old Taylor" Is Dead, but the New One Isn't Quite Ready to Come to the Phone". Slate. Retrieved April 8, 2023.
 DeCaro, Alessandro (October 19, 2022). "10 best Taylor Swift scene covers". Alternative Press. Retrieved July 1, 2023.
 Barker, Andrew (November 27, 2020). "Folklore: The Long Pond Studio Sessions Review". Variety. Retrieved May 31, 2021.
 Lipshutz, Jason (March 18, 2023). "The 13 Best Moments From Taylor Swift's Eras Tour Kickoff". Billboard. Retrieved April 8, 2023.
 Kornhaber, Spencer (July 28, 2020). "Taylor Swift Is No Longer Living in the Present". The Atlantic. Retrieved August 23, 2022.
 Brehian, Tom (July 24, 2020). "Review: Taylor Swift's 'folklore' Is An Indie Record Unconcerned With Being Cool". Stereogum. Retrieved August 23, 2022.
 Willman, Chris (December 11, 2020). "Taylor Swift Has Her Second Great Album of 2020 With 'Evermore': Album Review". Variety. Retrieved August 23, 2022.
 Sodomsky, Sam (December 15, 2020). "Taylor Swift: evermore". Pitchfork. Archived from the original on December 15, 2020. Retrieved December 14, 2020.
 McCormick, Neil (April 9, 2021). "Taylor Swift copies her younger self – and she sounds even more Fearless today". The Daily Telegraph. Retrieved August 23, 2022.
 "Taylor Swift forges ahead with a dreamy throwback in Fearless (Taylor's Version)". The A.V. Club. April 9, 2021. Archived from the original on April 9, 2021. Retrieved August 23, 2022.
 Bernstein, Jonathan (April 9, 2021). "Taylor Swift Carefully Reimagines Her Past on 'Fearless: Taylor's Version'". Rolling Stone. Retrieved August 23, 2022.
 Solomon, Kate (November 12, 2021). "Taylor Swift, Red (Taylor's Version), Review: How Brilliant She Is When Her Heart Is in Tatters". i. Archived from the original on November 12, 2021. Retrieved November 13, 2021.
 Snapes, Laura (August 23, 2020). "Taylor Swift: Folklore review – bombastic pop makes way for emotional acuity". The Guardian. Retrieved August 23, 2022.
 Snapes, Laura (November 12, 2021). "Taylor Swift: Red (Taylor's Version) Review – Getting Back Together with a Classic". The Guardian. Archived from the original on November 12, 2021. Retrieved November 12, 2021.
 Kelly, Fred (October 21, 2022). "Taylor Swift's Midnights: what the critics are saying". The Week. Retrieved October 23, 2022.
 Petrusich, Amanda (June 12, 2023). "The Startling Intimacy of Taylor Swift's Eras Tour". The New Yorker. Retrieved June 12, 2023.
 "Taylor Swift's songwriting: how the star's music has changed, for better or worse". CBC Music. August 22, 2019. Retrieved November 10, 2021.
 Emily, Lee (November 5, 2021). "Here Are Taylor Swift's Best Bridges On 'Red' Ranked". iHeartRadio. Retrieved November 10, 2021.
 Eggertsen, Chris (September 20, 2022). "Taylor Swift's Iconic Songwriting Credits Amplified By Spotify With Dedicated Page". Billboard. Retrieved September 20, 2022.
 Bate, Jonathan (April 10, 2023). "Why Taylor Swift is a literary giant — by a Shakespeare professor". The Times. Retrieved April 10, 2023.
 Pazzanese, Christina (August 2, 2023). "So what exactly makes Taylor Swift so great?". Harvard Gazette. Retrieved August 7, 2023.
 Murphy, Sam (November 10, 2021). "How 'Red' Became The Most Pivotal Record In Taylor Swift's Career". Junkee. Retrieved November 10, 2021.
 Siroky, Mary (November 9, 2021). "Every Taylor Swift Album Ranked from Worst to Best". Consequence. Retrieved November 10, 2021.
 Bruner, Raisa (May 24, 2021). "How Olivia Rodrigo Become America's Biggest New Pop Star". Time. Retrieved November 10, 2021.
 Mulvey, John (September 16, 2023). "Arctic Monkeys, Taylor Swift, Kendrick Lamar, Lana Del Rey And The 30 Artists Who Will Shape The Next 30 Years". Mojo. Retrieved September 19, 2023.
 Farley, Christopher John (October 22, 2010). "Taylor Swift's Solo Act". The Wall Street Journal. Archived from the original on February 1, 2015. Retrieved May 24, 2012.
 Jo Sales, Nancy; Diehl, Jessica (April 2013). "Taylor Swift's Telltale Heart". Vanity Fair. Archived from the original on January 30, 2017. Retrieved February 4, 2017.
 Daly, Rhian (December 13, 2020). "Taylor Swift says her diaristic songwriting style was 'unsustainable' for her future". NME. Retrieved February 17, 2021.
 Gallo, Phil (October 22, 2012). "Taylor Swift's Red: The Billboard Cover Story". Billboard. Archived from the original on June 14, 2013.
 Caramanica, Jon (October 20, 2010). "Taylor Swift Is Angry, Darn It". The New York Times. Archived from the original on September 11, 2012. Retrieved July 2, 2012.
 Kelly, James (August 26, 2009). "Taylor Swift writing her own songs and rules". The Atlanta Journal-Constitution. Archived from the original on September 8, 2014. Retrieved July 30, 2012.
 Lansky, Sam (November 8, 2017). "Why Taylor Swift's Red Is Her Best Album". Billboard. Retrieved December 27, 2020.
 Hiatt, Brian (September 30, 2019). "9 Taylor Swift Moments That Didn't Fit in Our Cover Story". Rolling Stone. Archived from the original on October 1, 2019. Retrieved December 9, 2019.
 "Taylor Swift Talks Newfound 'Freedom,' 'Lover' Tour Plans and So Much More". On Air with Ryan Seacrest. August 27, 2019. Retrieved March 22, 2020.
 Yuan, Jada (December 30, 2009). "Microwaving a tragedy: The marriage of romance and romanticism in '00s pop". Las Vegas Weekly. Archived from the original on December 21, 2013. Retrieved August 17, 2012.
 Rotman, Natalie (January 9, 2009). "Colbie Caillat has 'Breakthrough' with sophomore CD". Reading Eagle. Archived from the original on December 21, 2013. Retrieved August 17, 2012.
 "Taylor Swift's songwriting: how the star's music has changed, for better or worse". CBC News. Retrieved February 2, 2021.
 Knibbs, Kate (August 21, 2019). "Ten Years of Taylor Swift: How the Pop Star Went From Sweetheart to Snake (and Back Again?)". The Ringer. Retrieved December 13, 2021.
 Stubbs, Dan (October 9, 2015). "Taylor Swift: Power, Fame And The Future – The Full NME Cover Interview". NME. Retrieved February 17, 2021.
 Weber, Theon (November 3, 2010). "The Iceberg Songs of Taylor Swift". The Village Voice. Archived from the original on November 4, 2015. Retrieved July 30, 2012.
 Beck, Julia (October 27, 2014). "Taylor Swift Is So Much More Fun Now That She's Jaded". The Atlantic. Archived from the original on September 27, 2016. Retrieved October 30, 2021.
 Willman, Chris (October 10, 2010). "Princess Crossover". New York. Archived from the original on July 27, 2013. Retrieved July 1, 2012.
 Rosen, Jody (November 13, 2008). "Fearless". Rolling Stone. Archived from the original on August 15, 2012. Retrieved July 1, 2012.
 Powers, Ann (October 30, 2014). "The Many New Voices of Taylor Swift". NPR. Retrieved June 2, 2022.
 Stoeffel, Kat (November 16, 2012). "Stop Asking Taylor Swift to Apologize for Writing Songs About Ex-Boyfriends – The Cut". New York. Archived from the original on November 27, 2012. Retrieved February 25, 2013.
 Raven, Robin (March 16, 2022). "10 Artists Who Have Stood Up For Women In Music: Taylor Swift, Lizzo & More". The Recording Academy.
 "Cover Preview: Taylor Swift Fights Back About Her Love Life, the Hyannis Port House—and Has Words for Tina Fey and Amy Poehler". Vanity Fair. March 5, 2013. Archived from the original on August 8, 2016. Retrieved August 3, 2016.
 Dominus, Susan (November 16, 2012). "The Many Insecurities of Taylor Swift". The New York Times. Archived from the original on June 17, 2016.
 Doyle, Patrick (November 13, 2020). "Musicians on Musicians: Paul McCartney and Taylor Swift". Rolling Stone. Archived from the original on November 30, 2020. Retrieved February 7, 2021.
 Olivier, Bobby (December 11, 2020). "Taylor Swift's 'Evermore' Is an Undeniable Folk-Pop Masterpiece". Spin. Retrieved February 1, 2021.
 Shutler, Ali (October 9, 2022). "Taylor Swift organises her lyrics into three 'dorky' pen-themed categories". NME. Retrieved October 18, 2022.
 Browne, Erin (October 21, 2022). "All of Taylor Swift's Famously Devastating Track 5's, Ranked". Vulture. Retrieved October 27, 2022.
 "NMPA to Honor Taylor Swift with Songwriter Icon Award Among Other 2021 Annual Meeting Honorees". National Music Publishers' Association. May 24, 2021. Retrieved May 24, 2021.
 Linker, Damon (November 26, 2021). "Taylor Swift, Phoebe Bridgers, and Rihanna: How women took over songwriting". The Week. Retrieved November 28, 2021.
 Greco, Patti (November 13, 2017). "A Harvard Professor Critiques Taylor Swift's New Poems". Cosmopolitan. Retrieved December 21, 2021.
 Sheffield, Rob (October 13, 2023). "Taylor Swift's 'Eras Tour' Movie Will Make You Sing, Scream, and Sob". Rolling Stone. Retrieved December 8, 2023.
 Zacharek, Stephanie (December 14, 2023). "The Eras Tour Movie Is Irresistible No Matter How Much You Think You Like Taylor Swift". TIME. Retrieved December 8, 2023.
 "Ticketing Shmicketing: Taylor Swift's 'Eras Tour' Debut Slays (And Could Break All-Time Touring Record)". Pollstar. March 18, 2023. Archived from the original on March 20, 2023. Retrieved June 30, 2023.
 Aramesh, Waiss David (March 18, 2023). "Taylor Swift's The Eras Tour Is a 3-Hour Career-Spanning Victory Lap". Rolling Stone. OCLC 1787396. Archived from the original on March 18, 2023. Retrieved June 30, 2023.
 Gambles, Sarah (July 23, 2023). "The ubiquitous power of Taylor Swift". Deseret News. Retrieved July 23, 2023.
 McCormick, Neil (March 18, 2023). "Taylor Swift: The Eras Tour, review: a roaring spectacle of a comeback". The Daily Telegraph. Archived from the original on March 18, 2023. Retrieved March 18, 2023.
 Kornhaber, Spencer (March 18, 2023). "What Made Taylor Swift's Concert Unbelievable". The Atlantic. Retrieved June 30, 2023.
 Foggatt, Tyler (June 3, 2023). "Look What Taylor Made Us Do". The New Yorker. Retrieved June 30, 2023.
 Seibert, Brian (August 9, 2023). "How to Command a Stage Without Great Dance Moves (Taylor's Version)". The New York Times. Retrieved August 11, 2023.
 Krelenstein, Greg (May 21, 2018). "TAYLOR SWIFT'S REPUTATION TOUR IS A POP TRIUMPH". V. Archived from the original on May 22, 2018. Retrieved May 22, 2018.
 Frere-Jones, Sasha (November 3, 2008). "Prodigy". The New Yorker. Retrieved June 30, 2023.
 Horton, Adrian (March 18, 2023). "Taylor Swift review – pop's hardest-working star gives Eras tour her all". The Guardian. Archived from the original on March 18, 2023. Retrieved March 18, 2023.
 Kaplan, Ilana (March 18, 2023). "Taylor Swift's 'Eras' tour is a thrilling spectacle from a pop mastermind". i. Archived from the original on March 18, 2023. Retrieved March 18, 2023.
 Young, Alex (March 27, 2023). "Taylor Swift's "The Eras Tour" Is a Triumph of Spectacle and Stamina: Review". Consequence. Retrieved June 30, 2023.
 Lipshutz, Jason (March 18, 2023). "The 13 Best Moments From Taylor Swift's Eras Tour Kickoff". Billboard. Retrieved July 1, 2023.
 O'Connor, Roisin (June 8, 2018). "Taylor Swift 'reputation' stadium tour review: Dazzling pop spectacle from the star who doesn't stand still". The Independent. Archived from the original on June 30, 2023. Retrieved December 22, 2019.Savage, Mark (March 18, 2023). "Taylor Swift launches Eras tour with three-hour, 44-song set". BBC News. Archived from the original on March 18, 2023. Retrieved March 18, 2023. Sisario, Ben (November 5, 2023). "How Taylor Swift's Eras Tour Conquered the World". The New York Times. Retrieved August 12, 2023. DeVille, Chris (July 12, 2018). "Big Reputation: A Trip To Taylor Swift's Hyper-Maximalist Stadium Tour". Stereogum. Retrieved June 30, 2023.
 Procell, Carlie; Padilla, Ramon (April 28, 2023). "Taylor Swift tour has many 'eras.' We tracked her movements to give you the look and feel". USA Today. Retrieved June 30, 2023.
 "Taylor Swift Shares Stunning 'Wildest Dreams' Performance from Grammy Museum". Billboard. January 5, 2016. Retrieved October 29, 2023.
 Burgham, Lydia (November 10, 2018). "Taylor Swift in Auckland, reviewed: Despite the snakes, her Reputation shines on". The Spinoff. Retrieved December 10, 2019.
 Dodd, Sophie (November 15, 2023). "All About Taylor Swift's Parents, Scott and Andrea Swift". People. Retrieved December 3, 2023.
 Swift, Taylor (March 15, 2013). ""Sparks Fly" (acoustic) Live on the RED Tour!" – via YouTube.
 Lewis, Randy (April 3, 2011). "Academy of Country Music Awards: Las Vegas welcomes Miranda Lambert, Taylor Swift with open arms". Los Angeles Times. Retrieved August 20, 2023.
 Ritchie, Mike (March 8, 2020). "Why Taylor Swift is making the ukulele cool again". The Herald. Retrieved July 1, 2023.
 Gensler, Andy (August 17, 2023). "The Showgoer: The Greatest Show On Earth — Taylor Swift's 'Eras Tour' — Is All That And Far More". Pollstar. Retrieved August 19, 2023.
 Sheffield, Rob (May 9, 2018). "Why Taylor Swift's 'Reputation' Tour Is Her Finest Yet". Rolling Stone. Retrieved June 30, 2023.
 Willman, Chris (May 16, 2018). "Taylor Swift's 'Reputation' Tour Has Bad Blood, Good Will, Sex Appeal and Serpents". Variety. Retrieved December 22, 2019.
 Willman, Chris (March 18, 2023). "Taylor Swift's 'Eras' Show Is a Three-Hour, 44-Song Epic That Leaves 'Em Wanting More: Concert Review". Variety. Archived from the original on March 18, 2023. Retrieved June 30, 2018.
 Ordoña, Michael (September 9, 2022). "Taylor Swift wants an Oscar. So she took 'All Too Well' to TIFF". Los Angeles Times. Retrieved September 10, 2022.
 CMT.com Staff (May 4, 2011). "Taylor Swift's "Mean" Video Debuts Friday". CMT. Archived from the original on June 19, 2019. Retrieved June 19, 2019.
 Anitai, Tamar (August 27, 2010). "Video Premiere: Taylor Swift, 'Mine'". MTV News. Archived from the original on April 29, 2019. Retrieved June 19, 2019.
 Bonaguro, Alison (May 6, 2011). "OFFSTAGE: Taylor Swift Isn't 'Mean' at All, Director Says". CMT News. Archived from the original on June 19, 2019. Retrieved June 20, 2019.
 Tailor, Leena (September 1, 2017). "Exclusive: Taylor Swift's Director Joseph Kahn on How Her Image Invokes a Double Standard: 'She's a Genius'". Entertainment Tonight. Archived from the original on June 19, 2019.
 O'Connell, Michael (October 9, 2015). "Taylor Swift and Jimmy Fallon Among Early Emmy Winners". The Hollywood Reporter. Archived from the original on June 19, 2019.
 Forbes, Jihan (May 14, 2015). "Peep Taylor Swift's Star-Studded Cast for Her 'Bad Blood' Music Video". The Fashion Spot. Retrieved May 13, 2020.
 "9 Things You Might Have Missed in Taylor Swift's Netflix Concert Film". E! News. December 31, 2018. Retrieved September 10, 2022.
 Spanos, Brittany (April 25, 2019). "Watch Taylor Swift, Brendon Urie's Colorful 'ME!' Video". Rolling Stone. Archived from the original on April 26, 2019. Retrieved April 25, 2019.
Moore, Sam (August 23, 2019). "Watch Taylor Swift's colourful new video for 'Lover'". NME. Archived from the original on August 27, 2019. Retrieved August 27, 2019.
Zemler, Emily (June 17, 2019). "Watch Taylor Swift Reunite With Katy Perry in 'You Need to Calm Down' Video". Rolling Stone. Archived from the original on June 17, 2019. Retrieved June 17, 2019.
 Mylrea, Hannah (February 28, 2020). "Every incredible Easter Egg in Taylor Swift's 'The Man' video". NME. Retrieved March 9, 2020.
Spanos, Brittany; Legaspi, Althea (July 24, 2020). "Taylor Swift Blends Fantastical With Personal in 'Cardigan' Video". Rolling Stone. Retrieved July 27, 2020.
"Justin Bieber & Megan Thee Stallion Lead 2021 MTV VMA Nominations". Billboard. Retrieved August 17, 2021.
 Weatherby, Taylor (February 5, 2023). "Taylor Swift Makes GRAMMY History (Again) With Best Music Video Win For "All Too Well: The Short Film"". Grammy Awards. Retrieved February 6, 2023.
 Lansky, Sam (December 6, 2023). "Taylor Swift Is TIME's 2023 Person of the Year". Time. Retrieved December 6, 2023.
 "Taylor Swift". The Recording Academy. Archived from the original on August 12, 2016. Retrieved August 3, 2016.
 Friedlander, Whitney (September 10, 2015). "Taylor Swift, Jimmy Fallon Among Juried Emmy Award Winners". Variety. Archived from the original on September 15, 2015. Retrieved August 3, 2016.
 "Taylor Swift dominates AMAs with 6 wins, extending lead as show's most-decorated artist". KTRK-TV. November 21, 2022. Retrieved November 21, 2022.
 Grein, Paul (November 19, 2023). "After the 2023 Billboard Music Awards, Who Is the All-Time Biggest Winner?". Billboard. Retrieved November 19, 2023.
 See Guinness World Records by Taylor Swift
 Lewis, Randy (November 4, 2013). "Taylor Swift to receive rare Pinnacle Award at CMA Awards Nov. 6". Los Angeles Times. Retrieved May 13, 2020.
 "Taylor Swift Nashville Tickets". Excite. Archived from the original on February 3, 2015. Retrieved February 2, 2015.
 Shelburne, Craig (October 18, 2010). "Taylor Swift Named NSAI's Songwriter-Artist of the Year". CMT. Archived from the original on March 14, 2014. Retrieved February 2, 2015.
 "Songwriters Hall of Fame". Songwriters Hall of Fame. Archived from the original on November 29, 2014. Retrieved February 2, 2015.
 "The 100 Greatest Songwriters of All Time". Rolling Stone. Archived from the original on September 2, 2017. Retrieved August 28, 2017.
 Polanco, Luis (April 5, 2016). "Taylor Swift to Receive First-Ever Taylor Swift Award From BMI". Billboard. Retrieved October 21, 2020.
 Jolly, Nathan (November 17, 2019). "Why Taylor Swift is to blame for latest twist in music rights drama". News.com.au. Retrieved November 17, 2019.
 "10 Life mantras by Taylor Swift to live by". India Today. December 13, 2016. Archived from the original on February 9, 2019. Retrieved July 16, 2020.
 Lipshutz, Jason (December 11, 2019). "Billboard Woman of the Decade Taylor Swift: 'I Do Want My Music to Live On'". Billboard. Retrieved December 11, 2019.
 "Taylor Swift to receive BRITs Global Icon award". Official Charts Company. May 9, 2021. Retrieved May 10, 2021.
 "Taylor Swift to receive Global Icon Award!". Brit Awards. May 9, 2021. Retrieved May 10, 2021.
 Paine, Andre (February 22, 2023). "Taylor Swift wins IFPI's 2022 Global Recording Artist Of The Year Award". Music Week. Retrieved February 22, 2023.
 Brandle, Lars (November 5, 2023). "Taylor Swift's '1989 (Taylor's Version)' Debuts at U.K. No. 1 With 'Massive' Sales". Billboard. Retrieved November 5, 2023.
 "Female artists with the most Irish Number 1 albums since 2000". Official Charts Company. November 5, 2020. Retrieved February 23, 2021.
 Wang, Dennis (April 16, 2021). "Taylor Swift's Fearless hits the right note in China, again". People's Daily. Retrieved June 26, 2021.
 Brandle, Lars (July 7, 2023). "Taylor Swift Sets Chart Record In Australia With Top-Five Sweep". Billboard. Retrieved February 13, 2024.
 Brandle, Lars (February 9, 2024). "Ahead of 'The Eras Tour' of Australia, Taylor Swift Sweeps Top 5". Billboard. Retrieved February 13, 2024.
 "Taylor beats Swift". Australian Recording Industry Association. July 14, 2023. Archived from the original on July 14, 2023. Retrieved July 19, 2023.
 "Taylor sweeps the record". Australian Recording Industry Association. July 7, 2023. Retrieved July 7, 2023.
 Cumulative touring gross:
"Top Touring Artist of the Pollstar Era" (PDF). Pollstar. June 10, 2022. Archived (PDF) from the original on August 5, 2022. Retrieved August 4, 2022.
Allen, Bob (September 26, 2023). "What A Friggin' Year! 2023 Boxoffice Results Remain At Record Highs". Pollstar. Retrieved October 18, 2023.
 Gensler, Andy (December 8, 2023). "Taylor Swift Sets All-Time Touring Record With $1 Billion Gross". Pollstar. Archived from the original on December 8, 2023. Retrieved December 8, 2023.
 Willman, Chris (December 21, 2020). "Taylor Swift's 'Evermore' Sells a Million Worldwide in First Week". Variety. Archived from the original on January 11, 2021. Retrieved December 21, 2020.
 Paine, Andre (December 22, 2022). "Taylor Swift Achieves More Than 6 Million Global Units for Midnights and 37 Billion Total Streams in 2022". Music Week. Archived from the original on December 22, 2022. Retrieved July 19, 2023.
 Young, Alex (October 31, 2022). "Taylor Swift broke 73 records with release of new album Midnights". Consequence. Archived from the original on October 31, 2022. Retrieved July 19, 2023.
 Grein, Paul (November 8, 2023). "Taylor Swift Is Apple Music's 2023 Artist of the Year". Billboard. Retrieved November 8, 2023.
 Willman, Chris (October 28, 2023). "Taylor Swift Beats Her Own Spotify Record for Most Single-Day Streams for an Artist With '1989 (Taylor's Version)' Release". Variety. Archived from the original on October 29, 2023. Retrieved October 29, 2023.
 @billboardcharts (January 19, 2022). "Most entries on the #Global200 chart in a single week" (Tweet) – via Twitter.
 "Taylor Swift Chart History (Billboard Global 200)". Billboard. Retrieved July 17, 2023.
 "Greatest of All Time Artists". Billboard. Archived from the original on November 14, 2019. Retrieved November 15, 2019.
 Zellner, Xander (January 25, 2024). "Taylor Swift Tallies Record-Extending 95th Week at No. 1 on Artist 100 Chart". Billboard. Retrieved January 26, 2024.
 Caulfield, Keith (December 31, 2023). "Taylor Swift Surpasses Elvis Presley for Most Weeks at No. 1 on Billboard 200 Among Soloists". Billboard. Retrieved January 3, 2024.
 Trust, Gary (January 22, 2024). "Ariana Grande's 'Yes, And?' Debuts at No. 1 on Billboard Hot 100". Billboard. Retrieved January 22, 2024.
 "Taylor Swift Chart History (Top Country Albums)". Billboard. Retrieved July 17, 2023.
 "Taylor Swift Chart History (Digital Song Sales)". Billboard. Retrieved January 14, 2021.
 Trust, Gary (July 28, 2023). "Taylor Swift Breaks Record for Most No. 1s on Pop Airplay Chart As 'Cruel Summer' Becomes Her 12th". Billboard. Archived from the original on September 29, 2023. Retrieved October 19, 2023.
 Caulfield, Keith (December 3, 2023). "Taylor Swift Makes History With Five of the Top 10 Albums on the Billboard 200". Billboard. Retrieved January 3, 2024.
 Caulfield, Keith (July 17, 2023). "Taylor Swift Has 11 Albums on the Billboard 200 Chart for the First Time". Billboard. Retrieved July 19, 2023.
 McIntyre, Hugh (January 22, 2024). "Taylor Swift Made Billboard History–Now Only She Can Match Her Own Feat". Forbes. Retrieved January 22, 2024.
 Caulfield, Keith (December 29, 2023). "Taylor Swift Has the Top Four on the Album Sales Chart for the First Time". Billboard. Retrieved January 3, 2024.
 Caulfield, Keith (January 3, 2024). "Taylor Swift Has 7 of the Top 10 on Billboard's Album Sales Chart". Billboard. Retrieved January 23, 2024.
 "RIAA – Top Artists (Digital Singles)". Recording Industry Association of America. Retrieved July 17, 2023.
 Ahlgrim, Callie (July 12, 2023). "There are 116 songs in history that have been certified diamond — here they all are". Insider Inc. Retrieved July 17, 2023.
 Caulfield, Keith (October 30, 2022). "Taylor Swift's Midnights Blasts in at No. 1 on Billboard 200 Chart With Biggest Week for an Album in 7 Years". Billboard. Retrieved October 30, 2022.
 Nicks, Stevie (April 29, 2010). "The 2010 Time 100: Taylor Swift". Time. Archived from the original on November 29, 2013. Retrieved April 22, 2012.
Vena, Jocelyn (April 16, 2015). "Taylor Swift, Kanye West, Kim Kardashian Make Time's 100 Most Influential People List". Billboard. Archived from the original on September 25, 2016. Retrieved September 1, 2016.
Jensen, Erin (April 17, 2019). "Dwayne Johnson, Taylor Swift, Gayle King, more cover Time's 100 most influential people issue". USA Today. Archived from the original on April 17, 2019. Retrieved April 17, 2019.
 Dockterman, Eliana (December 6, 2017). "'I Was Angry.' Taylor Swift on What Powered Her Sexual Assault Testimony". Time. Archived from the original on December 6, 2017. Retrieved December 7, 2017.
 Shah, Simmone (December 6, 2023). "Taylor Swift Makes History as Person of the Year". Time. Retrieved December 6, 2023.
 "Taylor Swift Is TIME's 2023 Person of the Year". Time. December 6, 2023. Retrieved December 11, 2023.
 "Taylor Swift". Forbes. Retrieved December 1, 2020.
 "30 Under 30 2017: All-Star Alumni". Forbes. Retrieved December 1, 2020.
 "#8 Taylor Swift". Forbes. Archived from the original on July 28, 2015. Retrieved July 27, 2015.
 "Forbes Unveils Its 20th Annual Ranking Of The World's Most Powerful Women". Forbes. December 5, 2023. Retrieved December 5, 2023.
 Schneider, Marc (July 24, 2023). "8 Ways Taylor Swift Has Changed the Music Business". Billboard. Retrieved July 24, 2023.
 Khan, Fawzia (June 18, 2021). "The Might Of Taylor Swift". Elle. Archived from the original on June 28, 2021. Retrieved October 20, 2021.
 Pattison, Kate (September 19, 2023). "How did Taylor Swift get so popular? She never goes out of style". The Conversation. Retrieved September 22, 2023.
 Grady, Constance (October 12, 2023). "The Eras concert movie is Taylor Swift leveling up". Vox. Retrieved October 13, 2023.
 Jericho, Greg (October 28, 2022). "Taylor Swift's incredible success in graphs – who can blame me for being a Swiftie as a 50-year-old man?". The Guardian. Retrieved October 29, 2022.
 Weatherhead, Shaina (April 20, 2023). "Taylor Swift's 'Miss Americana' Is Required Viewing Right Now". Collider. Retrieved April 24, 2023.
 Bartsch, Kayla (September 24, 2023). "Taylor Swift: Empress of the Zeitgeist". National Review. Retrieved September 24, 2023.
 "Taylor Swift: 'My Confidence Is Easy To Shake'". NPR. November 2, 2012. Retrieved May 20, 2021.
Thanki, Juli (September 24, 2015). "Taylor Swift: Teen idol to 'biggest pop artist in the world'". The Tennessean. Retrieved May 13, 2021.
Yahr, Emily (June 16, 2016). "Taylor Swift's first song came out 10 years ago. Here's what she was like as a teen songwriter". Arts and Entertainment. The Washington Post. Archived from the original on May 13, 2021. Retrieved May 13, 2021.
 El Mosselli, Sabrina (December 24, 2022). "How Taylor Swift rose from 'Miss Americana' to global megastar". Le Monde. Retrieved December 28, 2022.
 Cragg, Michael. "Is Taylor Swift our last remaining real popstar?". i-D. Archived from the original on May 6, 2023. Retrieved December 3, 2022.
 Seisdedos, Iker (December 27, 2022). "Pop music in the era of Taylor Swift: Behind the success of today's biggest star". El País. Retrieved December 28, 2022.
 Sanders, Sam (October 20, 2022). "Taylor Swift Is the Greatest Self-Portraitist of Our Time". Vulture. Retrieved October 21, 2022.
 "Taylor Swift's 40 Biggest Hot 100 Hits". Billboard. March 23, 2022. Retrieved April 12, 2022.
 Leonard, Devin (November 12, 2014). "Taylor Swift Is the Music Industry". Bloomberg News. Archived from the original on March 13, 2016. Retrieved August 11, 2020.
 Zoladz, Lindsay (December 30, 2019). "What Were the 2010s?". New York. Retrieved November 22, 2020.
 Savaira, Augusta (November 23, 2022). "Welcome to 'Swiftonomics': What Taylor Swift reveals about the U.S. economy". Los Angeles Times. Retrieved November 23, 2022.
 Vincent, Alice (November 3, 2017). "Taylor Swift: the rise, fall and re-invention of America's sweetheart". The Daily Telegraph. Archived from the original on January 10, 2022. Retrieved July 5, 2020.
 Fischer & Seidman 2016, p. 529.
 Wahi, Sukriti (March 3, 2021). "Every Time Taylor Swift Perfectly Shut Down A Sexist Interview Question". Elle. Retrieved November 4, 2021.
 Davis, Allison P. (June 28, 2018). "The Taylor Swift Slut-Shaming Continues". The Cut. Retrieved November 4, 2021.
 Meddings, Jacqui (October 31, 2014). "Taylor Swift is our new cover goddess". Cosmopolitan. Archived from the original on July 14, 2016. Retrieved August 2, 2016.
 Chiu, Allyson (August 3, 2022). "Celebrities use private jets excessively. It's a climate nightmare". Washington Post. ISSN 0190-8286. Retrieved February 14, 2024.
 Mendez, Lola (February 14, 2024). "Taylor Swift claims she offsets her travel carbon footprint - how does that work?". BBC. Retrieved February 14, 2024.
 O'Malley, Isabella (February 2, 2024). "Why Taylor Swift's globe-trotting in private jets is getting scrutinized". Associated Press. Retrieved February 6, 2024.
 "Taylor Swift and Travis Kelce romance is bad for the planet. Couple burns a whopping $70,779 jet fuel in the last three months". The Economic Times. December 21, 2023. Retrieved February 6, 2024.
 Blistein, Jon (February 6, 2024). "Taylor Swift's Lawyers Threaten Private Jet Flight Tracker Over 'Stalking and Harassing Behavior'". Rolling Stone. Retrieved February 6, 2024.
 Matza, Max (February 7, 2024). "Taylor Swift threatens to sue student who tracks her private jet". BBC News. Retrieved February 7, 2024.
 Benitez-Eves, Tina (July 2, 2022). "Man Arrested for Harassing Taylor Swift, Attempting to Break Into Her New York City Apartment". American Songwriter. Retrieved June 10, 2023.
 Kreps, Daniel (July 2, 2022). "Brooklyn Man Arrested on Charges of Stalking Taylor Swift". Rolling Stone. Retrieved June 10, 2023.
 Della Cava, Marco; Ryan, Patrick (August 31, 2023). "Taylor Swift is 'in a class of her own right now,' as Eras tour gives way to Eras movie". USA Today. Archived from the original on September 1, 2023. Retrieved September 1, 2023.
 Crummy, Colin (March 13, 2014). "Detwanging country music: how Nashville took the UK". The Guardian. Retrieved September 23, 2021.
 Roland, Tom (July 7, 2016). "Love Story: The Impact of Taylor Swift's First Decade in Music". Billboard. Archived from the original on July 14, 2016. Retrieved July 7, 2016.
 Tannembaum, Rob (September 12, 2014). "4 Ways Billboard Woman of the Year Taylor Swift Changed Country Music". Billboard. Retrieved December 31, 2019.
 O'Connor, Roisin (October 18, 2018). "Half of all new guitar players are women, finds study". The Independent. Archived from the original on May 26, 2022. Retrieved October 20, 2020.
 Wang, Amy X. (May 22, 2018). "Guitars Are Getting More Popular. So Why Do We Think They're Dying?". Rolling Stone. Retrieved October 20, 2020.
 "The 200 Most Important Artists of Pitchfork's First 25 Years". Pitchfork. October 4, 2021. Retrieved October 4, 2021.
 Lipshutz, Jason (December 16, 2021). "Billboard's Greatest Pop Stars of 2021: No. 1 — Taylor Swift". Billboard. Retrieved December 16, 2021.
 Catucci, Nick (August 23, 2019). "Taylor Swift Reaches For New Heights of Personal and Musical Liberation on 'Lover'". Rolling Stone. Retrieved August 18, 2021.
 Gormely, Ian (December 3, 2014). "Taylor Swift leads poptimism's rebirth". The Guardian. Retrieved August 2, 2021.
 Gambles, Sarah (July 23, 2023). "The ubiquitous power of Taylor Swift". Deseret News. Retrieved September 7, 2023.
 Friedman, Vanessa (April 24, 2023). "Taylor Swift and the Sparkling Trap of Constant Reinvention". The New York Times. Retrieved September 7, 2023.
 Rowley, Glenn (March 9, 2023). "Taylor Swift's 'All Too Well (10 Minute Version)' Is Now a Class at Stanford University". Billboard. Retrieved March 10, 2023.
 Ahlgrim, Callie (March 18, 2022). "How Taylor Swift inspired a new generation of singer-songwriters, in their own words". Business Insider. Retrieved March 18, 2022.
 Sutherland, Mark (December 13, 2019). "'I come with opinions about how we can better our industry': Taylor Swift". Music Week. Retrieved April 27, 2020.
Sisario, Ben (November 6, 2014). "Sales of Taylor Swift's '1989' Intensify Streaming Debate". The New York Times. Retrieved August 11, 2020.
Rosseinsky, Katie (October 29, 2021). "How Taylor Swift is changing the music industry a re-record at a time". Evening Standard. Archived from the original on April 9, 2021. Retrieved April 23, 2021.
 Smith, Neil (June 22, 2015). "Five ways Taylor Swift is changing the world". BBC News. Retrieved December 31, 2019.
 Brooks, Dave (November 3, 2022). "How Taylor Swift Changed the Course of the Concert Ticketing Businesses". Billboard. Retrieved November 4, 2022.
 Sources on Swift and millennial era
 • McDuling, John (November 3, 2014). "Why Taylor Swift is the most important artist of the millennial era". Quartz. Archived from the original on June 2, 2021. Retrieved April 2, 2021.
 • "What Taylor Swift tells us about millennials". National Post. October 6, 2015. Retrieved October 30, 2021.
 • Kidd, Joel (March 2, 2020). "Taylor Swift Is the Surprising Face of Millennial Anxiety". The Walrus. Retrieved October 30, 2021.
 • Winograd, Morley; D. Hais, Michael (January 29, 2010). "Move over Kanye West, Taylor Swift and the Millennial generation are taking over music". The Christian Science Monitor. Retrieved October 30, 2021.
 • "The 17 Musicians Gen Z & Millennials Say Represent Their Generations". YPulse. October 8, 2019. Retrieved October 30, 2021.
 • Nicolau, Elena (November 29, 2022). "How Taylor Swift is transforming millennial weddings". Today. Retrieved November 30, 2022.
 • "Taylor Swift is peak millennial vibes". NPR. October 21, 2022. Retrieved December 19, 2022.
 VanDerWerff, Emily (July 31, 2020). "Taylor Swift is the millennial Bruce Springsteen". Vox. Retrieved October 30, 2021.
 Ditum, Sarah (October 8, 2022). "Taylor Swift: the Bob Dylan of our age". The Times. Archived from the original on December 24, 2022. Retrieved October 9, 2022.
 "Taylor Swift Will Receive First-Ever Woman of the Decade Honor at Billboard's Women in Music". Billboard. November 18, 2019. Retrieved December 31, 2019.
 Elizabeth, De (November 24, 2019). "Taylor Swift Is Named Artist of the Decade at AMAs 2019". Teen Vogue. Retrieved December 31, 2019.
 Kemp, Sam (January 27, 2022). "The surprising Paul McCartney song inspired by Taylor Swift". Far Out. Retrieved July 5, 2023.
 Golsen, Tyler (May 14, 2023). "Mick Jagger discusses his performance with Taylor Swift". Far Out. Retrieved July 5, 2023.
 Brandle, Lars (January 28, 2015). "Taylor Swift on Madonna Praise: 'Now I'm Dead'". Billboard. Retrieved July 5, 2023.
 Aniftos, Rania (January 27, 2021). "Dolly Parton Says Taylor Swift Is 'Magnificent' Following Damon Albarn Feud: 'She Knows Who She Is'". Billboard. Retrieved July 5, 2023.
 Graff, Gary (October 31, 2021). "Taylor Swift Helps Induct Carole King, Sings "Will You Love Me Tomorrow" at Rock Hall Ceremony". The Hollywood Reporter. Retrieved September 20, 2022.
 Skinner, Tom (November 2, 2022). "Bruce Springsteen praises Taylor Swift's new album 'Midnights': "She's a tremendous writer"". NME. Retrieved July 5, 2023.
 Ruggieri, Melissa (November 3, 2021). "Billy Joel says Taylor Swift is like the Beatles of her generation: 'She knows music'". USA Today. Retrieved July 5, 2023.
 Williams, Joe (July 5, 2023). "Ringo Starr says Taylor Swift is "biggest star in the world"". Far Out. Retrieved July 5, 2023.
 Iasimone, Ashley (November 12, 2023). "Britney Spears Shares Throwback Photo With Taylor Swift From 20 Years Ago: 'Most Iconic Pop Woman of Our Generation'". Billboard. Retrieved November 12, 2023.
 Lutz, Ashley (October 8, 2022). "Taylor Swift's new album proves she is a business genius". Fortune. Retrieved October 8, 2022.
 Steele, Anne (July 1, 2023). "How to Succeed in Business Like Taylor Swift". The Wall Street Journal. Retrieved July 2, 2023.
 Lynch, Joe (January 31, 2024). "Taylor Swift Tops Billboard's Annual Power 100 List for the First Time". Billboard. Retrieved January 31, 2024.
 Curto, Justin (December 22, 2020). "Did 2020 Kill the Long, Fancy Pop-Album Rollout for Good?". Vulture. Archived from the original on December 23, 2020. Retrieved December 27, 2020.
 Leight, Elias (July 23, 2020). "Taylor Swift Finally Abandoned the Traditional Album Rollout". Rolling Stone. Retrieved September 15, 2022.
 Kheraj, Alim (November 9, 2022). "The curate's Easter egg: how Taylor Swift turned pop into a multiplayer puzzle". The Guardian. Retrieved November 9, 2022.
 Tannenbaum, Emily. "Taylor Swift Hid a Bunch of Easter Eggs in the All Too Well Short Film". Glamour. Retrieved May 13, 2022.
 Suskind, Alex (May 9, 2019). "New Reputation: Taylor Swift shares intel on TS7, fan theories, and her next era". Entertainment Weekly. Archived from the original on August 12, 2019. Retrieved May 13, 2022.
 Yahr, Emily (December 26, 2022). "2022: The year in review (Taylor's version)". The Washington Post. Retrieved December 26, 2022.
 Stein, Jeannine (November 18, 2011). "Taylor Swift weighs in on being a role model". Los Angeles Times. Archived from the original on July 15, 2012. Retrieved April 18, 2012.
 Collins, Katie (June 8, 2018). "How Taylor Swift flipped online fandom on its head for the better". CNET. Retrieved March 19, 2023.
 King, Ashley (July 21, 2023). "Taylor Swift's 13 Management is Rarely Discussed, Changes Abound". Digital Music News. Retrieved September 23, 2023.
 Yotka, Steff (August 20, 2019). "Taylor Swift and Stella McCartney Reveal Their Lover Fashion Collaboration". Vogue. Retrieved October 20, 2020.
 Kaplan, Julee (January 29, 2009). "Taylor Swift Launches Sundress Line – Ready-to-Wear and Sportswear – Media". Women's Wear Daily. Archived from the original on July 29, 2013. Retrieved August 7, 2012.
 "Taylor Swift to have greeting card line". USA Today. November 18, 2009. Archived from the original on December 14, 2009. Retrieved August 7, 2012.
 Serpe, Gina (October 29, 2008). "Taylor Swift Gets All Dolled Up". E! News (Blog). Archived from the original on March 5, 2011. Retrieved September 20, 2010.
 Mychaskiw, Marianne (October 1, 2014). "Taylor Swift Is Launching Her Fourth Fragrance!". InStyle. Archived from the original on January 7, 2015. Retrieved December 3, 2015.
 McIntyre, Hugh. "Taylor Swift Has Signed A Multi-Year Deal With AT&T". Forbes. Archived from the original on September 11, 2018. Retrieved September 11, 2018.
 Chiu, Melody (July 26, 2019). "Watch Taylor Swift Hilariously Moonlight — and Fail — as a Bartender and Server in New Commercial". People. Retrieved May 7, 2022.
 "Taylor Swift: NHL's New Spokesperson". AOL. February 3, 2009. Archived from the original on February 7, 2009. Retrieved September 20, 2010.
 "Sony Electronics News and Information". Sony. April 26, 2010. Retrieved August 7, 2012.
 Lee, Ashley (October 27, 2014). "Taylor Swift Named New York City Welcome Ambassador". Billboard. Retrieved September 26, 2023.
 Kaufman, Gil (January 20, 2022). "Taylor Swift Tapped as First Record Store Day Global Ambassador". Billboard. Retrieved January 20, 2022.
 Snapes, Laura (August 24, 2019). "Taylor Swift: 'Pop music can feel like it's The Hunger Games, and like we're gladiators'". The Guardian. Archived from the original on August 24, 2019. Retrieved August 24, 2019.
 "The Original 300+ Time's Up Signatories". Time's Up. October 21, 2019. Archived from the original on May 3, 2020. Retrieved May 20, 2020.
 Mier, Tomás (June 24, 2022). "Taylor Swift Says She's 'Absolutely Terrified' After Roe v. Wade Decision". Rolling Stone. Retrieved June 24, 2022.
 "Taylor Swift sends 'love' and 'respect' to LGBT fans". BBC News. June 4, 2018. Archived from the original on November 14, 2018. Retrieved December 15, 2018.
 Kreps, Daniel (June 1, 2019). "Taylor Swift Urges Senate to Pass Equality Act in Letter to Tennessee Senator". Rolling Stone. Retrieved August 12, 2020.
 Lynch, Joe (June 26, 2020). "Taylor Swift Calls Out U.S. Census for Transgender Erasure". Billboard. Retrieved June 29, 2020.
 Lynch, Joe (June 14, 2019). "Taylor Swift Wishes NYC's Stonewall Inn 'Happy Pride' at Surprise Set". Billboard. Retrieved August 7, 2022.
 Aniftos, Rania (April 8, 2019). "Taylor Swift Donates $113,000 to Tennessee Equality Project to Fight Anti-LGBTQ Bills". Billboard. Archived from the original on April 9, 2019. Retrieved April 9, 2019.
 "Taylor Swift Makes a Generous Donation to GLAAD in Support of the LGBTQ Community" (Press release). GLAAD. June 1, 2019. Archived from the original on June 1, 2019. Retrieved June 1, 2019.
 Lynch, John (March 23, 2018). "Taylor Swift Shares Support For Gun a Control Reform In Instagram Post". Business Insider. Retrieved May 20, 2020.
 Hiatt, Brian (September 18, 2019). "The Rolling Stone Interview Taylor Swift". Rolling Stone. Archived from the original on September 18, 2019. Retrieved September 19, 2019.
 Bailey, Alyssa (June 9, 2020). "Taylor Swift Tweeted About the Black Lives Matter Movement and Urged People to Vote". Elle. Retrieved July 27, 2020.
 Kreps, Daniel (June 12, 2020). "Taylor Swift Calls for Removal of Monuments That 'Celebrate Racist Historical Figures'". Rolling Stone. Retrieved August 12, 2020.
 France, Lisa Respers (June 19, 2020). "Taylor Swift got educated on Juneteenth and wants you to be, too". CNN. Retrieved July 27, 2020.
 Haag, Matthew (October 9, 2018). "Voter Registrations Spike as Deadlines Loom. Taylor Swift Had Something to Do With It". The New York Times. Archived from the original on October 9, 2018. Retrieved October 10, 2018.
 Stevens, Matt (October 7, 2020). "Taylor Swift, leaning further into politics, endorses Joe Biden and Kamala Harris". The New York Times. Archived from the original on October 7, 2020. Retrieved October 8, 2020.
 Schuster, Steve (May 19, 2023). "Taylor Swift comes out against Trump, again". Wisconsin Law Journal. Retrieved November 4, 2023.
 Dellatto, Marisa (October 27, 2023). "Taylor Swift's New Era: The Pop Star Becomes A Billionaire". Forbes. Retrieved October 29, 2023.
 Pendleton, Devon; Ballentine, Claire; Patino, Marie; Whiteaker, Chloe; Li, Diana (October 26, 2023). "Taylor Swift Vaults Into Billionaire Ranks With Blockbuster Eras Tour". Bloomberg News. Retrieved October 26, 2023.
 Greenburg, Zack O'Malley (November 2, 2016). "The World's Highest-Paid Women in Music 2016". Forbes. Archived from the original on May 25, 2019. Retrieved July 13, 2019.
Greenburg, Zack O'Malley (December 6, 2019). "The World's Top-Earning Musicians Of 2019". Forbes. Retrieved July 29, 2022.
Voytko, Lisette (February 9, 2022). "The Highest-Paid Entertainers 2022". Forbes. Retrieved July 29, 2022.
Voytko, Lisette (February 13, 2023). "The World's 10 Highest-Paid Entertainers". Forbes. Retrieved July 29, 2022.
 "Highest annual earnings ever for a female pop star". Guinness World Records. Archived from the original on November 12, 2017. Retrieved November 15, 2017.
 Mercuri, Monica (July 10, 2019). "Taylor Swift Is The World's Highest-Paid Celebrity With $185 Million in 2019". Forbes. Archived from the original on July 12, 2019. Retrieved July 13, 2019.
Greenburg, Zack O'Malley (July 11, 2016). "Taylor Swift Is The World's Top-Earning Celebrity With $170 Million in 2016". Forbes. Archived from the original on August 3, 2016. Retrieved August 3, 2016.
 Greenburg, Zack O'Malley (December 23, 2019). "From Taylor Swift To Dr. Dre: The 10 Top-Earning Musicians Of The Decade". Forbes. Archived from the original on December 30, 2019. Retrieved December 31, 2019.
 Silva, Rachel; Tonelli, Lucia (October 3, 2023). "Inside Taylor Swift's 8 Incredible Homes". Elle Decoration. Retrieved October 12, 2023.
 Dailey, Hannah; Aniftos, Rania (December 21, 2023). "A Timeline of Taylor Swift's Generosity". Billboard. Retrieved December 24, 2023.
 Couch, Robbie (March 2, 2015). "Taylor Swift Named Most Charitable Celeb For Supporting Feminist Causes, Education, More". HuffPost. Archived from the original on October 21, 2016. Retrieved August 2, 2016.
 Pacella, Megan (June 13, 2012). "Taylor Swift Receives Star of Compassion Award". Taste of Country. Archived from the original on July 13, 2015. Retrieved November 27, 2015.
 Cooper, Brittany Joy (March 27, 2012). "Taylor Swift to Receive 'Big' Honor From Michelle Obama at Kids' Choice Awards". Taste of Country. Archived from the original on July 12, 2012. Retrieved August 30, 2012.
 Lee, Ken (August 9, 2009). "Taylor Swift donates $100,000 to victims of Iowa Flood". People. Archived from the original on June 7, 2009. Retrieved September 9, 2009.
 "How Taylor Swift helps people worldwide". Borgen. February 2, 2023. Retrieved June 17, 2023.
 Moran, Jonathon (March 8, 2009). "Kylie to play at Sound Relief with Coldplay, Midnight Oil". The Daily Telegraph. Archived from the original on October 10, 2016. Retrieved December 30, 2009.
 "Taylor Swift donates $500,000 after Nashville floods". BBC News. May 7, 2010. Retrieved May 7, 2022.
 Lewis, Randy (May 23, 2011). "Taylor Swift benefit concert raises more than $750,000 for tornado victims". Los Angeles Times (Blog). Archived from the original on April 6, 2012. Retrieved April 18, 2012.
 "Taylor Swift gives $1m to help Louisiana flood relief efforts". The Guardian. August 17, 2016. Archived from the original on August 17, 2016. Retrieved August 17, 2016.
 Muhammad, Latifah (December 9, 2016). "Taylor Swift Donates $100,000 to Dolly Parton Fire Fund". Entertainment Tonight. Archived from the original on January 13, 2017. Retrieved January 11, 2017.
 Blair, Olivia (September 4, 2019). "Taylor Swift Makes 'Very Sizeable Donation' To A Food Bank in Houston After Hurricane Harvey". Elle. Archived from the original on July 2, 2019. Retrieved July 3, 2019.
 Rowley, Glenn (April 13, 2023). "Taylor Swift Donated Enough to Feed 'Over 125,000' at Tampa Food Bank Ahead of The Eras Tour Stop". Billboard. Archived from the original on April 21, 2023. Retrieved May 11, 2023.
 Dementri, Nikki (May 16, 2023). "'It's a blessing': South Philly pizza, cheesesteak shop thankful for T. Swift's business". CBS News. Archived from the original on May 17, 2023. Retrieved May 17, 2023.
 Grow, Kory (August 3, 2023). "Taylor Swift Is Helping Truck Drivers Buy First Homes With $100,000 Bonuses". Rolling Stone. Retrieved August 3, 2023.
 Paulson, Dave (March 5, 2020). "Taylor Swift donates $1 million for Tennessee tornado relief". The Tennessean. Retrieved March 5, 2020.
 Ruggieri, Melissa. "Taylor Swift donates $1 million to help communities ravaged by Tennessee tornadoes". The Tennessean. Retrieved December 11, 2023.
 "Nashville Songwriters Hall of Fame". Nashville Songwriters Foundation. Retrieved July 2, 2020.
 "Taylor Swift becomes namesake of Hendersonville High School auditorium". The Tennessean. September 23, 2010. Archived from the original on July 12, 2015. Retrieved May 29, 2011.
 McKinley, James C. (May 17, 2012). "Taylor Swift to Donate $4 Million to the Country Hall of Fame". The New York Times. Archived from the original on June 5, 2012. Retrieved July 1, 2012.
 Golden, Zara (October 1, 2012). "And The Taylor Swift VH1 Storytellers Contest Winner Is ... Harvey Mudd College!". VH1. Archived from the original on July 12, 2015. Retrieved April 21, 2014.
 "$100,000 Gift from Taylor Swift Provides Vital Support to Nashville Symphony". Nashville Symphony. December 16, 2013. Retrieved April 7, 2020.
 Sanz, Cynthia (December 14, 2009). "Taylor Swift Gives Big as She Turns 20 – Good Deeds, Taylor Swift". People. Archived from the original on June 17, 2012. Retrieved May 29, 2012.
Urban, Mike (October 14, 2011). "Taylor Swift donates 6,000 books to Reading Library". Reading Eagle. Archived from the original on May 12, 2012. Retrieved May 29, 2012.
Zhu, Danielle (November 17, 2015). "Taylor Swift partners with Scholastic to donate 25,000 books". Entertainment Weekly. Archived from the original on August 14, 2016. Retrieved August 3, 2016.
 Cooper, Brittany Joy (February 2, 2012). "Taylor Swift Donates 14,000 Books to Nashville Public Library". Taste of Country. Archived from the original on June 17, 2012. Retrieved May 29, 2012.
 "Taylor Swift Helps With 'Delete Online Predators' Campaign". Country Weekly. September 18, 2007. Archived from the original on January 16, 2016. Retrieved November 29, 2015.
 "Taylor Swift Teams Up With UNICEF Tap Project Initiative". Taste of Country. March 17, 2011. Archived from the original on May 1, 2013. Retrieved August 7, 2012.
"Taylor Swift Among Participants in MusiCares Back To School Charity Auction". RTTNews. July 28, 2011. Archived from the original on May 17, 2013. Retrieved August 7, 2012.
 "Taylor Swift and ACM Lifting Lives present $50,000 donation to St. Jude". The Tennessean. June 13, 2011. Archived from the original on July 12, 2015. Retrieved May 29, 2012.
 "New and Hot Video: Taylor Swift Debuts 'Ronan' at Stand Up To Cancer Benefit". Rolling Stone. September 8, 2012. Archived from the original on September 13, 2012. Retrieved September 14, 2012.
 Knight, Joey (May 17, 2014). "Dick Vitale holds annual fundraiser for pediatric cancer". Tampa Bay Times. Archived from the original on July 14, 2014. Retrieved July 30, 2014.
 George, John. "Taylor Swift donates $50K to CHOP to help teens with cancer". The Business Journals (Blog). Archived from the original on July 12, 2015. Retrieved June 5, 2014.
 Hardiman, Tim (April 20, 2007). "Taylor Swift Encourages Teen Volunteers". CMT. Archived from the original on November 13, 2012. Retrieved July 1, 2012.
 Vagianos, Alanna (August 17, 2017). "This Is How Taylor Swift's Donations Will Impact Sexual Assault Survivors". HuffPost. Retrieved April 7, 2020.
 Peppin, Hayley (March 24, 2020). "Taylor Swift responded to the leaked 'Famous' phone call with Kanye West by urging fans to instead focus on coronavirus aid". Insider Inc. Retrieved May 13, 2020.
 Grow, Kory (April 1, 2020). "How Taylor Swift Is Helping a Small Nashville Record Store Survive". Rolling Stone. Retrieved August 1, 2023.
 Hissong, Samantha (August 20, 2020). "Taylor Swift Starts Frenzy at Indie Record Stores With Surprise Signed 'Folklore' CDs". Rolling Stone. Retrieved August 1, 2023.
 Havens, Lyndsey (April 18, 2020). "Taylor Swift Sings Heart-Rending 'Soon You'll Get Better' During 'One World' Concert". Billboard. Retrieved April 19, 2020.
 @RAINN (April 8, 2021). "In 2017, @taylorswift13 fearlessly shared her story of sexual assault in court. In honor of Sexual Assault Awareness & Prevention Month & #RAINNDay, Taylor Swift has made a generous gift to RAINN. Thank you for helping us ensure all survivors are believed and supported! https://t.co/oE7P1BVnNJ" (Tweet). Retrieved April 21, 2021 – via Twitter.
 "Taylor Swift's cash gift helps student take up degree". BBC News. August 20, 2020. Retrieved November 9, 2020.
 M Holloway, Sophie (December 14, 2023). "Taylor Swift and Selena Gomez among celebrities attending Gaza aid fundraiser". The Independent.
Cited literature
Fischer, Nancy; Seidman, Steven (2016). Introducing the New Sexuality Studies (3rd ed.). Routledge. ISBN 978-1-317-44918-8. OCLC 953030187.
Fogarty, Mary; Arnold, Gina (2021). "Are You Ready for It? Re-Evaluating Taylor Swift". Contemporary Music Review. 40 (1): 1–10. doi:10.1080/07494467.2021.1976586.
Fulford, Phyllis (2014). An Idiots Guide: Singing Second Edition. Penguin Publishing. ISBN 978-1-61564-622-7.
Hughes, Charles (2017). "Country Music and the Recording Industry". In Stimeling, Travis D. (ed.). The Oxford Handbook of Country Music. Oxford University Press. pp. 205–228. ISBN 978-0-19-024817-8.
Jepson, Louisa (2013). Taylor Swift. Simon & Schuster. ISBN 978-1-4711-3087-8.
McNutt, Myles (2020). "From 'Mine' to 'Ours': Gendered Hierarchies of Authorship and the Limits of Taylor Swift's Paratextual Feminism". Communication, Culture and Critique. 13 (1): 72–91. doi:10.1093/ccc/tcz042.
McGrath, John (2023). "The Return to Craft: Taylor Swift, Nostalgia, and Covid-19". Popular Music and Society. 46 (1): 70–84. doi:10.1080/03007766.2022.2156761.
Perone, James E. (2017). The Words and Music of Taylor Swift. The Praeger Singer-Songwriter Collection. ABC-Clio. ISBN 978-1-4408-5294-7.
Provenzano, Catherine (2018). "Auto-Tune, Labor, and the Pop-Music Voice". In Fink, Robert; Latour, Melinda; Wallmark, Zachary (eds.). The Relentless Pursuit of Tone: Timbre in Popular Music. Oxford University Press. pp. 159–182. ISBN 978-0-19-998522-7.
Sloan, Nate (2021). "Taylor Swift and the Work of Songwriting". Contemporary Music Review. Routledge. 40 (1): 11–26. doi:10.1080/07494467.2021.1945226. S2CID 237695045.
External links
Taylor Swift
at Wikipedia's sister projects
Media from Commons
News from Wikinews
Quotations from Wikiquote
Data from Wikidata
Official website Edit this at Wikidata
Taylor Swift at AllMusic Edit this at Wikidata
Taylor Swift discography at Discogs Edit this at Wikidata
Taylor Swift at IMDb Edit this at Wikidata
Taylor Swift discography at MusicBrainz Edit this at Wikidata
vte
Taylor Swift
vte
Taylor Swift songs
Awards for Taylor Swift
vte
IFPI Global Year-End Charts
vte
Billboard Year-End number one albums
vte
Billboard Year-End Top Artist
Portals:
 Biography
icon Pop music
flag Pennsylvania
flag United States
Authority control databases Edit this at Wikidata
Categories: Taylor Swift1989 birthsLiving people21st-century American actresses21st-century American philanthropists21st-century American songwriters21st-century American women guitarists21st-century American women pianists21st-century American women singers21st-century women philanthropistsActresses from Nashville, TennesseeActresses from PennsylvaniaAmerican acoustic guitaristsAmerican billionairesAmerican country banjoistsAmerican country guitaristsAmerican country pianistsAmerican country record producersAmerican country singer-songwritersAmerican feministsAmerican film actressesAmerican folk guitaristsAmerican folk musiciansAmerican folk singersAmerican mezzo-sopranosAmerican multi-instrumentalistsAmerican music video directorsAmerican people of English descentAmerican people of German descentAmerican people of Italian descentAmerican people of Scottish descentAmerican pop guitaristsAmerican pop pianistsAmerican synth-pop musiciansAmerican television actressesAmerican voice actressesAmerican women country singersAmerican women guitaristsAmerican women philanthropistsAmerican women pop singersAmerican women record producersAmerican women rock singersAmerican women singer-songwritersAmerican women songwritersBig Machine Records artistsBrit Award winnersChristians from TennesseeCountry musicians from PennsylvaniaCountry musicians from TennesseeCountry pop musiciansFemale billionairesFemale music video directorsFeminist musiciansFilm directors from PennsylvaniaFilm directors from TennesseeGrammy Award winnersGuitarists from PennsylvaniaGuitarists from TennesseeMTV Europe Music Award winnersMTV Video Music Award winnersNME Awards winnersPeople from Bucks County, PennsylvaniaPhilanthropists from PennsylvaniaPhilanthropists from TennesseePrimetime Emmy Award winnersRCA Records artistsRecord producers from PennsylvaniaRecord producers from TennesseeRepublic Records artistsSinger-songwriters from PennsylvaniaSinger-songwriters from TennesseeSingers from Nashville, TennesseeSony Music Publishing artistsSynth-pop singersTime Person of the YearUniversal Music Group artistsBaldi family
This page was last edited on 16 February 2024, at 11:33 (UTC).
Text is available under the Creative Commons Attribution-ShareAlike License 4.0; additional terms may apply. By using this site, you agree to the Terms of Use and Privacy Policy. Wikipedia® is a registered trademark of the Wikimedia Foundation, Inc., a non-profit organization.
Privacy policyAbout WikipediaDisclaimersContact WikipediaCode of ConductDevelopersStatisticsCookie statementMobile view
Wikimedia FoundationPowered by MediaWiki
'''

----
# 1. `BasicTokenizer`


---

In [3]:
import re
import unicodedata
from collections import defaultdict, Counter
from typing import List, Dict, Tuple, Optional
import tiktoken
import numpy as np

# %%
# =============================================================================
# STEP 1: BasicTokenizer Implementation
# =============================================================================

class BasicTokenizer:
    """
    Basic implementation of Byte Pair Encoding (BPE) tokenizer.
    
    This implements the core BPE algorithm:
    1. Start with all bytes (0-255) as initial vocabulary
    2. Iteratively find the most frequent pair of consecutive tokens
    3. Merge this pair into a new token
    4. Repeat until desired vocabulary size is reached
    """
    
    def __init__(self):
        self.merges = {}  # (int, int) -> int  Maps pairs to their merged token
        self.vocab = {}   # int -> bytes        Maps token IDs to their byte representation
        
    def train(self, text: str, vocab_size: int, verbose: bool = False):
        """
        Train the tokenizer on given text to create a vocabulary of specified size.
        
        The algorithm:
        1. Convert text to UTF-8 bytes
        2. Start with raw bytes as initial tokens
        3. Iteratively merge most frequent consecutive pairs
        4. Stop when we reach desired vocabulary size
        """
        assert vocab_size >= 256, "Vocab size must be at least 256 (for all bytes)"
        
        num_merges = vocab_size - 256
        
        # Convert text to UTF-8 bytes and treat each byte as initial token
        text_bytes = text.encode("utf-8")
        ids = list(text_bytes)  # Each byte becomes a token ID
        
        # Initialize vocabulary with all possible bytes (0-255)
        self.vocab = {idx: bytes([idx]) for idx in range(256)}
        
        # Perform BPE merges
        for i in range(num_merges):
            # Count frequency of all consecutive pairs
            stats = self._get_stats(ids)
            
            if not stats:
                break  # No more pairs to merge
                
            # Find the most frequent pair
            pair = max(stats, key=stats.get)
            idx = 256 + i  # New token ID starts after all byte tokens
            
            # Replace all occurrences of this pair with new token
            ids = self._merge(ids, pair, idx)
            
            # Update our data structures
            self.merges[pair] = idx
            self.vocab[idx] = self.vocab[pair[0]] + self.vocab[pair[1]]
            
            if verbose:
                print(f"Merge {i+1}/{num_merges}: {pair} -> {idx}")
                print(f"  Merged: {self.vocab[pair[0]]} + {self.vocab[pair[1]]} = {self.vocab[idx]}")
                print(f"  Frequency: {stats[pair]} occurrences")
                print()
    
    def _get_stats(self, ids: List[int]) -> Dict[Tuple[int, int], int]:
        """Count frequency of all consecutive pairs in the token sequence."""
        counts = defaultdict(int)
        for pair in zip(ids, ids[1:]):  # Sliding window of size 2
            counts[pair] += 1
        return counts
    
    def _merge(self, ids: List[int], pair: Tuple[int, int], idx: int) -> List[int]:
        """
        Replace all occurrences of 'pair' in 'ids' with new token 'idx'.
        
        Example: [5, 6, 7, 5, 6] with pair=(5,6) and idx=257
        Result:  [257, 7, 257]
        """
        newids = []
        i = 0
        while i < len(ids):
            # Check if current position matches our pair
            if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
                newids.append(idx)  # Replace pair with new token
                i += 2  # Skip both elements of the pair
            else:
                newids.append(ids[i])  # Keep current token
                i += 1
        return newids
    
    def encode(self, text: str) -> List[int]:
        """
        Encode text into token IDs using the learned BPE merges.
        
        Process:
        1. Convert text to bytes
        2. Apply all learned merges in order of priority
        3. Return final token sequence
        """
        text_bytes = text.encode("utf-8")
        ids = list(text_bytes)
        
        # Apply merges repeatedly until no more merges are possible
        while len(ids) >= 2:
            # Find the pair with the lowest merge priority (learned earliest)
            stats = self._get_stats(ids)
            pair = min(stats, key=lambda p: self.merges.get(p, float("inf")))
            
            if pair not in self.merges:
                break  # No more applicable merges
                
            # Apply this merge
            idx = self.merges[pair]
            ids = self._merge(ids, pair, idx)
        
        return ids
    
    def decode(self, ids: List[int]) -> str:
        """
        Decode token IDs back to text.
        
        Process:
        1. Look up byte representation for each token ID
        2. Concatenate all bytes
        3. Decode as UTF-8
        """
        text_bytes = b"".join(self.vocab[idx] for idx in ids)
        text = text_bytes.decode("utf-8", errors="replace")
        return text

In [4]:
tokenizer = BasicTokenizer()
text = "aaabdaaabac"
tokenizer.train(text, 256 + 3) # 256 are the byte tokens, then do 3 merges
print(tokenizer.encode(text))
# [258, 100, 258, 97, 99]
print(tokenizer.decode([258, 100, 258, 97, 99]))
# aaabdaaabac

[258, 100, 258, 97, 99]
aaabdaaabac


In [5]:
tokenizer.merges

{(97, 97): 256, (256, 97): 257, (257, 98): 258}

In [6]:
# %%
# Test BasicTokenizer with a simple example
print("=== Testing BasicTokenizer ===")

# Simple test text with obvious patterns
text = "aaabdaaabac"
print(f"Training text: '{text}'")

tokenizer = BasicTokenizer()
tokenizer.train(text, vocab_size=256+3, verbose=True)  # 276: 256 base + 20 merges

# Show learned vocabulary for new tokens
print("Learned merges:")
for pair, idx in tokenizer.merges.items():
    print(f"  {pair} -> {idx}: {tokenizer.vocab[idx]}")

print()

# Test encoding and decoding
test_texts = ["aaabdaaabac", "hello world", "abc"]
for test_text in test_texts:
    encoded = tokenizer.encode(test_text)
    decoded = tokenizer.decode(encoded)
    print(f"Text: '{test_text}'")
    print(f"Encoded: {encoded}")
    print(f"Decoded: '{decoded}'") 
    print(f"Match: {test_text == decoded}")
    print()

=== Testing BasicTokenizer ===
Training text: 'aaabdaaabac'
Merge 1/3: (97, 97) -> 256
  Merged: b'a' + b'a' = b'aa'
  Frequency: 4 occurrences

Merge 2/3: (256, 97) -> 257
  Merged: b'aa' + b'a' = b'aaa'
  Frequency: 2 occurrences

Merge 3/3: (257, 98) -> 258
  Merged: b'aaa' + b'b' = b'aaab'
  Frequency: 2 occurrences

Learned merges:
  (97, 97) -> 256: b'aa'
  (256, 97) -> 257: b'aaa'
  (257, 98) -> 258: b'aaab'

Text: 'aaabdaaabac'
Encoded: [258, 100, 258, 97, 99]
Decoded: 'aaabdaaabac'
Match: True

Text: 'hello world'
Encoded: [104, 101, 108, 108, 111, 32, 119, 111, 114, 108, 100]
Decoded: 'hello world'
Match: True

Text: 'abc'
Encoded: [97, 98, 99]
Decoded: 'abc'
Match: True



In [7]:
tokenizer.train(taylorswift_txt, vocab_size=4096)

# # Show learned vocabulary for new tokens
# print("Learned merges:")
# for pair, idx in tokenizer.merges.items():
#     print(f"  {pair} -> {idx}: {tokenizer.vocab[idx]}")

encoded = tokenizer.encode(taylorswift_txt)
decoded = tokenizer.decode(encoded)
print(f"Match: {taylorswift_txt == decoded}")

Match: False


In [8]:
tokens = taylorswift_txt.encode("utf-8") # raw bytes
tokens = list(map(int, tokens)) 
print("tokens length:", len(tokens))
print("ids length:", len(encoded))
print(f"compression ratio: {len(tokens) / len(encoded):.2f}X")

tokens length: 185768
ids length: 35895
compression ratio: 5.18X


----
# 2. `RegexTokenizer`


---

In [9]:
# %%
# =============================================================================
# STEP 2: RegexTokenizer Implementation  
# =============================================================================
import regex as re

class RegexTokenizer:
    """
    Regex-based tokenizer that splits text before applying BPE.
    
    This prevents merges from crossing semantic boundaries like:
    - Numbers and letters
    - Different types of punctuation
    - Whitespace patterns
    
    The GPT-4 pattern ensures tokens don't span across these categories.
    """
    
    def __init__(self, pattern: Optional[str] = None):
        """
        Initialize with a regex pattern for text splitting.
        If no pattern provided, treats entire text as one chunk.
        """
        self.pattern = pattern
        self.compiled_pattern = re.compile(self.pattern) if pattern else None
        self.merges = {}
        self.vocab = {}
        
    def train(self, text: str, vocab_size: int, verbose: bool = False):
        """Train tokenizer with regex pre-splitting."""
        assert vocab_size >= 256
        num_merges = vocab_size - 256
        
        # Step 1: Split text using regex pattern
        if self.compiled_pattern:
            text_chunks = re.findall(self.compiled_pattern, text)
            if verbose:
                print(f"Text split into {len(text_chunks)} chunks:")
                for i, chunk in enumerate(text_chunks[:10]):  # Show first 10
                    print(f"  Chunk {i}: '{chunk}'")
                if len(text_chunks) > 10:
                    print(f"  ... and {len(text_chunks) - 10} more")
                print()
        else:
            text_chunks = [text]
        
        # Step 2: Convert all chunks to token IDs
        ids = []
        for chunk in text_chunks:
            chunk_bytes = chunk.encode("utf-8")
            chunk_ids = list(chunk_bytes)
            ids.extend(chunk_ids)
        
        # Step 3: Initialize vocabulary with all bytes
        self.vocab = {idx: bytes([idx]) for idx in range(256)}
        
        # Step 4: Perform BPE merges
        for i in range(num_merges):
            stats = self._get_stats(ids)
            if not stats:
                break
                
            pair = max(stats, key=stats.get)
            idx = 256 + i
            
            ids = self._merge(ids, pair, idx)
            self.merges[pair] = idx
            self.vocab[idx] = self.vocab[pair[0]] + self.vocab[pair[1]]
            
            if verbose:
                print(f"Merge {i+1}/{num_merges}: {pair} -> {idx}")
                print(f"  {self.vocab[pair[0]]} + {self.vocab[pair[1]]} = {self.vocab[idx]}")
                print(f"  Frequency: {stats[pair]}")
    
    def _get_stats(self, ids: List[int]) -> Dict[Tuple[int, int], int]:
        counts = defaultdict(int)
        for pair in zip(ids, ids[1:]):
            counts[pair] += 1
        return counts
    
    def _merge(self, ids: List[int], pair: Tuple[int, int], idx: int) -> List[int]:
        newids = []
        i = 0
        while i < len(ids):
            if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
                newids.append(idx)
                i += 2
            else:
                newids.append(ids[i])
                i += 1
        return newids
    
    def encode(self, text: str) -> List[int]:
        """
        Encode text with regex splitting.
        Each chunk is processed independently, then results are concatenated.
        """
        # Split text using same pattern as training
        if self.compiled_pattern:
            text_chunks = re.findall(self.compiled_pattern, text)
        else:
            text_chunks = [text]
        
        ids = []
        for chunk in text_chunks:
            # Convert chunk to bytes
            chunk_bytes = chunk.encode("utf-8")
            chunk_ids = list(chunk_bytes)
            
            # Apply BPE merges to this chunk
            while len(chunk_ids) >= 2:
                stats = self._get_stats(chunk_ids)
                pair = min(stats, key=lambda p: self.merges.get(p, float("inf")))
                if pair not in self.merges:
                    break
                idx = self.merges[pair]
                chunk_ids = self._merge(chunk_ids, pair, idx)
            
            ids.extend(chunk_ids)
        
        return ids
    
    def decode(self, ids: List[int]) -> str:
        """Decode token IDs back to text."""
        text_bytes = b"".join(self.vocab[idx] for idx in ids)
        text = text_bytes.decode("utf-8", errors="replace")
        return text

In [10]:
# %%
# Test RegexTokenizer with GPT-4 pattern
print("=== Testing RegexTokenizer with GPT-4 Pattern ===")

# GPT-4's regex pattern - this is quite complex!
# GPT2_SPLIT_PATTERN = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
GPT4_SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""

# Test text with various types of content
text = """Hello world! This is a test sentence with numbers 123 and symbols @#$%. 
Let's see how it handles contractions like don't, won't, and I'll.
Also testing    multiple spaces and
newlines."""

print(f"Training text length: {len(text)} characters")

# Create and train regex tokenizer
regex_tokenizer = RegexTokenizer(GPT4_SPLIT_PATTERN)
regex_tokenizer.train(text, vocab_size=300, verbose=True)

print("\n" + "="*50)
print("COMPARISON: BasicTokenizer vs RegexTokenizer")
print("="*50)

# Train basic tokenizer on same text for comparison
basic_tokenizer = BasicTokenizer()
basic_tokenizer.train(text, vocab_size=300, verbose=False)

# Test both on various inputs
test_cases = [
    "Hello world!",
    "don't",
    "123abc", 
    "test   spaces",
    "símbolos especiais"
]

for test_text in test_cases:
    print(f"\nTest text: '{test_text}'")
    
    # Basic tokenizer
    basic_encoded = basic_tokenizer.encode(test_text)
    basic_decoded = basic_tokenizer.decode(basic_encoded)
    
    # Regex tokenizer  
    regex_encoded = regex_tokenizer.encode(test_text)
    regex_decoded = regex_tokenizer.decode(regex_encoded)
    
    print(f"Basic:    {basic_encoded} -> '{basic_decoded}'")
    print(f"Regex:    {regex_encoded} -> '{regex_decoded}'")
    print(f"Lengths:  Basic={len(basic_encoded)}, Regex={len(regex_encoded)}")


=== Testing RegexTokenizer with GPT-4 Pattern ===
Training text length: 185 characters
Text split into 43 chunks:
  Chunk 0: 'Hello'
  Chunk 1: ' world'
  Chunk 2: '!'
  Chunk 3: ' This'
  Chunk 4: ' is'
  Chunk 5: ' a'
  Chunk 6: ' test'
  Chunk 7: ' sentence'
  Chunk 8: ' with'
  Chunk 9: ' numbers'
  ... and 33 more

Merge 1/44: (115, 32) -> 256
  b's' + b' ' = b's '
  Frequency: 8
Merge 2/44: (101, 32) -> 257
  b'e' + b' ' = b'e '
  Frequency: 4
Merge 3/44: (97, 110) -> 258
  b'a' + b'n' = b'an'
  Frequency: 4
Merge 4/44: (258, 100) -> 259
  b'an' + b'd' = b'and'
  Frequency: 4
Merge 5/44: (111, 110) -> 260
  b'o' + b'n' = b'on'
  Frequency: 4
Merge 6/44: (116, 101) -> 261
  b't' + b'e' = b'te'
  Frequency: 3
Merge 7/44: (116, 105) -> 262
  b't' + b'i' = b'ti'
  Frequency: 3
Merge 8/44: (32, 32) -> 263
  b' ' + b' ' = b'  '
  Frequency: 3
Merge 9/44: (108, 108) -> 264
  b'l' + b'l' = b'll'
  Frequency: 2
Merge 10/44: (111, 32) -> 265
  b'o' + b' ' = b'o '
  Frequency: 2
Merge 11/44

In [11]:
regex_tokenizer.train(taylorswift_txt, vocab_size=4096)

# # Show learned vocabulary for new tokens
# print("Learned merges:")
# for pair, idx in tokenizer.merges.items():
#     print(f"  {pair} -> {idx}: {tokenizer.vocab[idx]}")

regex_enc = tokenizer.encode(taylorswift_txt)
regex_dec = tokenizer.decode(regex_enc)
print(f"Match: {taylorswift_txt == regex_enc}")

Match: False


In [12]:
tokens = taylorswift_txt.encode("utf-8") # raw bytes
tokens = list(map(int, tokens)) 
print("tokens length:", len(tokens))
print("ids length:", len(regex_enc))
print(f"compression ratio: {len(tokens) / len(regex_enc):.2f}X")

tokens length: 185768
ids length: 35895
compression ratio: 5.18X


In [13]:
# match this
import tiktoken
enc = tiktoken.get_encoding("cl100k_base") # this is the GPT-4 tokenizer
ids = enc.encode("hello world!!!? (안녕하세요!) lol123 😉")
text = enc.decode(ids) # get the same text back

----
# 3. `GPT4Tokenizer`


---

In [14]:
# %% 
# =============================================================================
# STEP 3: GPT-4 Compatible Tokenizer Implementation (FIXED)
# =============================================================================
def bpe(mergeable_ranks, token, max_rank):
    # helper function used in get_gpt4_merges() to reconstruct the merge forest
    parts = [bytes([b]) for b in token]
    while True:
        min_idx = None
        min_rank = None
        for i, pair in enumerate(zip(parts[:-1], parts[1:])):
            rank = mergeable_ranks.get(pair[0] + pair[1])
            if rank is not None and (min_rank is None or rank < min_rank):
                min_idx = i
                min_rank = rank
        if min_rank is None or (max_rank is not None and min_rank >= max_rank):
            break
        assert min_idx is not None
        parts = parts[:min_idx] + [parts[min_idx] + parts[min_idx + 1]] + parts[min_idx + 2:]
    return parts


def recover_merges(mergeable_ranks):
    # the `merges` are already the byte sequences in their merged state.
    # so we have to recover the original pairings. We can do this by doing
    # a small BPE training run on all the tokens, in their order.
    # also see https://github.com/openai/tiktoken/issues/60
    # also see https://github.com/karpathy/minbpe/issues/11#issuecomment-1950805306
    merges = {}
    for token, rank in mergeable_ranks.items():
        if len(token) == 1:
            continue # skip raw bytes
        pair = tuple(bpe(mergeable_ranks, token, max_rank=rank))
        assert len(pair) == 2
        # recover the integer ranks of the pair
        ix0 = mergeable_ranks[pair[0]]
        ix1 = mergeable_ranks[pair[1]]
        merges[(ix0, ix1)] = rank

    return merges
    

class GPT4Tokenizer2:
    """
    GPT-4 compatible tokenizer that matches tiktoken's cl100k_base encoding.
    
    This handles:
    1. Loading merges from GPT-4's actual vocabulary
    2. Handling the byte shuffle permutation
    3. Special tokens (optional)
    """
    
    def __init__(self):
        self.merges = {}
        self.vocab = {}
        self.inverse_vocab = {}
        self.special_tokens = {}
        self.inverse_special_tokens = {}  # Add reverse mapping for special tokens
        self.byte_shuffle = {}
        self.inverse_byte_shuffle = {}
        self.pattern = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""
        
        # Try to use regex module for Unicode support, fall back to re
        try:
            import regex as re_module
            self.compiled_pattern = re_module.compile(self.pattern)
        except ImportError:
            import re as re_module
            # Convert Unicode patterns for standard re module
            python_pattern = self.pattern.replace(r'\p{L}', r'[a-zA-ZÀ-ÿĀ-žА-я]').replace(r'\p{N}', r'[0-9]')
            self.compiled_pattern = re_module.compile(python_pattern)
        
    def load_gpt4_tokenizer(self):
        """Load the actual GPT-4 tokenizer parameters."""
        import tiktoken
        enc = tiktoken.get_encoding("cl100k_base")
        
        # Get the mergeable ranks (vocab)
        mergeable_ranks = enc._mergeable_ranks
        
        # Recover the byte shuffle from the first 256 entries
        self.byte_shuffle = {i: mergeable_ranks[bytes([i])] for i in range(256)}
        self.inverse_byte_shuffle = {v: k for k, v in self.byte_shuffle.items()}
        
        # Recover merges
        self.merges = recover_merges(mergeable_ranks)
        
        # Build vocabulary
        self.vocab = {rank: token for token, rank in mergeable_ranks.items()}
        self.inverse_vocab = mergeable_ranks
        
        # Get special tokens and create reverse mapping
        self.special_tokens = enc._special_tokens
        self.inverse_special_tokens = {v: k for k, v in self.special_tokens.items()}
        
        print(f"Loaded GPT-4 tokenizer with {len(self.vocab)} tokens")
        print(f"Found {len(self.merges)} merges")
        print(f"Special tokens: {list(self.special_tokens.keys())[:5]}...")
        
    def _apply_byte_shuffle(self, byte_ids):
        """Apply GPT-4's byte shuffle to raw bytes."""
        return [self.byte_shuffle[byte_id] for byte_id in byte_ids]
    
    def _reverse_byte_shuffle(self, shuffled_ids):
        """Reverse GPT-4's byte shuffle."""
        return [self.inverse_byte_shuffle[token_id] for token_id in shuffled_ids if token_id in self.inverse_byte_shuffle]
    
    def encode(self, text, allowed_special=None):
        """
        Encode text to token IDs, matching GPT-4's behavior exactly.
        
        Args:
            text: Input text to encode
            allowed_special: How to handle special tokens ("all", "none", or set of allowed tokens)
        """
        # Handle special tokens if present
        if allowed_special == "all":
            # Replace special tokens with placeholders temporarily
            special_token_map = {}
            for special_token in self.special_tokens:
                if special_token in text:
                    placeholder = f"__SPECIAL_{len(special_token_map)}__"
                    special_token_map[placeholder] = special_token
                    text = text.replace(special_token, placeholder)
        
        # Split text using GPT-4's regex pattern
        text_chunks = self.compiled_pattern.findall(text)
        
        ids = []
        for chunk in text_chunks:
            # Check if this chunk is a special token placeholder
            if allowed_special == "all" and chunk in special_token_map:
                special_token = special_token_map[chunk]
                ids.append(self.special_tokens[special_token])
                continue
                
            # Convert to bytes and apply shuffle
            chunk_bytes = chunk.encode("utf-8")
            chunk_ids = list(chunk_bytes)
            chunk_ids = self._apply_byte_shuffle(chunk_ids)
            
            # Apply BPE merges
            while len(chunk_ids) >= 2:
                # Find the pair with the lowest merge rank (earliest merge)
                stats = self._get_stats(chunk_ids)
                if not stats:
                    break
                    
                pair = min(stats, key=lambda p: self.merges.get(p, float("inf")))
                if pair not in self.merges:
                    break
                    
                # Apply this merge
                new_id = self.merges[pair]
                chunk_ids = self._merge(chunk_ids, pair, new_id)
            
            ids.extend(chunk_ids)
        
        return ids
    
    def decode(self, ids):
        """Decode token IDs back to text."""
        # Process tokens and handle special tokens properly
        byte_sequences = []
        result_parts = []
        
        for token_id in ids:
            # Check if this is a special token first
            if token_id in self.inverse_special_tokens:
                # If we have accumulated byte sequences, decode them first
                if byte_sequences:
                    combined_bytes = b"".join(byte_sequences)
                    result_parts.append(combined_bytes.decode("utf-8", errors="replace"))
                    byte_sequences = []
                
                # Add the special token as text
                special_token = self.inverse_special_tokens[token_id]
                result_parts.append(special_token)
                
            elif token_id in self.vocab:
                # Regular token - add to byte sequences
                token_bytes = self.vocab[token_id]
                byte_sequences.append(token_bytes)
            else:
                # Unknown token - skip or handle as needed
                print(f"Warning: Unknown token ID {token_id}")
        
        # Process any remaining byte sequences
        if byte_sequences:
            combined_bytes = b"".join(byte_sequences)
            result_parts.append(combined_bytes.decode("utf-8", errors="replace"))
        
        return "".join(result_parts)
    
    def _get_stats(self, ids):
        """Count frequency of consecutive pairs."""
        from collections import defaultdict
        counts = defaultdict(int)
        for pair in zip(ids, ids[1:]):
            counts[pair] += 1
        return counts
    
    def _merge(self, ids, pair, idx):
        """Replace all occurrences of pair with new token idx."""
        newids = []
        i = 0
        while i < len(ids):
            if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
                newids.append(idx)
                i += 2
            else:
                newids.append(ids[i])
                i += 1
        return newids


# # %% 
# # =============================================================================
# # STEP 3: GPT-4 Compatible Tokenizer Implementation
# # =============================================================================

# def recover_merges(mergeable_ranks):
#     """
#     Recover the BPE merges from GPT-4's mergeable_ranks.
    
#     This function reconstructs the merge operations by working backwards
#     from the final vocabulary to determine which pairs were merged at each step.
#     """
#     merges = {}
    
#     # Create a mapping from token to its rank
#     vocab_to_rank = mergeable_ranks
#     rank_to_vocab = {rank: token for token, rank in vocab_to_rank.items()}
    
#     # Process tokens in order of their rank (merge order)
#     for rank in sorted(rank_to_vocab.keys()):
#         if rank < 256:
#             continue  # Skip base bytes
            
#         token = rank_to_vocab[rank]
        
#         # Try to find the best split of this token into two parts
#         # that are both in our vocabulary
#         best_split = None
#         best_rank_sum = float('inf')
        
#         for i in range(1, len(token)):
#             left_part = token[:i]
#             right_part = token[i:]
            
#             if left_part in vocab_to_rank and right_part in vocab_to_rank:
#                 left_rank = vocab_to_rank[left_part]
#                 right_rank = vocab_to_rank[right_part]
                
#                 # We want the split where both parts have the lowest ranks
#                 # (were merged earliest)
#                 rank_sum = max(left_rank, right_rank)
#                 if rank_sum < best_rank_sum:
#                     best_rank_sum = rank_sum
#                     best_split = (left_part, right_part)
        
#         if best_split:
#             left_token, right_token = best_split
#             left_id = vocab_to_rank[left_token]
#             right_id = vocab_to_rank[right_token]
#             merges[(left_id, right_id)] = rank
    
#     return merges 
    
class GPT4Tokenizer:
    """
    GPT-4 compatible tokenizer that matches tiktoken's cl100k_base encoding.
    
    This handles:
    1. Loading merges from GPT-4's actual vocabulary
    2. Handling the byte shuffle permutation
    3. Special tokens (optional)
    """
    
    def __init__(self):
        self.merges = {}
        self.vocab = {}
        self.inverse_vocab = {}
        self.special_tokens = {}
        self.byte_shuffle = {}
        self.inverse_byte_shuffle = {}
        self.pattern = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""
        self.compiled_pattern = re.compile(self.pattern)
        
    def load_gpt4_tokenizer(self):
        """Load the actual GPT-4 tokenizer parameters."""
        enc = tiktoken.get_encoding("cl100k_base")
        
        # Get the mergeable ranks (vocab)
        mergeable_ranks = enc._mergeable_ranks
        
        # Recover the byte shuffle from the first 256 entries
        self.byte_shuffle = {i: mergeable_ranks[bytes([i])] for i in range(256)}
        self.inverse_byte_shuffle = {v: k for k, v in self.byte_shuffle.items()}
        
        # Recover merges
        self.merges = recover_merges(mergeable_ranks)
        
        # Build vocabulary
        self.vocab = {rank: token for token, rank in mergeable_ranks.items()}
        self.inverse_vocab = mergeable_ranks
        
        # Get special tokens
        self.special_tokens = enc._special_tokens
        
        print(f"Loaded GPT-4 tokenizer with {len(self.vocab)} tokens")
        print(f"Found {len(self.merges)} merges")
        print(f"Special tokens: {list(self.special_tokens.keys())[:5]}...")
        
    def _apply_byte_shuffle(self, byte_ids: List[int]) -> List[int]:
        """Apply GPT-4's byte shuffle to raw bytes."""
        return [self.byte_shuffle[byte_id] for byte_id in byte_ids]
    
    def _reverse_byte_shuffle(self, shuffled_ids: List[int]) -> List[int]:
        """Reverse GPT-4's byte shuffle."""
        return [self.inverse_byte_shuffle[token_id] for token_id in shuffled_ids if token_id in self.inverse_byte_shuffle]
    
    def encode(self, text: str, allowed_special: Optional[str] = None) -> List[int]:
        """
        Encode text to token IDs, matching GPT-4's behavior exactly.
        
        Args:
            text: Input text to encode
            allowed_special: How to handle special tokens ("all", "none", or set of allowed tokens)
        """
        # Handle special tokens if present
        if allowed_special == "all":
            # Replace special tokens with placeholders temporarily
            special_token_map = {}
            for special_token in self.special_tokens:
                if special_token in text:
                    placeholder = f"__SPECIAL_{len(special_token_map)}__"
                    special_token_map[placeholder] = special_token
                    text = text.replace(special_token, placeholder)
        
        # Split text using GPT-4's regex pattern
        text_chunks = re.findall(self.compiled_pattern, text)
        
        ids = []
        for chunk in text_chunks:
            # Check if this chunk is a special token placeholder
            if allowed_special == "all" and chunk in special_token_map:
                special_token = special_token_map[chunk]
                ids.append(self.special_tokens[special_token])
                continue
                
            # Convert to bytes and apply shuffle
            chunk_bytes = chunk.encode("utf-8")
            chunk_ids = list(chunk_bytes)
            chunk_ids = self._apply_byte_shuffle(chunk_ids)
            
            # Apply BPE merges
            while len(chunk_ids) >= 2:
                # Find the pair with the lowest merge rank (earliest merge)
                stats = self._get_stats(chunk_ids)
                if not stats:
                    break
                    
                pair = min(stats, key=lambda p: self.merges.get(p, float("inf")))
                if pair not in self.merges:
                    break
                    
                # Apply this merge
                new_id = self.merges[pair]
                chunk_ids = self._merge(chunk_ids, pair, new_id)
            
            ids.extend(chunk_ids)
        
        return ids
    
    def decode(self, ids: List[int]) -> str:
        """Decode token IDs back to text."""
        # Handle special tokens
        text_parts = []
        for token_id in ids:
            if token_id in self.vocab:
                token_bytes = self.vocab[token_id]
                
                # Check if this is a special token
                is_special = False
                for special_token, special_id in self.special_tokens.items():
                    if token_id == special_id:
                        text_parts.append(special_token)
                        is_special = True
                        break
                
                if not is_special:
                    text_parts.append(token_bytes)
        
        # Concatenate all byte sequences
        if text_parts:
            # Separate special tokens from byte sequences
            result_parts = []
            current_bytes = b""
            
            for part in text_parts:
                if isinstance(part, str):  # Special token
                    if current_bytes:
                        result_parts.append(current_bytes.decode("utf-8", errors="replace"))
                        current_bytes = b""
                    result_parts.append(part)
                else:  # Byte sequence
                    current_bytes += part
            
            if current_bytes:
                result_parts.append(current_bytes.decode("utf-8", errors="replace"))
            
            return "".join(result_parts)
        
        return ""
    
    def _get_stats(self, ids: List[int]) -> Dict[Tuple[int, int], int]:
        """Count frequency of consecutive pairs."""
        counts = defaultdict(int)
        for pair in zip(ids, ids[1:]):
            counts[pair] += 1
        return counts
    
    def _merge(self, ids: List[int], pair: Tuple[int, int], idx: int) -> List[int]:
        """Replace all occurrences of pair with new token idx."""
        newids = []
        i = 0
        while i < len(ids):
            if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
                newids.append(idx)
                i += 2
            else:
                newids.append(ids[i])
                i += 1
        return newids

In [15]:
test_text = "Hello world!!!? (안녕하세요!) lol123 😉"


print("\n" + "="*50)
print("STEP 3: GPT-4 COMPATIBILITY TEST")
print("="*50)

# Load reference tiktoken encoder
tiktoken_enc = tiktoken.get_encoding("cl100k_base")
tiktoken_ids = tiktoken_enc.encode(test_text)
tiktoken_decoded = tiktoken_enc.decode(tiktoken_ids)

print(f"Test text: '{test_text}'")
print(f"tiktoken encoding: {tiktoken_ids}")
print(f"tiktoken decoded: '{tiktoken_decoded}'")

# Test our GPT-4 tokenizer
gpt4_tokenizer = GPT4Tokenizer()
gpt4_tokenizer.load_gpt4_tokenizer()

our_ids = gpt4_tokenizer.encode(test_text)
our_decoded = gpt4_tokenizer.decode(our_ids)

print(f"Our encoding:     {our_ids}")
print(f"Our decoded:      '{our_decoded}'")
print(f"Encoding match:   {tiktoken_ids == our_ids}")
print(f"Decoding match:   {tiktoken_decoded == our_decoded}")
        


STEP 3: GPT-4 COMPATIBILITY TEST
Test text: 'Hello world!!!? (안녕하세요!) lol123 😉'
tiktoken encoding: [9906, 1917, 12340, 30, 320, 31495, 230, 75265, 243, 92245, 16715, 28509, 4513, 57037]
tiktoken decoded: 'Hello world!!!? (안녕하세요!) lol123 😉'
Loaded GPT-4 tokenizer with 100256 tokens
Found 100000 merges
Special tokens: ['<|endoftext|>', '<|fim_prefix|>', '<|fim_middle|>', '<|fim_suffix|>', '<|endofprompt|>']...
Our encoding:     [9906, 1917, 12340, 30, 320, 31495, 230, 75265, 243, 92245, 16715, 28509, 4513, 57037]
Our decoded:      'Hello world!!!? (안녕하세요!) lol123 😉'
Encoding match:   True
Decoding match:   True


----
# 4. `EnhancedGPT4Tokenizer`


---

In [16]:
# %%
# =============================================================================
# STEP 4: Enhanced Tokenizer with Special Token Support
# =============================================================================

class EnhancedGPT4Tokenizer(GPT4Tokenizer):
    """
    Enhanced GPT-4 tokenizer with full special token support.
    """
    
    def __init__(self):
        super().__init__()
        self.allowed_special_cache = {}
    
    def encode(self, text: str, allowed_special: Optional[str] = None, disallowed_special: Optional[set[str]] = None) -> List[int]:
        """
        Enhanced encoding with comprehensive special token handling.
        
        Args:
            text: Input text
            allowed_special: "all", "none", or set of allowed special tokens
            disallowed_special: Set of special tokens that should raise an error
        """
        if allowed_special is None:
            allowed_special = "none"
        
        # Check for disallowed special tokens
        if disallowed_special:
            for special_token in disallowed_special:
                if special_token in text:
                    raise ValueError(f"Disallowed special token '{special_token}' found in text")
        
        # Handle different allowed_special modes
        if allowed_special == "none":
            # Check if any special tokens are present and raise error
            for special_token in self.special_tokens:
                if special_token in text:
                    raise ValueError(f"Special token '{special_token}' found in text but not allowed")
        
        elif allowed_special == "all":
            # Allow all special tokens
            return self._encode_with_special_tokens(text, set(self.special_tokens.keys()))
        
        elif isinstance(allowed_special, (set, list)):
            # Allow only specified special tokens
            return self._encode_with_special_tokens(text, set(allowed_special))
        
        # Default encoding without special tokens
        return super().encode(text)
    
    def _encode_with_special_tokens(self, text: str, allowed_tokens: set[str]) -> List[int]:
        """Encode text while properly handling allowed special tokens."""
        # Split text around special tokens
        parts = []
        current_pos = 0
        
        while current_pos < len(text):
            # Find the earliest occurring allowed special token
            earliest_pos = len(text)
            earliest_token = None
            
            for special_token in allowed_tokens:
                pos = text.find(special_token, current_pos)
                if pos != -1 and pos < earliest_pos:
                    earliest_pos = pos
                    earliest_token = special_token
            
            if earliest_token:
                # Add text before the special token
                if earliest_pos > current_pos:
                    parts.append(('text', text[current_pos:earliest_pos]))
                
                # Add the special token
                parts.append(('special', earliest_token))
                current_pos = earliest_pos + len(earliest_token)
            else:
                # No more special tokens, add remaining text
                parts.append(('text', text[current_pos:]))
                break
        
        # Encode each part
        ids = []
        for part_type, content in parts:
            if part_type == 'special':
                ids.append(self.special_tokens[content])
            else:
                ids.extend(super().encode(content))
        
        return ids

In [17]:
# -------------------------------------------------------------------------
# Step 4: Test special tokens
# -------------------------------------------------------------------------
print("\n" + "="*50)
print("STEP 4: SPECIAL TOKENS TEST")
print("="*50)

special_test_text = "<|endoftext|>hello world"

# tiktoken with special tokens
tiktoken_special_ids = tiktoken_enc.encode(special_test_text, allowed_special="all")
tiktoken_special_decoded = tiktoken_enc.decode(tiktoken_special_ids)

print(f"Special test text: '{special_test_text}'")
print(f"tiktoken special encoding: {tiktoken_special_ids}")
print(f"tiktoken special decoded: '{tiktoken_special_decoded}'")

try:
    enhanced_tokenizer = EnhancedGPT4Tokenizer()
    enhanced_tokenizer.load_gpt4_tokenizer()
    
    our_special_ids = enhanced_tokenizer.encode(special_test_text, allowed_special="all")
    our_special_decoded = enhanced_tokenizer.decode(our_special_ids)
    
    print(f"Our special encoding:     {our_special_ids}")
    print(f"Our special decoded:      '{our_special_decoded}'")
    print(f"Special encoding match:   {tiktoken_special_ids == our_special_ids}")
    print(f"Special decoding match:   {tiktoken_special_decoded == our_special_decoded}")
    
except Exception as e:
    print(f"Special tokens test failed: {e}")


STEP 4: SPECIAL TOKENS TEST
Special test text: '<|endoftext|>hello world'
tiktoken special encoding: [100257, 15339, 1917]
tiktoken special decoded: '<|endoftext|>hello world'
Loaded GPT-4 tokenizer with 100256 tokens
Found 100000 merges
Special tokens: ['<|endoftext|>', '<|fim_prefix|>', '<|fim_middle|>', '<|fim_suffix|>', '<|endofprompt|>']...
Our special encoding:     [100257, 15339, 1917]
Our special decoded:      'hello world'
Special encoding match:   True
Special decoding match:   False


----
# 5. `UnicodeTokenizer`

---

In [18]:
# %%
# =============================================================================
# STEP 5: SentencePiece-style Unicode Tokenizer (Llama-style)
# =============================================================================

class UnicodeTokenizer:
    """
    SentencePiece-style tokenizer that operates on Unicode code points
    instead of UTF-8 bytes, similar to Llama 2's approach.
    """
    
    def __init__(self):
        self.merges = {}
        self.vocab = {}
        self.inverse_vocab = {}
        
    def train(self, text: str, vocab_size: int, verbose: bool = False):
        """
        Train tokenizer on Unicode code points instead of bytes.
        
        Key difference from BPE on bytes:
        - Each Unicode character becomes an initial token
        - Merges happen at the character level, not byte level
        """
        # Convert text to Unicode code points
        unicode_points = [ord(char) for char in text]
        
        # Get unique characters and assign them IDs
        unique_chars = sorted(set(unicode_points))
        char_to_id = {char: i for i, char in enumerate(unique_chars)}
        id_to_char = {i: char for char, i in char_to_id.items()}
        
        # Convert text to initial token IDs
        ids = [char_to_id[char] for char in unicode_points]
        
        # Initialize vocabulary with Unicode characters
        self.vocab = {i: chr(char) for i, char in id_to_char.items()}
        
        # Perform BPE merges
        num_merges = vocab_size - len(unique_chars)
        next_id = len(unique_chars)
        
        if verbose:
            print(f"Starting with {len(unique_chars)} unique characters")
            print(f"Will perform {num_merges} merges")
        
        for i in range(num_merges):
            # Count pairs
            stats = self._get_stats(ids)
            if not stats:
                break
            
            # Find most frequent pair
            pair = max(stats, key=stats.get)
            
            # Merge
            ids = self._merge(ids, pair, next_id)
            self.merges[pair] = next_id
            
            # Update vocabulary
            left_str = self.vocab[pair[0]]
            right_str = self.vocab[pair[1]]
            self.vocab[next_id] = left_str + right_str
            
            if verbose and i < 10:  # Show first 10 merges
                print(f"Merge {i+1}: {pair} -> {next_id}")
                print(f"  '{left_str}' + '{right_str}' = '{self.vocab[next_id]}'")
                print(f"  Frequency: {stats[pair]}")
            
            next_id += 1
        
        # Build inverse vocabulary
        self.inverse_vocab = {v: k for k, v in self.vocab.items()}
        
        if verbose:
            print(f"\nFinal vocabulary size: {len(self.vocab)}")
    
    def encode(self, text: str) -> List[int]:
        """Encode text using Unicode-based BPE."""
        # Convert to character-level tokens
        char_to_id = {}
        for token_id, token_str in self.vocab.items():
            if len(token_str) == 1:  # Single character
                char_to_id[ord(token_str)] = token_id
        
        # Convert text to initial token IDs
        ids = []
        for char in text:
            char_code = ord(char)
            if char_code in char_to_id:
                ids.append(char_to_id[char_code])
            else:
                # Handle unknown characters - could add UNK token or skip
                continue
        
        # Apply merges
        while len(ids) >= 2:
            stats = self._get_stats(ids)
            if not stats:
                break
            
            # Find the pair with lowest merge priority
            pair = min(stats, key=lambda p: self.merges.get(p, float("inf")))
            if pair not in self.merges:
                break
            
            merge_id = self.merges[pair]
            ids = self._merge(ids, pair, merge_id)
        
        return ids
    
    def decode(self, ids: List[int]) -> str:
        """Decode token IDs back to text."""
        return "".join(self.vocab.get(token_id, "�") for token_id in ids)
    
    def _get_stats(self, ids: List[int]) -> Dict[Tuple[int, int], int]:
        """Count consecutive pairs."""
        counts = defaultdict(int)
        for pair in zip(ids, ids[1:]):
            counts[pair] += 1
        return counts
    
    def _merge(self, ids: List[int], pair: Tuple[int, int], new_id: int) -> List[int]:
        """Replace all occurrences of pair with new_id."""
        newids = []
        i = 0
        while i < len(ids):
            if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
                newids.append(new_id)
                i += 2
            else:
                newids.append(ids[i])
                i += 1
        return newids

In [19]:
# -------------------------------------------------------------------------
# Step 5: Test Unicode tokenizer
# -------------------------------------------------------------------------
print("\n" + "="*50)
print("STEP 5: UNICODE TOKENIZER TEST (LLAMA-STYLE)")
print("="*50)

# Train a small Unicode tokenizer
unicode_test_text = "Hello world! 안녕하세요! 🌟"

unicode_tokenizer = UnicodeTokenizer()
unicode_tokenizer.train(unicode_test_text, vocab_size=100, verbose=True)

unicode_ids = unicode_tokenizer.encode("Hello! 안녕!")
unicode_decoded = unicode_tokenizer.decode(unicode_ids)

print(f"\nUnicode test:")
print(f"Text: 'Hello! 안녕!'")
print(f"Encoded: {unicode_ids}")
print(f"Decoded: '{unicode_decoded}'")
print(f"Round-trip match: {'Hello! 안녕!' == unicode_decoded}")

# Show some learned tokens
print(f"\nSample learned tokens:")
for i, (token_id, token_str) in enumerate(sorted(unicode_tokenizer.vocab.items())[:20]):
    print(f"  {token_id}: '{token_str}'")


STEP 5: UNICODE TOKENIZER TEST (LLAMA-STYLE)
Starting with 15 unique characters
Will perform 85 merges
Merge 1: (1, 0) -> 15
  '!' + ' ' = '! '
  Frequency: 2
Merge 2: (2, 4) -> 16
  'H' + 'e' = 'He'
  Frequency: 1
Merge 3: (16, 5) -> 17
  'He' + 'l' = 'Hel'
  Frequency: 1
Merge 4: (17, 5) -> 18
  'Hel' + 'l' = 'Hell'
  Frequency: 1
Merge 5: (18, 6) -> 19
  'Hell' + 'o' = 'Hello'
  Frequency: 1
Merge 6: (19, 0) -> 20
  'Hello' + ' ' = 'Hello '
  Frequency: 1
Merge 7: (20, 8) -> 21
  'Hello ' + 'w' = 'Hello w'
  Frequency: 1
Merge 8: (21, 6) -> 22
  'Hello w' + 'o' = 'Hello wo'
  Frequency: 1
Merge 9: (22, 7) -> 23
  'Hello wo' + 'r' = 'Hello wor'
  Frequency: 1
Merge 10: (23, 5) -> 24
  'Hello wor' + 'l' = 'Hello worl'
  Frequency: 1

Final vocabulary size: 34

Unicode test:
Text: 'Hello! 안녕!'
Encoded: [19, 15, 11, 9, 1]
Decoded: 'Hello! 안녕!'
Round-trip match: True

Sample learned tokens:
  0: ' '
  1: '!'
  2: 'H'
  3: 'd'
  4: 'e'
  5: 'l'
  6: 'o'
  7: 'r'
  8: 'w'
  9: '녕'
  10: '

In [20]:
unicode_tokenizer.train(taylorswift_txt, vocab_size=4096)

# # Show learned vocabulary for new tokens
# print("Learned merges:")
# for pair, idx in tokenizer.merges.items():
#     print(f"  {pair} -> {idx}: {tokenizer.vocab[idx]}")

unicode_enc = unicode_tokenizer.encode(taylorswift_txt)
unicode_dec = unicode_tokenizer.decode(unicode_enc)
print(f"Match: {taylorswift_txt == unicode_dec}")

Match: False
